<a href="https://colab.research.google.com/github/ReplacedSpace17/Indentifier-Pathogens-of-Plants-ModelLearningTransfers/blob/main/IdentificadorDePatogenos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importa los módulos necesarios.

In [ ]:
import Bio
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split



ModuleNotFoundError: ignored

# Carga el dataset de secuencias de ADN.

In [ ]:
import re
import json

# Ejemplo de entrada FASTA
fasta_input = """
>AF271224.1 Pythium vexans
CCACACCTAAAAACACCCTTCCACGTGAACCGTTTTGTTTTGCTTTCGAGTGCTTTGTTGCGCTCGGAGCATGTTTGGGCTTCGCTGCTGGCGCTTGATTGTGCTGGCGGCTCGAGGCCATCAAGCGGCGTTTTGAGTGTGCTTGCGCAATTGAAACGTCCAAACCTTTTAAACCCATTTGATTGCAAACTGAAGTATACTGTGGGGACGAAAGTCCTCGCTTTGAAACTAGATAACAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGGATTGCAGGATTCAGTGAGTCATCGAACTTTTGAACGCATATTGCACTTTCGGGTTACGCCTGGAAGTATGTCTGTATCAGTGTCCGTACACTAAACTTGCGTCTCTTCCGTCGTGTAGTCGTCGGTTGTTTAGATTGCAGATGTGAGGTTGTCTCGCAACGTGCCCCCCCTTGTTTTTTTGGGGGGTGGATAGGTTGCGAGTCCCTTTAAAGTCGGACGGTGTTTTTTTCCGTTTTGTGCTCGATGGGGTGCGGCTGCGGCCGTGTCTGCTGGCGGGTCCGGTGACCTTTGGCGATGGCATGAGAGTGGATTGCTCGATTTGCGGTATGTTAGGCTTCGGCTTTGACAATGCAGCTTATTGGGTGTGTTCGCTTGGCTGTTGCTGTATGGGGTGAGCTGGATGGTCGGTGGGTGCGTTGTTGCGTGTCGTTTTCATTGTGTGCGTTTCGGTTGTCGTCGCCATTTGGGAATTTAATGTTTTGGGGGACCCCTCAATACATCTCAT
>AF271225.1 Pythium ultimum
CCACACTTTAAAAAACTGTCCACGTGAACTGTAAGCAAGTCTAGCGCTGTGACTGAGCTGGTGTTTTCATTTTTGGACACTGGAACGGGAGTCAGCAGGACGAAGGTTGGTCTGTTGTAATGCAAGTTATGATGGACTAGCTGATGAACTTTTGTTTTTAAACCCTTACCTAAATACTGATTTATACTGTGGGGACGAAAGTCCTTGCTTTTACTAGATAACAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGAATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTTCGGGTTATGCCTGGAAGTATGTCTGTATCAGTGTCCGTAAATCAAACTTGCCTTTCTTTTTCTGTGTAGTCAGGGATGGAATGTGCAGATGTGAAGTGTCTCGCATGGTTGCGTTCGTTTTTTCGATCGAGAATCTGTCGAGTCCTTTTAAATGGACACGGTCTTTTCTATGGTTTCTATGAAGTGTAATGGTTGGAAGGCAGTGATTTTCGGATTGCTGGCGGCTTTTGGCGACTTCGGTATGAACGTATGGAGACTAGCTCAATTCGTGGTATGTTAGGCTTCGGCTCGACAATGTTGCGTAATTGTGTGTGGTCTTTGTTTGTGCCTTGAGGTGTACTAGAGGTTGTCGGTTTGAACCGTAAGTGATTGTTTAGTAGAGCATTTTCACGATGTATGGAGACGCTGCATTTAGTTGCGTAGAGAGATTGATTTGGGAAATTTTTTATCATTGTCAATTGCAAGATTGTGTATGGTATCTCAA
>AF271226.1 Pythium irregulare
CCACACCTAAAGAAACTTCCCACGTGAACTGTCGTTATTTGTTGTGTGTGTGCGTGTTGGTAGCATGCGTGTTTGCTTACGCTTTGGTGTTTGCGAGTGTGTGTGTTGTCGGTGCGCAGACTGAACGAAGGTCGTGTGTTGCTGTGTGCCTGCTGCACTGCTGACTTTGCATTGATTTGCATGGTGTTGGCGGAGCGGCGGGTGCTGTTGCATGCGCGGCTGACCTATTTTTTTCAAACCCCATACCTAAATGACTGATTATACTGTGAGAACGAAAGTTCTTGCTTTTAACTAGATAACAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGAATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTATGCCTGGAAGTATGTCTGTATCAGTGTCCGTAAATCAAACTTGCGTTTCTTCCTTCCGTGTATTCGGTGGAGGAGAGTTGCAGATGTGAAGTGTCTCGCTGTGGTTGGTGTTTGTTGTTTGCAATGAATGCACAGCTTGCGAGTCCTTTTAAATGGACACGACTTTCTCTTTTTTGTATGTGCGCGGTGCTGTGCGTGAACGCGGTGGTTTTCGGATCGCTCGCGGCTGTCGGCGACTTCGGTGAATGCATAATGGAGTGGACCTCGATTCGCGGTATGTTGGGCTTCGGCTGGACAATGTTGCTTATTGTGTGTCTGTTCCGTGTTCGCCTGAGGTGTACTGATGGCTGTGGGATTGAACTGGTTACTGTTGTTAGTAGTGTGTGTAGCACGTTGTCGTGGATGCATCTGTGTTTTTGCATACTTGTGTGTGTGCAATTGTACAGAAGAGGAGTCTCAATTTGGGAAAAGTTTTGTATACTCCGGGTTGATCCTGCGTGTATATCTCAA
>AF271227.1 Pythium aphanidermatum
CCACACCATAAAAACTTTCCACGTGAACCGTTGAAATCATGTTCTGTGCTCTCTTTCGGGAGGGCTGAACGAAGGTGGGCTGCTTAATTGTAGTCTGCCGATGTATTTTTCAAACCCATTTACCTAATACTGATCTATACTCCAAAAACGAAAGTTTATGGTTTTAATCTATAACAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGAATTCAGTGAGTCATCGAAATTTTGAACGCACATTGCACTTTCGGGTTATGCCTGGAAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGCCTTTCTTTTTCTGTGTAGTCAGGGAGAGAGATGGCAGAATGTGAGGTGTCTCGCTGGCTCCCTTTTCGGAGGAGAAGACGCGAGTCCCTTTAAATGTACGTTCGCTCTTTCTTGTGTCTAAGATGAAGTGTGATTCTCGAATCGCGGTGATCTGTTTGGATCGCTTTGCGCATTTGGGCGACTTCGGTTAGGACATTAAAGGAAGCAACCTCTATTGGCGGTATGTTAGGCTTCGGCCCGACGTTGCAGCTGACAGAGTGTGGTTTTCTGTTCTTTCCTTGAGGTGTACCTGAATTGTGTGAGGCAATGGTCTGGGCAAATGGTTGCTGTGTAGTAGGGTTTTGCTGCTCTTGGACGCCCTGTTTTCGGATAGGGTAAAGGAGGCAACACCAATTTGGGACTGTTTGCAATTTATTGTGAACAACTTTCTAA
>AF271228.1 Pythium dissotocum
CCACACCAAAAAAACTTTCCACGTGAACCGTTGTAACTATGTTCTGTGCTCTCTTCTCGGAGAGAGCTGAACGAAGGTGGGCTGCTTAATTGTAGTCTGCCGATGTACTTTTAAACCCATTAAACTAATACTGAACTATACTCCGAAAACGAAAGTCTTTGGTTTTAATCAATAACAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGAATTCAGTGAGTCATCGAAATTTTGAACGCACATTGCACTTTCGGGTTATGCCTGGAAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGCCTTTCTTTTTTTGTGTAGTCAAGAAGAGAGATGGCAGACTGTGAGGTGTCTCGCTGACTCCCTCTTCGGAGGAGAAGACGCGAGTCCCTTTAAATGTACGTTCGCTCTTTCTTGTGTTTAAAGATGAAGTGTGACTTTCGAACGCAGTGATCTGTTTGGATCGCTTTGCTCGAGTGGGCGACTTCGGTTAGGACATTAAAGGAAGCAACCTCTATTGGCGGTATGTTAGGCTTCGGCCCGACTTTGCAGCTGACTGGAGTTGTTTTCTGTTCTTTCCTTGAGGTGTACCTGTCTTGTGTGAGGCAATGGTCTGGGCAAATGGTTATTGTGTAGTAGGAAGTTGCTGCTCTTAAGCGCTCTAGCTTCGGTTAGAGTAAAGGAGGCAACACCAATTTGGGATAGTCGTTGATTTATCAATGGCGCTCTTTCTAA
>AF271229.1 Pythium graminicola
CCACACCATAAAACTTTCCACGTGAACCGTTACAATTATGTTCTGTGCTGTCTCTCGGGATGGCTGAACGAAGGTGGGCTGCATGTATGTGTAGTCTGCCGATGTACTTTTCAAACCCATTACTAAATACTGAACTATACTCCGAGAACGAAAGTTTTTGGGTTTAATCCATAACAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGAATTCAGTGAGTCATCGAAATTTGGAACGCACATTGAACTTCCGGGATATCCCTGGAAGTATGCTTGTTTCAGTGTCCGTACATCAAACTTGCCTTTCTTTTTTGGTGTAGCCAAGGAGAGAAATGGAAGAATGTGAGGCGTCTCGCTGGCTCCCCCTCCGGAGGAGAAAACGCGAGTCCCTTTAAATGTACGTCCGCCTTTCTTGTGTCGAAGTAGAAGTGTGACTATCGAACGCAGTGGTCTGTTTGGATCGCTTGGCGCGAGTTGGCGACTTCGGTTAGGACATTAAAGGAAGCAACCTCTATTGGCGGTATGTTAGGCTTCGGCCCGACTTTGCAGCTGACAGTGTGTAGTTTTCTGTTCTTTCCTTGAGGTGTACCTGTTTGTGTGAGGCAATGGTCTAGCCAAATGGTTATTGTGTAGTAGGTGGTTGCTGCTCTTTGGCGCCCTCTCGAGGGTAAAGGAGGCAACACCAATTTGGGATTAGTCTGTGGATTTATTCATGGGCGCTTTTCAAT
>AF271230.1 Pythium undulatum
CCACACCAAAAACTTTCCACGTGAACTGTCATTGCATGTTTTGTGCCTTTAATTAGGCTAAACGAAGGTCGGAGTAAAATCTGGCTGATCTATTTTTTAAACCCATTCTTAAACACTGATTTATACTGTGAGGACGAAAGTCTTTGCTTTTAACTAGATAACAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGAATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTTCGGGTTCTACCTGGAAGTATGTCTGTATCAGTGTCCGTACACTAAACTTGCCTTTCTTTTTCTGTGTAGTCAGAGGAGGAGATGGCCGATGTGAAGTGTCTCGATCTGTGCTCATTTTGTGCCACACTGATCGAGTCCTTTTAAATGTACTCTGTTTTGTCTCTTGTTTCTATGATGTACTGCTGGAACGGAGTGATCTTTGGATTGCTTGCATGTTTGGTGGCAACTTCGGTTTAGACGCTAGGAGATAATGCTCGATTCGCGGTATGTTAGGCTTCGGCTGGACAATGTTGCTTATTGTGCGTTGGCTCTGTTTTTACCTTGAGGTACCATTATTTGTGTGAGATGTGTCTAAGGATTGCTTGCAAGTAAGGTGTTGCAGCGTAGTGACGCTTTGGTGCTGTTTTTCGGAGCGGTGTTGAGGTTGAGTGAAGCACAATTTGGGAAAATTTTGTACACTGCTATTTTTGGTAATTGTGTATCTCAA
>AB217670.1 Phytophthora capsici
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCTTTTAGTTGGGGGTCTTGTACCCTATCATGGCGAATGTTTGGACTTCGGTCCGGGCGAGTAGCTTTTTGTTTTAAACCCATTTCACAATTCTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGATGTGCCAGATGTGAAGTGTCTTGCGTGTTGTCCTTCGGGTCGACTGCGAGTCCTTTTAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGTGTTGCTGGTTGTGGAGGCTGCCTGCGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAAAGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGGCGCTTATTGAATGCTTTTCCTGCTGTGGCGTGATGGGCTGGTGAACCGTAGCTGTGTTTGGCTTGGCTTTTGAATCGGTTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTTGGGAATTTTGTGTGCACTTCGGTGCGCATCTCAA
>AB217671.1 Phytophthora cactorum
CCACACCTAAAAACTTTCCACGTGAACCGTTTCAAACCAAATAGTTGGGGGTCTTGTCTGGTGGCGGCTGCTGGCTTTATTGTTGGCGGCTGCTGCTGGGTGAGCCCTATCATGGCGAGCGTTTGGGCTTCGGCCTGAGCTAGTAGCTTTTCTTTTAAACCCATTCCTTAATACTGATTATACTGTGGGGACGAAAGTCCTTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGAGATGCCAGATGTGAAGTGTCTTGCGGCTGGTTTTCGGACCGACTGCGAGTCCTTTTAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCTATTGTAGCAAGTTGGCGACCGGTTTGTCTGCTGCGGCGTTAATGGAAGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGATGTTTTTTCTGCTGTGGCGTGATGGACCGGTGAACCATAGCTCAGTTGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAAATGTGTGTGTACTTCGGTATGCATCTCAA
>AB217675.1 Phytophthora cinnamomi
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCAATTAGTTGGGGGCCTGCTCTGGGCGGCGGCTGTCGATGTCAAAGTCGACGGCTGCTGCTGCGTGGCGGGCCCTATCACTGGCGAGCGTTTGGGTCCCTCTCGGGGGAACTGAGCTAGTAGCCTCTCTTTTAAACCCATTCTGTAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCTCTTCCTTCCGTGTAGTCGGTGGATGGAGGTGCCAGACGTGAGGTGTCTTGCGGGCGGTCTTCGGACTGGCTGTGAGTCCCTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGCTGGTTGTGGAGGCTGCCTGTATGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAAAGCGCTTATTGGATGTTCTTCCTGCTGTGGCGGTACGGATCGGTGAACCGTAGCTGTGCTAGGCTTGGCGTTTGAACCGGCGGTGTTGTTGCGAAGTAAGGTGGCGGCTTCAGCTGTCGAGGGTCGATCCATTTGGGAACTCTGTGTCTCTGCAGGCACTTGTGTGCTTGTGGTGGCATCTCAA
>AB217682.1 Phytophthora nicotianae
CCACACCTAAAACTTTCCACGTGAACCGTTTCACCCAATAGTTGGGGGTCTTATTTGGCGGCGGCTGCTGGCTTAATTGTTGGCGGCTGCTGCTGAGTGAGCCCTATCAAAAAAAAGGCGAACGTTTGGGCTTCGGCATGATTTAGTAGTCTTTTTTCTTTAAACCCATTCGTTAATACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGAAGTATGCCTGTATCAGTGTCCGTACATTAAACTTGACTTTCTTCCTTCCGTGTAGTCGGTGGAGGAGATGTCAGATGTGAAGTGTCTTGCGATTGGTCTTCGGACCGGCTGCGAGTCCTTTTAAATGTACTAAACTGAACTTCTCTTTGCTCGAAAAGTGGTGGCGTTGCTGGTTGTGAAGGCTGCTATTGTGGCAAATTGGCGACTGGTTTGTCTGCTGCGGCATTAATGGAAGAGTGTTCGATTCGTGGTATGGTTGGCTTCGGCTGAACAATGCACTTATTGGACGTTTTTCCTGCTGTGGCGTGATGGACTGGTGAACCATAGCTCGGTGGCTTGGCTTTTGAATTGGCTTTGCTGTTGCGAAGTAGGGTGGCAGCTTCGGTTGTCGAGGGTCGATCCATTTGGGAACTTAATGTGTACTTCGGTATGCATCTCAA
>AB217684.1 Phytophthora sojae
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACAAGTAGTTGGGGGCCTGCTCTGTGTGGCTGTCTGTCGATGTCAAAGTCGGCGGCTGGCTGCTGTGTGGCGGGCTCTATCATGGCGATTGGTTTGGGTCCTCCTCGTGGGGAACTGGATCATGAGCCCACTTTTTAAACCCATTCTTAAATACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCTCTTCCTTCCGTGTAGTCGGTGGATGGAGACGCCAGACGTGAGGTGTCTTGCGGCGTGGCCTTCGGGCTGCCTGCGAGTCCCTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGTTGGTTGTGGAGGCTGCCTGTATGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGATGCTTTTCCTGCTGTGGCGGTATGGGCTGGTGAACCGTAGCTGTGTGAGGCTTGGCTTTTGAACCGGCGGTGTTGTTGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAACTCTGTGTTGTCTCTGCGGCTTGCTGCGGAGGTGGCATCTCAA
>AB217687.1 Phytophthora vignae
CCACACCTAAAAAACTTTCCACGTGAACCGTTTCAACAAGTAGTTGGGGGCCTGCTCTGTGTGGCTAGCTGTCGATGTCAAAGTCGGCGACTGGCTGCTATGTGACGGGCTCTATCATGGCAATTGGTTTGGGTCCTCCTCGTGGGGAACTAGATCATGAGCCCACTTTTTAAACCCATTCTTGATTACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCTCTTCCTTCCGTGTAGTCGGTGGATGGAGACGCCAGACGTGAGGTGTCTTGCGGCGCGGCCTTCGGGCTGCCTGCGAGTCCCTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGTTGGTTGTGGAGGCTGCCTGTATGGCCAGTCGGCGACCAGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGATGCTTTTCCTGCTGTGGCGGTATGGGCTGGTGAACCGTAGTTGTGCAAGGCTTGGCTTTTGAACCGGTGGTGTTGTTGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAACTTTGTGTTGTCTCTGCGGCTTGCTGTGGAGGTAGCATCTCAA
>AF139367.1 Phytophthora alni subsp. uniformis
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCACTTAGTTGGGGGCTAGTCCCGGCGGCTGGCTGTCGATGTCAAAGTTGACGGCTGCTGCTGTGTGTCGGGCCCTATCATGGCGAGCGTTTGGGTCCCTCTCGGGGGAACTGAGCCAGTAGCCCTTTTCTTTTAAACCCATTCTTGAATACTGAATATACTGYGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCTCTTCCTTCCGTGTAGTCGGTGGATGGRGACGCCAGACGTGAGGTGTCTTGCGGGTGGTCTTCGGGCTGCCTGCGAGTCCCTTGAAATGTACTGAACTGTACTTCTCTTTGCTYGAAAAGCGTGACGTTGTTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGACGTTCTTCCTGCTGTGGCGGTACGGATCGGTGAACCGTAGCTGTGCGGAGCTTGGCCTTTGAACCGGCGGTGTTGGTCGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTGGGAACTTGTGTCTCTGCGGCGCGCTTCGGTGTGCTGCGGGTGGCATCTCAA
>AF139369.1 Phytophthora cambivora
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCACTTAGTTGGGGGCTAGTCCCGGCGGCTGGCTGTCGATGTCAAAGTTGACGGCTGCTGCTGTGTGTCGGGCCCTATCATGGCGAGCGTTTGGGTCCCTCTCGGGGGAACTGAGCCAGTAGCCCTTATTTTTTAAACCCATTCTTGAATACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCTCTTCCTTCCGTGTAGTCGGTGGATGGGGACGCCAGACGTGAGGTGTCTTGCGGGTGGTCTTCGGGCTGCCTGCGAGTCCCTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGTTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGACGTTCTTCCTGCTGTGGCGGTACGGATCGGTGAACCGTAGCTGTGCGRRGCTTGGCCTTTGAACCGGCGGTGTTGGTCGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTGGGAACTTGTGTCTCTGCGGCGCGCTTCGGTGTGCTGCGGGTGGCATCTCAA
>AF266761 Phytophthora rubi
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCACTTAGTTGGGGGCCTGTCCTGGCGGCTGGCTGTCGATGTCAAAGTTGACGGCTGCTGCTGTGTGTCGGGCCCTATCATGGCGAGCGTTTGGGTCCCTCTCGGGGGAACTGAGCCAGTAGCCCTTTTCTTTTAAACCCATTCTTGAATACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCTCTTCCTTCCGTGTAGTCGGTGGATGGGGACGCCAGACGTGAGGTGTCTTGCGGGTGGCCTTCGGGCTGCCTGCGAGTCCCTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGTTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGACGTTCTTCCTGCTGTGGCGGTACGGATCGGTGAACCGTAGCTGTGCGAGGCTTGGCCTTTGAACCGGCGGTGTTGGTCGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTGGGAACTTGTGTCTTCTGCGGCGCGCTTCGGTGTGCTGCGGGTGGCATCTCAA
>AF266762 Phytophthora fragariae
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCACTTAGTTGGGGGCCTGTCCTGGCGGCTGGCTGTCGATGTCAAAGTTGACGGCTGCTGCTGTGTGTCGGGCCCTATCATGGCGAGCGTTTGGGTCCCTCTCGGGGGAACTGAGCCAGTAGCCCTTTTCTTTTAAACCCATTCTTGAATACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCTCTTCCTTCCGTGTAGTCGGTGGATGGGGACGCCAGACGTGAGGTGTCTTGCGGGTGGCCTTCGGGCTGCCTGCGAGTCCCTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGTTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGACGTTCTTCCTGCTGTGGCGGTACGGATCGGTGAACCGTAGCTGTGCGAGGCTTGGCCTTTGAACCGGCGGTGTTGGTCGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTGGGAACTTGTGTCTCTGCGGCGCGCTTCGGTGTGCTGCGGGTGGCATCTCAA
>AF266763 Phytophthora cambivora
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCACTTAGTTGGGGGCTAGTCCCGGCGGCTGGCTGTCGATGTCAAAGTTGACGGCTGCTGCTGTGTGTCGGGCCCTATCATGGCGAGCGTTTGGGTCCCTCTCGGGGGAACTGAGCCAGTAGCCCTTATTTTTTAAACCCATTCTTGAATACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCTCTTCCTTCCGTGTAGTCGGTGGATGGGGACGCCAGACGTGAGGTGTCTTGCGGGTGGTCTTCGGGCTGCCTGCGAGTCCCTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGTTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGACGTTCTTCCTGCTGTGGCGGTACGGATCGGTGAACCGTAGCTGTGCGAAGCTTGGCCTTTGAACCGGCGGTGTTGGTCGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGGTCAATCCATTTGGGAACTTGTGTCTCTGCGGCGCGCTTCGGTGTGCTGCGGGTGGCATCTCAA
>AF266764 Phytophthora cinnamomi
CCACACCTAAAAAACTTTCCACGTGAACCGTTTCAACCCAATTAGTTGGGGGCCTGCTCTGGGCGGCGGTTGTCGATGTCAAAGTCGACGGTTGCTGTTGCGTGGGGGGCCCTATCACTGGCGAGCGTTTGGGTCCCCCTCGGGGGAACTGAGCTAGTAGCCTCTCTTTTAAACCCATCCTGTAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTGGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCTCTTCCTTCCGTGTAGTCGGTGGATGGAGGTGCCAGACGTGAGGTGTCTTGCGGGCGGTCTTCGGACTGGCTGTGAGTCCCTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGCTGGTTGTGGAGGCTGCCTGTATGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTCCGATTCGCGGTATGGTTGGCTCCGGCTGAACAAAGCGCTTATTGGATGTTCCTCCCTGCTGTGGCGGTACGGATCGGTGAACCGTAGCTGTGCTAGGCTTGGCGTTTGAACCGGCGGTGTTGTTGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAACTCTGTGTCTCTCCGGCGCACTTGTGTGCTTGTGGTGGCATCTCAA
>AF266765 Phytophthora cajani
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACAAGTAGTTGGGGGCCTGCTCTGTGTGGCTAGCTGTCGATGTCAAAGTCGGCGACTGGCTGCTATGTGGCGGGCTCTATCATGGCAATTGGTTTGGGTCCTCCTCGTGGGGAACTAGATCATGAGCCCACTTTTTAAACCCATTCTTGATTACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCTCTTCCTTCCGTGTAGTCGGTGGATGGAGACGCCAGACGTGAGGTGTCTTGCGGCGCGGCCTTCGGGCTGCCTGCGAGTCCCTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTCGTTGGTTGTGGAGGCTGCCTGTATGGCCAGTCGGCGACCAGTTTGTATGGTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGATGCTTTTCCTGCTGTGGCGGTATGGGCTGGTGAACCGTAGTTGTGCAAGGCTTGGCTTTTGAACCGGTGGTGTTGTTGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAACTCTGTGTTGTCTCTGCGGCTTGCTGTGGAGGTAGCATCTCAA
>AF266766 Phytophthora vignae
CCACACCTAAAAAAACTTTCCACGTGAACCGTTTCAACAAGTAGTTGGGGGCCTGCTCTGTGTGGCTAGCTGTCGATGTCAAAGTCGGCGACTGGCTGCTATGTGACGGGCTCTATCATGGCAATTGGTTTGGGTCCTCCTCGTGGGGAACTAGATCATGAGCCCACTTTTTAAACCCATTCTTGATTACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCTCTTCCTTCCGTGTAGTCGGTGGATGGAGACGCCAGACGTGAGGTGTCTTGCGGCGCGGCCTTCGGGCTGCCTGCGAGTCCCTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGTTGGTTGTGGAGGCTGCCTGTATGGCCAGTCGGCGACCAGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGATGCTTTTCCTGCTGTGGCGGTATGGGCTGGTGAACCGTAGTTGTGCAAGGCTTGGCTTTTGAACCGGTGGTGTTGTTGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAACTTTGTGTTGTCTCTGCGGCTTGCTGTGGAGGTAGCATCTCAA
>AF266767 Phytophthora melonis
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACAAGTAGTTGGGGGCCTGCTCTGTGTGGCTAGCTGTCGATGTCAAAGTCGGCGACTGGCTGCTATGTGGCGGGCTCTATCATGGCGATTGGTTTGGGTCCTCCTCGTGGGGAACTGGATCATGAGCCCACCTTTTAAACCCATTCTTGATTACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCTCTTCCTTCCGTGTAGTCGGTGGATGGAGACGCCAGACGTGAGGTGTCTTGCGGCGCGGCCTTCGGGCTGCCTGCGAGTCCCTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGTTGGTTGTGGAGGCTGCCTGTATGGCCAGTCGGCGACCAGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGATGCTTTTCCTGCTGTGGTGGTATGGGCTGGTGAACCGTAGTTGTGCGAGGCTTGGCTTTTGAACCGGCGGTGTTGTAGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAACTCTGTGTTGTCTCTGCGGCTTGCTGTGGAGGTAGCATCTCAA
>AF266768 Phytophthora sinensis
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACAAGTAGTTGGGGGCCTGCTCTGTGTGGCTAGCTGTCGATGTCAAAGTCGGCGACTGGCTGCTATGTGGCGGGCTCTATCATGGCGATTGGTTTGGGTCCTCCTCGTGGGGAACTGGATCATGAGCCCACCTTTTAAACCCATTCTTGATTACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCTCTTCCTTCCGTGTAGTCGGTGGATGGAGACGCCAGACGTGAGGTGTCTTGCGGCGCGGCCTTCGGGCTGCCTGCGAGTCCCTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGTTGGTTGTGGAGGCTGCCTGTATGGCCAGTCGGCGACCAGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGATGCTTTTCCTGCTGTGGTGGTATGGGCTGGTGAACCGTAGTTGTGCGAGGCTTGGCTTTTGAACCGGCGGTGTTGTAGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAACTCTGTGTTGTCTCTGCGGCTTGCTGTGGAGGTAGCATCTCAA
>AF266769 Phytophthora sojae
CCACACCTAAGAAACTTTCCACGTGAACCGTATCAACAAGTAGTTGGGGGCCTGCTCTGTGTGGCTGTCTGTCGATGTCAAAGTCGGCGGCTGGCTGCTGTGTGGCGGGCTCTATCATGGCGATTGGTTTGGGTCCTCCTCGTGGGGAACTGGATCATGAGCCCACTTTTTAAACCCATTCTTAAATACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCTCTTCCTTCCGTGTAGTCGGTGGATGGAGACGCCAGACGTGAGGTGTCTTGCGGCGTGGCCTTCGGGCTGCCTGCGAGTCCCTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGTTGGTTGTGGAGGCTGCCTGTATGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGATGCTTTTCCTGCTGTGGCGGTATGGGCTGGTGAACCGTAGCTGTGTGAGGCTTGGCTTTTGAACCGGCGGTGTTGTTGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAACTCTGTGTTGTCTCTGCGGCTTGCTGCGGAGGTGGCATCTCAA
>AF266770 Phytophthora heveae
CCACACCTAAAAAATTTCCACGTGAACCGTAACAAACCAATAGTTGGGGGCGAGTTTGGCGGCTACTGCTGCTGGACGAGCTCTATCATGGCGAGCGTTTGGACTTCGGTCTGAACTAGTAGCCCTTCTTTTAAACCCATTCCTAATTACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACAACAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGAGATGCCAGATGTGAAGTGTCTTGCGGCTGGTTCTCGAACTGGCTGCGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCATGGCGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCATGTCGGCGACCGGTCTGTCTGCTGCAGCGTTAATGGAGGCGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGTATGTGTTTCCTGTTGTGGCGGTAATGGCTGGTGTACCGTAGCTATGTGGTGCTTGGCTTTTGAATCGGCTTTGCTGTTGTGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAAATTTGTGTGTGCCTTTATTGGCGTGCATCTCAA
>AF266771 Phytophthora katsurae
CCACACCTAAAAAACTTTCCACGTGAACCGTAACAAACCAATAGTTGGGGGCGAGTTTGGCGGCGGCTGCTGGCTTTGGCTGGCTGGCTGCTGCTGGACGAGCTCTATCATGGCGAGCGTTTGGACTTCGGTCTGAACTAGTAGCCCTTCTTTTAAACCCATTTCTAATTACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACAACAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGAGATGCCAGATGTGAAGTGTCTTGCGGCTGGTTCTCGAACTGGCTGCGAGTCCTTTGAAATGTACTGAACTGTACTCCTCTTTGCTCAAAAAGCATGGCGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCATGTCGGCGACCGGTCTGTCTGCTGCGGCGTTAATGGAGGCGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGATGTGTTTCCTGCTGTGGCGGTAATGGCTGGTGTACCGTAGCTATGTGGCGCTTGGCTTTTGAATCGGCTTTGCTGTTGTGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAAATTTGTGTGTGCCTTTATTGGCGTGCATCTCAA
>AF266772 Phytophthora cactorum
CCACACCTAAAAACTTTCCACGTGAACCGTTTCAAACCAAATAGTTGGGGGTCTTGTCTGGTGGCGGCTGCTGGCTTTATTGTTGGCGGCTGCTGCTGGGTGAGCCCTATCATGGCGAGCGTTTGGGCTTCGGCCTGAGCTAGTAGCTTTTCTTTTAAACCCATTCCTTAATACTGATTATACTGTGGGGACGAAAGTCCTTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGAGATGCCAGATGTGAAGTGTCTTGCGGCTGGTTTTCGGACCGACTGCGAGTCCTTTTAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCTATTGTAGCAAGTTGGCGACCGGTTTGTCTGCTGCGGCGTTAATGGAAGAGTGTTCGATTCGCGGTATGGTTAGCTTCGGCTGAACAATGCGCTTATTGGATGTTTTTTCTGCTGTGGCGTGATGGACCGGTGAACCATAGCTCAGTTGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAAATGTGTGTGTACTTCGGTATGCATCTCAA
>AF266773 Phytophthora idaei
CCACACCTAAAAACTTTCCACGTGAACCGTTTCAAACCAAATAGTTGGGGGTCTTGTCTGGTGGCGGCTGCTGGCTTTATTGTTGGTGGCTGCTGCTGGGTGAGCCCTATCATGGCGAGCGTTTGGGCTTCGGCCTGAGCTAGTAGCTTTTCTTTTAAACCCATTCCTTAATACTGATTATACTGTGGGGACGAAAGTCCTTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGAAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGAGATGCCAGATGTGAAGTGTCTTGCGGCTGGTTTTCGGACCGACTGCGAGTCCTTTTAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCTATTGTAGCAAGTTGGTGACCGGTTTGTCTGCTGCGGCGTTAATGGAAGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGATGTTTTTTCTGCTGTGGCGTGATGGACCGGTGAACCATAGCTCAGTGGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGGAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAAATGTGTGTGTACTTCGGTATGCATCTCAA
>AF266774 Phytophthora pseudotsugae
CCACACCTAAAAACTTTCCACGTGAACCGTTTCAAACCAAATAGTTGGGGGTCTTGTCTGGTGGCGGCTGCTGGCTTTGTTGTTGGCGGCTGCTGCTGGGTGAGCCCTATCATGGCGAGCGTTTGGGCTTCGGCCTGAGCTAGTAGCTTTTCTTTTAAACCCATTCCTTAATACTGATTATACTGTGGGGACGAAAGTCCTTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGAGATGCCAGATGTGAAGTGTCTTGCGGCTGGTTTTCGGACCGACTGCGAGTCCTTTTAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCTATTGTAGCAAGTTGGCGACCGGTTTGTCTGCTGCGGCGTTAATGGAAGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGATGATTTTTCTGCTGTGGCGTGATGGACCGGTGAACCATAGCTCAGTGGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATTCATTTGGGAAATGTGTGTGTACTTCGGTATGCATCTCAA
>AF266775 Phytophthora tentaculata
CCACACCTAAAAAATTTCCACGTGAACCGTTTCAACCAAAAGTTGGGGGTCTTGTTTGGCGGTGACTGCTGGCTTTATTGCTGGCGGCTGCTGCTGGACGAGCCCTATCATGGCGAAAGTTTGGACTTCGGTCTGAGCTAGTAGTTATTATTTTAAACCCATTCCTTAATACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGAAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGACTTTCTTTCTTCCGTGTAGTCGGTGGAGGAGATGTCAGATATGAAGTGTCTTGCGGTTGGTTTTCGGACCGACTGTGAGTCCTTTTAAATGTACTAAACTGTACTTCTCTTTGCTCCAAAAGTGGTGGCATTGCTGGTTGTGGAGGCTGTTACTGTGGCCAATTGGCGACCGGTTTGTCTGCTGCGGCGTTAATGGAAGAGTGCTTGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGGTGTTTATCCTGCTGTGGCGTGATGGACTGGTGAACCATAGCTCGGTGACTTGGCCTTTGAATCGGCTTTTGCTGTTGCGAAGTAGAGTGGCGACTTCGGTTGTCGAGGGTCGATCCATTTGGGAAATTTGTGTGTACTTCGGTATGCATCTCAA
>AF266776 Phytophthora nicotianae
CCACACCTAAAAAAACTTTCCACGTGAACCGTTTCAACCCAATAGTTGGGGGTCTTATTTGGCGGCGGCTGCTGGCTTAATTGTTGGCGGCTGCTGCTGAGTGAGCCCTATCAAAAAAAAGGCGAACGTTTGGGCTTCGGCCTGATTTAGTAGTCTTTTTTTCTTTTAAACCCATTCCTTAATACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGAAGTATGCCTGTATCAGTGTCCGTACATTAAACTTGACTTTCTTCCTTCCGTGTAGTCGGTGGAGGAGATGTCAGATGTGAAGTGTCTTGCGATTGGTCTTCGGACCGGCTGCGAGTCCTTTTAAATGTACTAAACTGAACTTCTCTTTGCTCGAAAAGTGGTGGCGTTGCTGGTTGTGAAGGCTGCTATTGTGGCAAATTGGCGACTGGTTTGTCTGCTGCGGCATTAATGGAAGAGTGTTCGATTCGTGGTATGGTTGGCTTCGGCTGAACAATGCACTTATTGGACGTTTTTCCTGCTGTGGCGTGATGGACTGGTGAACCATAGCTCGGTGGCTTGGCTTTTGAATTGGCTTTGCTGTTGCGAAGTAGGGTGGCAGCTTCGGTTGTCGAGGGTCGATCCATTTGGGAACTTAATGTGTACTTCGGTATGCATCTCAA
>AF266777 Phytophthora mirabilis
CCACACCTAAAAACTTTCCACGTGAACCGTTTCAACCCAATAGTTGGGGGTCTTACTTGGCGGCGGCTGCTGGCTTTATTGCTGGCGGCTACTCTGGGCGAGCCCTATCAAAAGGCGAGCGTTTGGACTTCGGTCTGAGCTAGTAGCTTTTTTATTTTAAACCCTTTACTTAATACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGAAGTATGCCTGTATCAGTGTCCGTACAACAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGAGATGCCAGATGTGAAGTGTCTTGCGGTTGGTTTTCGGACCGACTGCGAGTCCTTTTAAATGTACTAAACTGTACTTCTCTTTGCTCCAAAAGTGGTGGCATTGCTGGTTGTGGACGCTGCTATTGTAGCGAGTTGGCGACCGGTTTGTCTGCTGCGGCGTTAATGGAGAAATGCTCGATTCGTGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGGTGATTTTCCTGCTGTGGCGTGATGGACTGGTGAACCATGGCTCTTTAGCTTGGCATTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGCCGAGGGTCGATCCATTTGGGAAATGTTGTGTACTTCGGTATGCATCTCAA
>AF266778 Phytophthora phaseoli
CCACACCTAAAAACTTTCCACGTGAACCGTTTCAACCCAATAGTTGGGGGTCTTACTTGGCGGCGGCTGCTGGCTTTATTGCTGGCGGCTACTGCTGGGCGAGCCCTATCAAAAGGCGAGCGTTTGGGCTTCGGTCTGAGCTAGTAGCTTTTTTATTTTAAACCCTTTACTTAATACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGAAGTATGCCTGTATCAGTGTCCGTACAACAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGAGATGCCAGATGTGAAGTGTCTTGCGGTTGGTTTTCGGACCGACTGCGAGTCCTTTTAAATGTACTAAACTGTACTTCTCTTTGCTCCAAAAGTGGTGGCATTGCTGGTTGTGGACGCTGCTATTGTAGCGAGTTGGCGACCGGTTTGTCTGCTGCGGCGTTAATGGAGAAATGCTCGATTCGTGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGGTGATTTTCCTGCTGTGGCGTGATGGACTGGTGAACCATGGCTCTTTAGCTTGGCATTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGCCGAGGGTCGATCCATTTGGGAAATGTTGTGTACTTCGGTATGCATCTCAA
>AF266779 Phytophthora infestans
CCACACCTAAAAACTTTCCACGTGAACCGTTTCAACCCAATAGTTGGGGGTCTTACTTGGCGGCGGCTGCTGGCTTTATTGCTGGCGGCTACTGCTGGGCGAGCCCTATCAAAAGGCGAGCGTTTGGACTTCGGTCTGAGCTAGTAGCTTTTTTATTTTAAACCCTTTACTTAATACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGAAGTATGCCTGTATCAGTGTCCGTACAACAAACTTGGCTTTCTCCCTTCCGTGTAGTCGGTGGAGGAGATGCCAGATGTGAAGTGTCTTGCGGTTGGTTTCCGGACCGACTGCGAGTCCTTTGAAATGTACTAAACTGTACTTCTCTTTGCTCCAAAAGTGGTGGCATTGCTGGTTGTGGACGCTGCTATTGTAGCGAGTTGGCGACCGGTTTGTCTGCTGCGGCGTTAATGGAGAAATGCTCGATTCGTGGTATGGTTGCCTTCGGCTGAACAATGCGCTTATTGGGTGATTTTCCTGCTGTGGCGTGATGGACTGGTGAACCATGGCTCTTTAGCTTGGCATTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGCCGAGGGTCGATCCATTTGGGAAATGTTGTGTACTTCGGTATGCATCTCAA
>AF266782 Phytophthora megakarya
CCACACCTAAAAACTTTCCACGTGAACCGTATCAACCTTTAGTTGGGGGTCTTTTTCGGCGGCGGCTGCTGGTTTAATTACTGGCGGTTGCTGCTGGGAGAGCCCTATCATGGCGAGCGTTTGGGCTTCGGTCTGAACTAGTAGCTTTTTTAAACCCATTCATTATTACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACCTCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGAAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGATGTGAGGTGTCTTGCGGCTGGCCTTCGGGTCGGCTGTGAGTCCCTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGTAAAGCTTGCTTGTTGTGGAGGCTGCTTGTGTAACCAGTCGGCGACTAGTTTGTCTGCTGTGGCGTTAATGGAGGAGTGTTCGATTCGCGGTATGATTGGCTTCGGCTGAACAGAAGCTTATTGGGCGTTTTTCCTGCTATGGCGGTATGAAGTAGTGAACCGTAGTTATGTGGGCTTGGCTTTTGAATGTGCTCGCTGTGCGAAGTAGAGTGGCGACTTTGGTTGTCGAGGGTCGATCCATTTGGGAAATTGTGTGTACTTCGGTATGCATCTCAA
>AF266784 Phytophthora botryosa
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCTTTTAGTTGGGGGTGTTGCTTGGCGTTTTGCTGAGCCGCGCCCTATCATGGCGAATGTTTGGACTTCGGTCTGGGCTAGTAGCTTTTTTGTTTTAAACCCATTTAACAATACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGATGTGCCAGATGTGAAGTGTCTTGCGGTTTTTGTGCCTTCGGGCCGTGGCCGCGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGTGTTGCTGGTTGTGGAGGCTGCCTGCGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGGCGCTTATTGTATGCTTTTCCTGCTGTGGCGTGATGGGCTGGTGAACCGTAGCTGTGTGTGGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGTGTCGATCCATTTTGGGAAATTTTGTGTGCGCTTTCGAGTGTGCATCTCAA
>AF266785 Phytophthora citrophthora
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCTTTTAGTTGGGGGTGTTGCTTGGCGTTTTTGCTGAGCCACGCCCTATCATGGCGAATGTTTGGACTTCGGTCTGGGCTAGTAGCTTTTTTGTTTTAAACCCATTTAACAATACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGATGTGCCAGATGTGAAGTGTCTTGCGGTTTTTGTGCCTTCGGGCCGTGGCTGCGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGTGTTGCTGGTTGTGGAGGCTGCCTGCGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGGCGCTTATTGTATGCTTTTCCTGCTGTGGCGTGATGGGCTGGTGAACCGTAGCTGTGTGTGGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGTGTCGATCCATTTTGGGAACGTTTGTGTGCGCTTTCGAGTGTGCATCTCAA
>AF266786 Phytophthora colocasiae
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCTTTTAGTTGGGGGTGTTGCTTGGCATTTTGCTGAGGCGCCCTATCATGGCGAATGTTTGGACTTCGGTCTGGGCTAGTAGCTTTTTGTTTTAAACCCTTTAACAATACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGATGTGCCAGATGTGAAGTGTCTTGCGGTTTGTGTGCCTTCGGGCCGAGGCTGCGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGTGATGCTGGTTGTGGAGGCTGCCTGCGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGACGCTTATTGTATGCTTTTCCTGCTGTGGCGTGATGGGCTGGTGAACCGTAGCTGTGTGTGGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGTGTCGATCCATTTTGGGAACTTTTGTGTGCGCTTTCGAGTGTGCATCTCAA
>AF266787 Phytophthora capsici
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCTTTTAGTTGGGGGTCTTGTACCCTATCATGGCGAATGTTTGGACTTCGGTCCGGGCGAGTAGCTTTTTGTTTTAAACCCATTTCACAATTCTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGATGTGCCAGATGTGAAGTGTCTTGCGTGTTGTCCTTCGGGTCGACTGCGAGTCCTTTTAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGTGTTGCTGGTTGTGGAGGCTGCCTGCGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAAAGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGGCGCTTATTGAATGCTTTTCCTGCTGTGGCGTGATGGGCTGGTGAACCGTAGCTGTGTTTGGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTTGGGAACTTTGTGTGCACTTCGGTGCGCATCTCAA
>AF266789 Phytophthora plurivora
CCACACCTAAAAACTTTTCACGTGAACCGTATCAACCCTTTTAGTTGGGGGTCTTGCTTTTTTTGCGAGCCCTATCATGGCGAATGTTTGGACTTCGGTCTGGGCTAGTAGCTTTTTGTTTTAAACCCATTCTACAATACTGATTATACTGCGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGATGTGCCAGACGTGAAGTGTCTTGCGGGTGTCCTTCGGGTCGTCTGCGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGTGTTGCTGGTTGTGGAGGCTGCCTGCGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGGCGCTTATTGTATGCTTTTCCTGCTGTGGCGTGATGGGCTGGTGAACCGTAGCTGTGTGTGGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAACTTTGTGTGCACCTCGGTGCGCATCTCAA
>AF266790 Phytophthora multivesiculata
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCCTTTAGTTGGGGGTCTTGCTTGGCGTGCGGCTGCTGGCCTTTATTGGTTGGCTGGCTGCGTGCTGGGCGAGCCCTATCATGGCGATCGTTTGGACCTCGGTCTGAGCTAGTAGCTTTTGTTTTAAACCCATTCTTTAATACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGAGACGTCAGATGTGAAGTGTCTTGCGGTTGGCCTTCGGGTCGTCTGCGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGCTCGAAAAGCGTGGTATTGTTGGTTGTGGAGACTGCCTGTGTGGCCAGTCGGCGACCGGTTTGTCTGCTACGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGACGCTTATTGGACGTTTTTCCTGCTGTGGCGTGATGGGCTGGTGAACCGTAGCTGTGTTTGGCTTGGTCTTTGAATCGGCTTTGCTGTTGCGAAGTAGAGCGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAGTTTGTGTGTACTTCGGTGCGCATCTCAA
>AF266792 Phytophthora humicola
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCCTTAAGATTGGGAACTTGCTCGGCGGCGTGCGTGCTGGCCTGTAATGGGTCGGCGTGCTGCTGCTGCGCGGGCTCTATCATGGGCGAGCGTTTGGGCTTCGGCTCGAGCTAGTAGCTTTTTCTTTTAAACCCCTTCTTTAATTACTGAAATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTCGTCCTGGAAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTAGCGGTCTTTCGAGTCTGCCGGCGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGCTCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCCGTATGGCCAGTCGGCGACCGGTTTGTTAGCTGTAGTATATACTGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGGGTACTTTTCCTGCTATTGGCGGTACGAACTGGTGAACCGTAGCTGTGTGGAGCTTGGCTTTTGAATCGGCTTTGCTTGCGAAGTAGAGTGGCGGCTTCGGCTGCCGAGGGGTCGATCCATTTTGGGAAATTATGTGTGTGCGGCTTCGTGCTGCGCGCATCTCAA
>AF266794 Phytophthora megasperma
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCCTTAAATTTGGGGGCTTGCTCGGCGGCGTGTGTGCTGGCCTGTAATGGGTCGGCGTGCTGCTGCTGGGCGGGCTCTATCATGGGCGAGCGTTTGGGCTTCGGCTCGAGCTAGTAGCTATCAATTTTAAACCCTTTCTTAAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGATGTGAAGTGTCTTGCTAGCGGTCTTTCGAGTCTGCCGGTGAGTCCTTTTAAATGTACTGAACTGTACTCTCTCTTTGCGCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCCGTGTGGCCAGTCGGCGACCGGTTTGTTAGCTGTGGCGTTTGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTAAACAATCTGCTTATTGGGTGCTTTTCCTGTCATGGCGGTATGAACTGGTGAACCGTAGCTGTGTGGTGCTTGGCTTTTGAACCGGCTTTGCTTTGCGAAGTAGTGTGACGGCTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAACTTTGTGTGTGCGGCTTCGTGCTGCGCGCATCTCAA
>AF266795 Phytophthora taxon asparagi
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCAATTAGTTGGGGGCTTGCTCTGGCGTGCGGCTGTTGACCTGTAATGGGCTGACGGCTGCTGCTGGGCGGGCTCTATCAAAGGCGAGCGTTTGGACCTCGGTCCGAGCTAGTAGCTTTACTTTTTTAAACCCATTCTTTACATACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGATGGAGACGCCAGACGTGAAGTGTCTTGCTGCTGGTCTTTCGAGTCCGGCGGTGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCAAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCTGCGTGGCCAGTCGGCGACCGGTTTGTCTGCTGTGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGAATGTTTTTCCTGCTGTGGCGGTACGAACTGGTGAACCGTAGCTGTGTGTGGCTTGGCTTTTGAACCGGCTTTGCTGTTTGCGAAGTAGAGCGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAACTTTTGTGTGTGCGGCTTCGGCTGCGCGCATCTCAA
>AF266796 Phytophthora cryptogea
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCTTTTTAATTTGGGGGCTTCCGTCTGGCCGGCCGGTTTTCGGCTGGCTGGGTGGCGGCTCTATCATGGCGACCGCTTGGGCCTCGGCCTGGGCTAGTAGCGTATTTTTAAACCCATTCCTAATTACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACACTAAACTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGACGCGCAGATGTGAAGTGTCTTGCGGCTGGTCTTCGGTCCGGCTGCGAGTCCTTTGAAATGTACTACACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCATGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGACGCTTATTGGGTGCTTTTCCTGCTGTGGCTGGATGGACTGGTGAACCGTAGCTGTGCTAGGCTTGGCGTTTGAACCGGCGGTGTGGTGCGAAGTAGGGTGTCTGTTCTGGCGTAAGCTGGGGTGGACGAGGGTCGATCCATTTGGGAAACGTTGTGTGCGCTTCGGCGCGCATCTCAA
>AF266797 Phytophthora erythroseptica
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCTTTTTAAATTGGGGGCTTCCGTCTGGCCGGCCGGTTTTCGGCTGGCTGGGTGGCGGCTCTATCATGGCGACCGCTTGGGCCTCGGCCTGGGCTAGTAGCGTATTTTTAAACCATTCCTAATTACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACACTAAACTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGACGCGCAGATGTGAAGTGTCTTGCGGCTGGTCTTCGGTCCGGCTGCGAGTCCTTTGAAATGTACTACACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCATGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGACGCTTATTGGGTGCTTTTCCTGCTGTGGCTGGATGGACTGGTGAACCGTAGCTGTGCTAGGCTTGGCGTTTGAACCGGCGGTGTGGTGCGAAGTAGGGTGTCTGTTCCGGCGTAAGCTGGGGTGGACGAGGGTCGATCCATTTGGGAAACGTTGTGTGCGCTTCGGCGCGCATCTCAA
>AF266798 Phytophthora drechsleri
CCACACCTAAAAAACTTCCACGTGAACCGTATCAACCTTTTTAAATTGGGGGCTTCCGTCTGGCCGGCCGGTTCTCGGCTGGCTGGGTGGCGGCTCTATCATGGCGACCGCCTGGGCCTCGGCCTGGGTTAGTAGCGTATTTTTTAAACCATTCCTAATTACTGAAAAAACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACACTAAACTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGACGCGCAGATGTGAAGTGTCTTGCGGCTGGTCTTCGGTCCGGCTGCGAGTCCTTTTAAATGTACTACACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCATGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGACGCTTATTGGGTGCTTTTTCTGCTGCGGCGGGATGGACTGGTGAACCGTAGCTGTGCTAGGCTTGGCGTTTGAACCGGCGGTATTGTGCGAAGTAGGGTGTCTGTCTCGGCGCAAGCTGGGGTGGGCGAGGGTCGATCCATTTGGGAAACGTTGTGTGCGCTTCGGCGTGCATCTCAA
>AF266799 Phytophthora medicaginis
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCTTTTAAATTGGGGGCTTCCGTCTGGCCGGCCGGCTTTCGGCTGACTGGGTGGCGGCTCTATCATGGCGACCGCTTGGGCCTCGGCTTGGGCTAGTAGCTTCTTTTAAACCCATTCCTAATTACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACAATAAACTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGACGCGCAGATGTGAAGTGTCTTGCGGCTGGTCTTCGGTCCGGCTGCGAGTCCTTTGAAATGTACTAAACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCATGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGACGCTTATTGGGTGCTTTTCCTGCTGTGGTGGGACGGACTGGTGAACCGTAGCTGTACTAGGCTTGGCGTTTGAACTGGCGGTGTGGTGCGAAGTAGGGTGTCTGTTCCGGCGCAAGCTGGGGTGGGCGAGGGTCGATCCATTTGGGAAAGTTGTGTGCGCTTCGGCGCGCATCTCAA
>AF266800 Phytophthora trifolii
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCTTTTAAATTGGGAGCTTTCGTCTGGCCGGCCGGCTTTTGGCTGGTTGGGCGGCGGCTCTATCATGGCGACCGCTTGGGCCTCGGCTTGGGCTAGTAGCTTCTTTTAAACCCATTCCTAATTACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACAATAAACTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGACGCGCAGATGTGAAGTGTCTTGCGGCTGGTCTTCGGTCCGGCTGCGAGTCCTTTGAAATGTACTAAACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCATGTCGGCGACTGGTTTGTCTGCTGCAGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGACGCTTATTGGGTGCTTTTCCTGCTGTGGTAGTATGGACTGGTGAACCGTAGCTGTGCTAGGCTTGGCGTTTGAACTGGCGGTGTGGTGCGAAGTAGGGTGTCTGTTCCGGCGCAAGCTGGGGTGGGCGAGGGTCGATCCATTTGGGAAAATTGTGTGCGCTTCGGCGCGCATCTCAA
>AF266802 Phytophthora primulae
CCACACCTAAAAACTTCCCCCGTGAACCGTATCAACCCTTTTTTATTGGGGGCTGCTGACTGGACTGGCTTCGGCTGGACTGGGTGGCGGCTCTAGCGCGACCGTTCTGGGCTTCGGCTTGGGGCTAGTAGCTTTCTTTTTAAACCCATTCTTAATTTACTGAAATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTCGTCCTGGAAGTATGCCTGTATCAGTGTCCGTACAATAAACTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGATGCCAGATGTGAAGTGTCTTGCGACTGGACTTTGATCTAGATGCGAGTCCTTTTAAATGTACTAAAACTGTATTCTCTTTGCTCGAAAAGCGTGACGTTGCTGGTTGTGGAGGCTGCCTGTATGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGTGTTTATGGAGAAGTGTTTAATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTAAATGTTTTTCTTACTGTGGCGGATAGACTGGTGAACCGTAGCTGTGTAGGCTTGGCCTTTGAACCGGTGATATTGTGCGAAGTAGAGTGACGGCTGTTCCGGCGCAAGCTGGGGTGGCTGTCGAGGGTCGATCCATTTGGGAAATGTTGTGCCGTTGCGACCTCGGTTGCTTTGGCATCTCAA
>AF266803 Phytophthora syringae
CCACACCTAAAAAACTTCCACGTGAACCGTATCAAAACCCTTTTATTGGGGGCTTCTGTCTGGTCTGGCTTCGGCTGGATTGGGTGGCGGCTCTATCATGGCGACCGCTCTGAGCTTCGGCCTGGAGCTAGTAGCCCACTTTTTAAACCCATTCTTAATTACTGAACAAACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGATGCACAGACGTGAAGTGTCTTGCGACTGGGCTTCGGCTCGGCTGCGAGTCCTTTTAAATGTACAGAACTGTACTTCTCTTTGCTCGAAAAGCGTTATATTACTGGTTGTGGAGGCTGCCTGTGCGGCAAGTCGGCGACCGGTTTGTTAACTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGATGGCTTCGGCTGAACTGACGCTTATTGAGTACTTTTCCTGCTGTGGTGGTACGAACTGGTGAACCGTAGCTGTGTTTGGCTTGGCTTTTGAACTGGCGATGTGGTGCGAAGTAGAGTGACGGTTGTTCCGGCGCAAGCTGGAGTGACTGTCGAGGGTCGATCCATTTGGGAAATTTTGTGTCTGTGCGACTTCGGTTGCGTGGCATCTCAA
>AF266804 Phytophthora lateralis
CCACACCTAAAAAACTTCCACGTGAACCGTATCAAAACCCTTAGTTGGGGGCTTCTGTTCGGCTGGCTTCGGCTGGCTGGGCGGCGGCTCTATCATGGCGAGCGCATGGGCCTTCGGGTCTGAGCTAGTAGCCCTCTTTTTAAACCCATTCCTAAATACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGCCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGACGTGCAGACGTGAAGTGTCTTGCGATTGGTCTTCGGGCCGGCTGCGAGTCCTTTGAAATGTACAGAACTGTACTTCTCTTTGCTCGAAAAGCATGACGTTGTTGGTTGTGGAGGCTGTCCGTGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTAGCTTCGGCTGAACAATGCGCTTATTGGATGTTTTTTCTGCTGTGGCGGTAATGACTGGTGAACCGTAGCTATGCAGGGCTTGGCTTTTGAACCGACGGTGTTGTGCGAAGTAGAGTGGCGGTTTGGCGCAAGCTGGGCTGTCGAGGGTCGATCCATTTGGGAAATTTGTGTTGGCAGCTTCGGCTGTTGGCATCTCAA
>AF271221 Phytophthora richardiae
CCACACCTAAAATTTTCCACGTGAACCGTATTTTTGACGTTGGGGACTCCTGGCTTCGGCCGGTTGAGACCTATCACGCGACCGCTCGGGCTGAAAAGTCGGGGCCAGTAGCCTTTTTTTAAACCCATTCCGAACTACTGAACCATACTGTGGGGACGAAAGTCCTTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTATGCCTGGGAGTATGCCTGTATCAGTGTCCGTACATGAAACTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGACGCGCAGACGTGAAGTGTCTTGCGGCGGCGGCTTCTGCCGGCTGCTGCGAGTCCTTTGAAATGTAAGACCTGTTCTTCTCTTTGCCCGAAAAGCGTTGCGTGAGTGTGGTTGTGGAGGCTGCCGTTTGCGGCATGTCGGCGACTGATTTCGTGCTGAGGCGTGTGGGGAGAGGCTCGATTCGCGGTATGGTTGGCTTCGGCTGAACTTCTGCTTATTGGTCTTTTTTTCGCGGCTTTGGCGTTTTGCGGGGTCGGTGTACCGTAGCTGCGGGTGGCTTGGCCTTTGAACCGTGTTCGTGCGGAGCGAAGTAAGGTGGGCGTGACTGTAGTGTTATGGCCGAGGGGGACGCAACCATTTGGGAAAGTAATGCTTTTCTGCCGGCTTTGGGGCTGGCGGGGCTGCATCTCGA
>AF271222 Phytophthora insolita
CCACACCTAAAAACTGTCCACGTGAACTGTGTCGATGTTGGGGCGTGCTGGGCGTGCGTGTGTGTGTGGCTTCGGCCGCGTGCGTGCGCGTGTTCGGTTGTGCTCATCAAACGAAGAGGCGGGTCGCAAGGCCTGTGTTCTTTGTTACTGTTTTAAATCCTTCTTATACACTGATTGTACTGTGGGGACGAAAGTCTCTGCTTTGAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTATGCCTGGGAGTATGCCTGTATCAGTGTCCGTACATGAAACTTGGCTTCCTTCCTTCCGTGTAGTCGGTGGCGGGAACGCTCAGACGTGAAGTGTCTTGCCTTGGCAATTTGTAGGTTGTTGTGGCGAGTCCTTTGAAATGTAAGACGTGTGTTCTCTTTGCTGGAAAAGCGTGTGGCGCGTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACTGATTTCGTGCTGATGCGTGCGGAGAGCGGTTCGATTCGCGGTATGGTTGGCTTCGGCTGATTTTCTGCTTATTGGGTTGCTGTTTGCTGCGTCGGCGTTTGCGGGGTCGGTGAACCGTAGCTGCGTTGGGCTTGGCTTCTGAACTGTGTGCTGTGCTGCGAAGTATGGTGGCTGCGTGTGCCTTGTGGTACGCGCGGCCGAGAGACTGTCACCATTTGGGAAGTTTGTGTCGTCGGCCTCGGCCGGTGGCATCTCAA
>AF279128 Phytophthora glovera
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCTTTTAGTTGGGGGTCTTGTACCCTATCATGGCGAATGTTTGGACTTCGGTCCGGGCGAGTAGCTTTTTGTTTTAAACCCATTTCACAATTCTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGATGTGCCAGATGTGAAGTGTCTTGCGTGTTGTCCTTCGGGTCGACTGCGAGTCCTTTTAAATGTACTGAACTGTACTTCTCTTTGCTCAAAAAGCGTGGTGTTGCTGGTTGTGGAGGCTGCCTGCGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAAAGGAGGAGTGTCCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGGCGCTTATTGAATGCTTTTCCTGCTGTGGCGTGATGGGCTGGTGAACCGTAGCTGTGTTTGGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTTGGGAATTTTGTGTGCACTTCGGTGCGCATCTCAA
>AF380147.1 Phytophthora brassicae
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCTTGATTATTGGGGGCTGCTGACTGGACTGGCTTCGGCTAGACTGGGCGGCGGCTCTAGCGCGACCGTTCTGGGCTTCGGCTTGGGGCTAGTAGCTTATTTTAAACCCATTCTTAATTTACTGAAATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTCGTCCTGGAAGTATGCCTGTATCAGTGTCCGTACAATAAACTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGATGCGCAGATGTGAAGTGTCTTGCGACTGGGCTTCGGTCTGGATGCGAGTCCTTTTAAATGTACTAAAACTGTATTCTCTTTGCTCGAAAAGCGTGACGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGTGTTACTGGAGAAGTGTTTGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTAGATGTTTTTCTTACTGCGGCGGATGGACTGGTGAACCGTAGCTGTGTAGGCTTGGCTTTTGAACCGGCGATATTGTGCGAAGTAGAGTGACGGCTGTTCCGGCGCAAGCTGGGGTGGCTGTCGAGGGTCGATCCATTTGGGAAATGTTGTGCCGTTGCGACCTCGGTTGCTTTGGCATCTCAA
>AF380150.1 Phytophthora porri
CCACACCTAAAAACTTTCCCCGTGAACCGTATCAACCCTTTTTTATTGGGGGCTGCTGACTGGACTGGCTTCGGCTGGACTGGGTGGCGGCTCTAGCGCGACCGTTCTGGGCTTCGGCCTGGGGCTAGTAGCTTTCTTTTTAAACCATTCTTAATTTACTGAAATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTCGTCCTGGAAGTATGCCTGTATCAGTGTCCGTACAATAAACTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGATGCCAGATGTGAAGTGTCTTGCGACTGGACTTTGATCTAGATGCGAGTCCTTTTAAATGTACTAAAACTGTATTCTCTTTGCTCGAAAAGCGTGACGTTGCTGGTTGTGGAGGCTGCCTGTATGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGTGTTTATGGAGAAGTGTTTAATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTAAATGTTTTTCTTACTGCGGCGGATAGACTGGTGAACCGTAGCTGTGTAGGCTTGGCCTTTGAAYCGGYGATATTGTGCGAAGTAGAGTGACGGCTGTTCCGGCGCAAGCTGGAGTGGCTGTCGAGGGTCGATCCATTTGGGAAATGTTGTGCCTTTGCGACCTCGGTTGCTTTGGCATCTCAA
>AF403506.1 Phytophthora pistaciae
CCACACCTAAAAAACTTTCCACGTGAACCGCATCAACAAGTAGTTGGGGGCCTGCTCTGTGTGGCTGGCTGTCGATGTCAAAGTCGGCGGCTGGCTGCTATGTGGCGGGCTCTATCATGGCGATTGGTTTGGGTCCTCCTCGTGGGGAACTGGATCATGAGCCCACTTTTTAAACCCATTCTTGATTACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCTCTTCCTTCCGTGTAGTCGGTGGATGGAGACGCCAGACGTGGGGTGTCTTGCGGCGCGGCCTTCGGGCTGCCTGCGAGTCCCTTGAAATGTACTAAACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGTTGGTTGTGGAGGCTGCCTGTATGGCCAGTCGGCGACCAGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGATGCTTTTCCTGCTGTGGCGGTATGGGCTGGTGAACCGTAGCTGTGCGAGGCTTGGCTTTTGAACCGGCGGTGTTGTTGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGAGTCGATCCATTTGGGAACTCTGTGTTGTCTCTGCGGCTTGCTGCGGAGGTGGCATCTCAA
>AF408625.2 Phytophthora bisheria
CCACACCTAAAAAAAACTTTCCACGTGAACCGTATCAACCCTTTTAATTGGGGATTTGCTTGGTGTGCGGCTAGCTGGCCTTTATTGGCTGGCAGGCTGCGTGCTGGGCGAGTCCTATCATGGCAAACGTTTGGGCTTCGGTCTGAGCAAGTAGCTTTTTTTAAACCATTTCTTAATACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGATGTGAAGTGTCTTGCGGCTGGCTTTCGGGTCGGCTGCGCGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGTGAGGCGTTGCTGGTTGTGGAGGCTGCCCGCGTGACCAGTCGGCGACCGGTTTGTCTGCTGCGGTGTTTTACTGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGGCGCTTATTGGACGTTTTCCCTGCTGTGGCGTGATGAGCTGGTGAACCGTAGCTATGTAGGCTTGGCTTTTGAACCGGTGATACTATTGCGAAGTAGAGTGGCGGCTTCGGCTGCCGAGGGGTCGATCCATTTTGGGAAACTTTGTGTGTACTTCGGTGCGCATCTCAA
>AF449491.1 Phytophthora europaea
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCATTTAGTTGGGGGCTTGTCCTGGTGGCTGGCTGTCGATGTCAAAGTTGACGGCTGCTGCTGTGTGGCGGGCCCTATCATGGCGAGCGTTTGGGTCCCTCTCGGGGGAACTGAGCCAGTAGCCCTTTTCTTTTAAACCCATTCTTGAATACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCTCTTCCTTCCGTGTAGTCGGTGGATGGGGACGCCAGACGTGAGGTGTCTTGCGGGTGGTCTTCGGGCTGCCTGCGAGTCCCTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGTTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGRACGTTCTTCCTGCTGTGGCGGTACGGATCGGTGAACCGTAGCTGTGCGAGGCTTGGCTTTTGAACCGGCGGTGTTGGTCGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTGGGAACATGTGTCTCTGCGGCGCGCTTCGGTGTGCTGCGGGTGGCATCTCAA
>AF449494.1 Phytophthora psychrophila
CCACACCTAAAAAAACTTTCCACGTGAACTGTATCAACCAACTAGTTGGGGGTCTTGTTTGGCGTGCGGCTGCTTCGGTAGCTGTTGCTAGGCGAGCCCTATCACGGCGAGCGTTTGGACTTCGGTCTGAGCTAGTAGCTATTTTTTAAACCCATTCTTTAATACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGAGACGCCAGACGTGAAGTGTCTTGCGGCGAATTTTCGAATTTGCTGTGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCATGGCGTTGCTGGTTGTGGAGGCTGCCCGCGTGGCCAGTCGGCGACCGGTTTGTCTGCTGTGGCGTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTAGCTTCGGCTGAACAAGCGCTTATTGGATGTTCTTCTTGCTGTGGCGTGATGGACTGGTGAACCGTAGCTATGTGTGGCTTGGCCTTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAAATTTGTGTGTATGACTTCGGTTGTATGCATCTCAA
>AF449495.1 Phytophthora uliginosa
CCACACCTAAAAAAACTTTCCACGTGAACCGTATCAACCCATTTAGTTGGGGGCTTGTCCTGGTGGCTGGCTGTCGATGTCAAAGTTGACGGCTGCTGCTGTGTGGCGGGCCCTATCATGGCGAGCGTTTGGGTCCCTCTCGGGGGAACTGAGCCAGTAGCCCTTTTCTTTTAAACCCATTCTTGAATACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCTCTTCCTTCCGTGTAGTCGGTGGATGGGGACGCCAGACGTGAGGTGTCTTGCGGTCGGTCTTCGGGCTGCCTGCAAGTCCCTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGTTGGTTGTGGAGGCTGCCCGTGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGACGTTCTTCCTGCTGTGGCGGTACGGATCGGTGAACCGTAGCTGTGCGAGGCTTGGCTTTTGAACCGGCGGTGTTGGTCGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTGGGAACTTGTGTCTCTGCGGCGCGCTTCGGTGTGCTGCGGGTGGCATCTCAA
>AF541887.1 Phytophthora gonapodyides
CCACACCTAAAAAAACTTTCCACGTGAACCGTATCAACCCCTATAATTTGGGGGCTTGCTCGGCGGCGTGCGTGCTGGCCTGTAATGGGTCGGCGTGCTGCTGCTGGGCGGGCTCTATCATGGGCGAGCGTTTGGGCTTCGGCTTGAGCTAGTAGCTATCAATTTTAAACCCTTTCTTAAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTGGCGGTCTTTCGAGTCTGCCGGTGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGCGCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACCGGTTTGTTAGCTGTGACGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGGGTGCTTTTCCTGTCATTGGCGGTACGAACTGGTGAACCGTAGCTGTGTGGTGCTTGGCTTTTGAACCGGCTTTGCTTTGCGAAGTAGTGTGACGGCTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAACTTTTGTGTGTGCGGCTTCGTGCTGCGCGCATCTCAA
>AF541888.1 Phytophthora gonapodyides
CCACACCTAAAAAAACTTTCCACGTGAACCGTATCAACCCCTATAATTTGGGGGCTTGCTCGGCGGCGTGCGTGCTGGCCTGTAATGGGTCGGCGTGCTGCTGCTGGGCGGGCTCTATCATGGGCGAGCGTTTGGGCTTCGGCTCGAGCTAGTAGCTATCAATTTTAAACCCTTTCTTAAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTGGCGGTCTTTCGAGTCTGCCGGTGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGCGCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACCGGTTTGTTAGCTGTGACGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGGGTGCTTTTCCTGTCATTGGCGGTACGAACTGGTGAACCGTAGCTGTGTGGTGCTTGGCTTTTGAACCGGCTTTGCTTTGCGAAGTAGTGTGACGGCTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAACTTTTGTGTGTGCGGGTTCGTGCTGCGCGCATCTCAA
>AF541891.1 Phytophthora gonapodyides
CCACACCTAAAAAAACTTTCCACGTGAACCGTATCAACCCCTATAATTTGGGGGCTTGCTCGGCGGCGTGCGTGCTGGCCTGTAATGGGTCGGCGTGCTGCTGCTGGGCGGGCTCTATCATGGGCGAGCGTTTGGGCTTCGGCTCGAGCTAGTAGCTATCAATTTTAAACCCTTTCTTAAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTGGCGGTCTTTCGAGTCTGCCGGTGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGCGCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACCGGTTTGTTAGCTGTGACGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGGGTGCTTTTCCTGTCATTGGCGGTACGAACTGGTGAACCGTAGCTGTGTGGTGCTTGGCTTTTGAACCGGCTTTGCTTTGCGAAGTAGTGTGACGGCTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAACTTTTGTGTGTGCGGCTTCGTGCTGCGCGCATCTCAA
>AF541892.1 Phytophthora gonapodyides
CCACACCTAAAAAAGCTTTCCAAGTGAACCGTATCAACCCCTATAATTTGGGGGCTTGCTCGGCGGCGTGCGTGCTGGCCTGTAATGGGTCGGCGTGCTGCTGCTGGGCGGGCTCTATCATGGGCGAGCGTTTGGGCTTCGGCTCGAGCTAGTAGCTATCAATTTTAAACCCTTTCTTAAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTGGCGGTCTTTCGAGTCTGCCGGTGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGCGCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACCGGTTTGTTAGCTGTGACGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGGGTGCTTTTCCTGTCATTGGCGGTACGAACTGGTGAACCGTAGCTGTGTGGTGCTTGGCTTTTGAACCGGCTTTGCTTTGCGAAGTAGTGTGACGGCTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAACTTTTGTGTGTGCGGCTTCGTGCTGCGCGCATCTCAA
>AF541893.1 Phytophthora megasperma
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCCTTAAATTTGGGGGCTTGCTCGGCGGCGTGTGTGCTGGCCTGTAATGGGTCGGCGTGCTGCTGCTGGGCGGGCTCTATCATGGGCGAGCGTTTGGGCTTCGGCTCGAGCTAGTAGCTATCAATTTTAAACCCTTTCTTAAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGAGTCTTGCTAGCGGTCTTTCGAGTCTGCCGGTGAGTCCTTTTAAATGTACTGAACTGTACTCTCTCTTTGCGCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCCGTGTGGCCAGTCGGCGACCGGTTTGTTAGCTGTGGCGTTTGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGGGTGCTTTTCCTGTCATGGCGGTATGAACTGGTGAACCGTAGCTGTATGGTGCTTGGCTTTTGAACCGGCTTTGCTTTGCGAAGTAGTGTGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAACTTTGTGTGTGCGGCTTCGTGCTGCGCGCATCTCAA
>AF541895.1 Phytophthora megasperma
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCCTTAAATTTGGGGGCTTGCTCGGCGGCGTGTGTGCTGGCCTGTAATGGGTCGGCGTGCTGCTTCTGGGCGGGCTCTATCATGGGCGAGCGTTTGGGCTTCGGCTCGAGCTAGTAGCTATCAATTTTAAACCCTTTCTTAAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTAGCGGTCTTTCGAGTCTGCCGGTGAGTCCTTTTAAATGTACTGAACTGTACTCTCTCTTTGCGCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCCGTGTGGCCAGTCGGCGACCGGTTTGTTAGCTGTGGCGTTTGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGGGTGCTTTTCCTGTCATGGCGGTATGAACTGGTGAACCGTAGCTGTRTGGTGCTTGGCTTTTGAACCGGCTTTGCTTTGCGAAGTAGTGTGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAACTTTGTGTGTGCGGCTTCGTGCTGCGCGCATCTCAA
>AF541896.1 Phytophthora megasperma
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCCTTAAATTTGGGGGCTTGCTCGGCGGCGTGTGTGCTGGCCTGTAATGGGTCGGCGTGCTGCTGCTGGGCGGGCTCTATCATGGGCGAGCGTTTGGGCTTCGGCTCGAGCTAGTAGCTATCAATTTTAAACCCTTTCTTAAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTAGCGGTCTTTCGAGTCTGCCGGTGAGTCCTTTTAAATGTACTGAACTGTACTCTCTCTTTGCGCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCCGTGTGGCCAGTCGGCGACCGGTTTGTTAGCTGTGGCGTTTGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGGGTGCTTTTCCTGTCATGGCGGTATGAACTGGTGAACCGTAGCTGTGTGGTGCTTGGCTTTTGAACCGGCTTTGCTCTGCGAAGTAGTGTGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAACTTTGTGTGTGCGGCTTCGTGCTGCGCGCATCTCAA
>AF541898.1 Phytophthora megasperma
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCCTTAAATTTGGGGGCTTGCTCGGCGGCGTGCGTGCTGGCCTGTAATGGGTCGGCGTGCTGCTGCTGGGCGGGCTCTATCATGGGCGAGCGTTTGGGCTTCGGCTCGAGCTAGTAGCTATCAATTTTAAACCCTTTCTTAAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTAGCGGTCTTTCGAGTCTGCCGGTGAGTCCTTTTAAATGTACTGAACTGTACTCTCTCTTTGCGCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCCGTGTGGCCAGTCGGCGACCGGTTTGTTAGCTGTGGCGTTTGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGGGTGCTTTTCCTGTCATGGCGGTATGAACTGGTGAACCGTAGCTGTGTGGTGCTTGGCTTTTGAACCGGCTTTGCTTTGCGAAGTAGTGTGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAACTTTGTGTGTGCGGCTTCGTGCTGCGCGCATCTCAA
>AF541900.1 Phytophthora taxon pgchlamydo
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCCTTAAATTTGGGGGCTTGCTCGGCGGCGTGCGTGCTGGCCTGTAATGGGTCGGCGTGCTGCTGCTGGGCAGGCTCTATCATGGGCGAGCGTTTGGGCTTCGGCTCGAACTAGTAGCTATCAATTTTAAACCCTTTCTTTAAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTGGCGGTCTTTCGAGTCTGCCGGTGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGCTCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCCGTGTGGCCAGTCGGCGACCGGTTTGTTAGCTGTGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGGGTGCTTTTCCTGTCATTGGCGGTACGAACTGGTGAACCGTAGCTGTGTGGTGCTTGGCTTTTGAACCGGCTTTGCTTTGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAACTTTGTGTGTGCGGCTTCGTGCTGCGCGCATCTCAA
>AF541901.1 Phytophthora taxon pgchlamydo
CCACACCTAAAAAAACTTTCCACGTGAACCGTATCAACCCCTTAAATTTGGGGGCTTGCTCGGCGGCGTGCGTGCTGGCCTGTAATGGGTCGGCGTGCTGCTGCTGGGCAGGCTCTATCATGGGCGAGCGTTTGGGCTTCGGCTCGAACTAGTAGCTATCAATTTTAAACCCTTTCTTTAAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTGGCGGTCTTTCGAGTCTGCCGGTGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGCTCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCCGTGTGGCCAGTCGGCGACCGGTTTGTTAGCTGTGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGGGTGCTTTTCCTGTCATTGGCGGTACGAACTGGTGAACCGTAGCTGTGTGGTGCTTGGCTTTTGAACCGGCTTTGCTTTGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAACTTTGTGTGTGCGGCTTCGTGCTGCGCGCATCTCAA
>AF541903.1 Phytophthora gregata
CCACACCTAAAAAAACTTTCCACGTGAACCGTATCAACCCCTTAAATTTGGGGGCTTGCTCGGCGGCGTGCGTGCTAACCTGTAATGGGTCGGCGTGCTGCTGCTGGGCGGGCTCTATCATGGGCGAGCGTTTGGGCTTCGGCTCGAGCTAGTAGCTTTAATTTTAAACCCATTCTTTAAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTGGCGGTCTTTCGAGTCTGCCGGTGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGCTCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCCGTGTGGCTTGTCGGCGACCGGTTTGTTAGCTGTGACGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGGGTGCTTTTTCCTGTCATTGGCGGTGCGAACTGGTGAACCGTAGTCATGTGGTGCTTGGCTTTTGAATCGGTTTTGCTGTTGCGAAGTAGTGTGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAACTTTGTGTGTGCGGCTTCGTGCTGCGCGCATCTCAA
>AF541904.1 Phytophthora gregata
CCACACCTAAAAAAACTTTCCACGTGAACCGTATCAACCCCTTAAATTTGGGGGCTTGCTCGGCGGCGTGCGTGCTGGCCTGTAATGGGTCGGCGTGCTGCTGCTGGGCGGGCTCTATCATGGGCGAGCGTTTGGGCTTCGGCTCGAGCTAGTAGCTTTAATTTTAAACCCATTCTTTAAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTGGCGGTCTTTCGAGTCTGCCGGTGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGCTCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCCGTGTGGCTTGTCGGCGACCGGTTTGTTAGCTGTGACGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGGGTGCTTTTTCCTGTCATTGGCGGTGCGAACTGGTGAACCGTAGTCATGTGGTGCTTGGCTTTTGAATCGGTTTTGCTGTTGCGAAGTAGTGTGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAACTTTGTGTGTGCGGCTTCGTGCTGCGCGCATCTCAA
>AF541905.1 Phytophthora taxon raspberry
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCCTTAAATTTGGGGGCTTGCTCGGCGGCGTGCGTGCTGGCCTGTAATGGGTCGGCGTGCTGCTGCTGGGCGGGCTCTATCATGGGCGAGCGTTTGGGCTTCGGCTCGAGCTAGTAGCTTTAATTTTAAACCCATTCTTTAAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTGGCGGTCTTTCGAGTCTGCCGGTGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGCTCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCCGTGTGGCTTGTCGGCGACCGGTTTGTTAGCTGTGACGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGGGTGCTTTTTCCTGTCATTGGCGATACGAACTGGTGAACCGTAGTCATGTGGTGCTTGGCTTTTGAACCGGTTTTGCTGTTGCGAAGTAGTGTGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAACTTTGTGTGTGCGGCTTCGTGCTGCGCGCATCTCAA
>AF541908.1 Phytophthora taxon forestsoil
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCCTTAAATTTGGGGGCTTTCTCGGCGGCGTGCGTGCTGGCCTGTAATGGGTCGGCGTGCTGCTGCTGGGCGGGCTCTATCATGGGCGAGCGTTTGGGCTTCGGCTCGAGCTAGTAGCTTTTCATTTTAAACCCATTCTTAAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTGGCGGTCTTTCGAGTCTGCCGGTGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGCTCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCCGTGTGGCCAGTCGGCGACCGGTTTGTTAGCTGTGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGGGTGCTTTTCCTGTCATTGGCGGTACGAACTGGTGAACCGTAGCTGTGTGGTGCTTGGCTTTTGAACCGGCTTTGCTGTTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAACTTTGTGTGTGCGGCTTCGTGCTGCGCGCATCTCAA
>AF541910.1 Phytophthora taxon walnut
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCCTATAGTTTGGGGGCTTGTTTGGCGGCGTGCGTGCTGGCCTGTTACGGGTTGGCGTGCTGCTGCTGGGCGAGCTCTATCATGGGCGACTCGTGTTGGGCTTCGGCTCGAGCGAGTAGCTTTTCCTTTTAAACCCATTCCTAATTACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTGTCGGTCTTTCGAGTCTGTCGGCGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGCTCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCTATATGGCCAGTCGGCGACCGGTTTGTTAGCTGTAGTGTTTACTGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAAATCTGCTTATTGGGCGTTTTTCCTGCTATTGGCGGTACAAACTGGTGAACCGTAGCTGTGTGGTGCTTGGCTTTTGAATCGGCTTTGCTGTTTGCGAAGTAGAGTGGCGGCTTCGGCTGCCGAGGGGTCGATCCATTTTGGGAACTTTTTTGTGTGTGCGGCTTCGTGCTGCGCGCATCTCAA
>AF541911.1 Phytophthora rosacearum
CCACACCTAAAAAAACTTTCCACGTGAACCGTATCAACCCCTATAATTTGGGAGCTTGTTTGGCGGCGTGTGTGCTGGCCTGTTAAGGGTCGGCGTGCTGCTGCTGGGCGGGCTCTATCATGGGCGATCGTTTTGGGCTTCGGCTCGAGCGAGTAGCTTTAATTTTAAACCCATTCTTAATTACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGATGGTGGTCTTTCGAGTCTGCCGGCGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGCTCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCTATATGGCCAGTCGGCGACCGGTTTGTTAGCTGTAGTGTTTACTGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAAATCTGCTTATTGGGTGTTTTTCCTGCTATTGGCGGTACAAACTGGTGAACCGTAGCTGTGTGGTGCTTGGCTTTTGAATCGGCTTTGCTGTTTGCGAAGTAGAGTGGCGGCTTCGGCTGCCGAGGGGTCGATCCATTTTGGGAAATTTTTGTGTGTGCGGCTTCGTGCTGCGCGCATCTCAA
>AF541912.1 Phytophthora inundata
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCCTTAAGATTGGGAGCTTGCTCGGCGGCGTGCGTGCTGGCCTGTAATGGGTCGGCGTGCTGCTGCTGGGCGGGCTCTATCATGGGCGAGCGTTTGGGCTTCGGCTCGAGCTAGTAGCTTTTTCTTTTAAACCCCTTCTTTAATTACTGAAATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTCGTCCTGGAAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTAGCGGTCTTTCGAGTCTGCCGGCGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGCTCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCCGTATGGCCAGTCGGCGACCGGTTTGTTAGCTGTAGTATATACTGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGGGTGCTTTTCCTGCTATTGGCGGTACAAACTGGTGAACCGTAGCTGTGTGGAGCTTGGCTTTTGAATCGGCTTTGCTTGCGAAGTAGAGTGGCGGCTTCGGCTGCCGAGGGGTCGATCCATTTTGGGAAATTATGTGTGTGCGGCTTCGTGCTGCGCGCATCTCAA
>AJ131986 Phytophthora quercina
CCACACCTAAAAAAACTTTCCACGTGAACCGTTTCAACCAAATATTTTGGGGGTCTTGTCTGGCGTATGGCTGCTGCTGTAAAAGGCGGCGGCTGTTGCTGGGTGAGCCCTATCATGGCGAACGTTTGGGCTTCGGTCTGAACAAGTAGCTCTTTTTTAAACCATTACTTATTACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGAGACGCCAGACGTGAAGTGTCTTGCGACTGGTTTTCGGACCGGATGTGAGTCCTTTGAAATGTACTTTACTGTACTTCTCTTTGCTCGAAAAGCGTGGCGTTGCTAATTGTGGAGGCTGCCAGTGTGGCCAGTCGGCGACCGGTTTTGTCTGCTGTAGCGTTTAATGGAGGAGTGTTTGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGACGTTTTTCCTGCTGTGGCGTGATGGACTGGTGAACCGTAGCTGTGCATGGCTTGGCCTTTGAATTGGCTTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAAATTTGTGTTGGTAGCTTCGGTTGCTGGCATCTCAA
>AJ131987 Phytophthora iranica
CCACACCTAAAATATTTCCACGTGAACCGTTTCAACCAAAAAAATTTGGGGGTCTTGTTTGGCGGCGGCTACTGGCTTTATTGCTGGCGGCTGCTGCTGGACGAGCCCTAACGCGAAAGTTTAGACTTCGGTCTGGGCTAGTAGCTTTATTTTAAACCCATTCCTTAATACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGAAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGACTTTCTTCCTTCCGTGTAGTCGGTGGAGGAGATGTCAGATATGAAGTGTCTTGCGGTTGGTTTTCGGACCGACTGCGAGTCCTTTTAAATGTACTAAACTGTACTTCTCTTTGCTCCAAAAGTGGTGGCATTGCTGGTTGTGGAGGCTGTTACTGTGGCCAATTGGCGACCGGTTTGTCTGCTGCGGCGTTAATGGAAGAATGCTTGATTCGCGGTATGATTGGCTTCGGCTGAACAATGCGCTTATTGGGTGTTTATCCTGCTGCGGCGTGATGGACTGGTGAACCATAGCTCGGTGACTTGGCTTTTGAATTGGCTATTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAAATTAGTGTGTACTTCGGTATGCATCTCAA
>AJ131989 Phytophthora clandestina
CCACACCTAAAAATTTCCACGTGAACCGTTTCAACCCAAAAAATTGGGGGTCTTGTTTGGTGGTGGCTGCTGGCTTTATTGCTGGCGGCTGCTGCTGGATGAGCCCTAACGCGAAAGTTTAGACTTCGGTCCGGGCTAGTAGCTTTATTTTAAACCCATTCCTTAATACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGAAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGACTTTCTTCCTTCCGTGTAGTCGGTGGAGGAGATGTCAGATATGAAGTGTCTTGCGGTTGGTTTTCGGACCGACTGCGAGTCCTTTTAAATGTACTAAACTGTACTTCTCTTTGCTCCAAAAGTGGTGGCATTGCTGGTTGTGGAGGCTGTTACTGTGGCCAATTGGCGACCGGTTTGTCTGCTGTGGCGTTAATGGAAGAATGCTTGATTCGCGGTATGATTAGCTTCGGCTGAACAATGCGCTTATTGGGTGTTTATCCTGCTGTGGCGTGATGGACTGGTGAACCATAGCTCGGTGACTTGGCTTTTGAATCGGCTATTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAAATTAGTGTGTACTTCGGTATGCATCTCAA
>AJ131990 Phytophthora ilicis
CCACAACTAAAAATTCCACGTGAACTGTATCGAACAACTAGTTGGGGGTCTTGTTTGGCGTGCGGCTGCTTCGGTAGCTGCTGCTAGGCGAGCCCTATCACGGCGAGCGTTTGGACTTCGGTCTGAGCTAGTAGCTATTTTTTAAACCCATTCTTTAATACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGAGACGCCAGACGTGAAGTGTCTTGCGGCGAATTTTCGAATTTGCTGTGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGCGTTGCTGGTTGTGGAGCCTGCCCGCGTGGCCAGTCGGCGACCGGTTTGTCTGCTGTGGCGTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTAGCTTCGGCTGAACAAGCGCTTATTGGATGTTCTTCTTGCTGTGGCGTGATGGACTGGTGAACCGTAGCTATGTGTGGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAAATTTGTGTGTATGACTTCGGTTGTAAGCATCTCAA
>AY208125.1 Phytophthora tropicalis
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCTTTTAGTTGGGGGTCTTGTACCCTATCTTGGCGAATGTTTGGACTTCGGTCTGGGCGAGTAGCCTTTTGTTTTAAACCCATTTCACAAAACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGATGTGCCAGATGTGAAGTGTCTTGCGTGTTGTCCTTCGGGTCGGCTGCGAGTCCTTTTAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGTGTTGCTGGTTGTGGAGGCTGCCTGCGCGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTTGCGGTATGGTTGGCTTCGGCTGAACAGGCGCTTATTGTATGCTTTTCCTGCTGTGGCGTGATGGGCTGGTGAACCGTAGCTGTGTGTGGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTTGGGAACTTTGTGTGCACTTCGGTGCGCATCTCAA
>AY230190.1 Phytophthora pseudosyringae
CCACACCTAAAAAACTTTCCACGTGAACTGTATCGAACAACTAGTTGGGGGTCTTGTTTGGCGTGCGGCTGCTTCGGTAGCTGCTGCTAGGCGAGCCCTATCACGGCGAGCGTTTGGACTTCGGTCTGAGCTAGTAGCTATTTTTTAAACCCATTCTTTAATACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGAGACGCCAGACGTGAAGTGTCTTGCGGCGAATTTTCGAATTTGCTGTGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCCGCGTGGCCTGTCGGCGACCGGTTTGTCTGCTGTGGCGTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTAGCTTCGGCTGAACAAGCGCTTATTGGATGTTCTTCTTGCTGTGGCGTGATGGACTGGTGAACCGTAGCTATGTGTGGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAAATTTGTGTGTATGACTTCGGTTGTATGCATCTCAA
>AY242980.1 Phytophthora pseudosyringae
CCACACCTAAAAAACTTTCCACGTGAACTGTATCGAACAACTAGTTGGGGGTCTTGTTTGGCGTGCGGCTGCTTCGGTAGCTGCTGCTAGGCGAGCCCTATCACGGCGAGCGTTTGGACTTCGGTCTGAGCTAGTAGCTATTTTTTAAACCCATTTTTTAATACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGAGACGCCAGACGTGAAGTGTCTTGCGGCGAATTTTCGAATTTGCTGTGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCCGCGTGGCCTGTCGGCGACCGGTTTGTCTGCTGTGGCGTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTAGCTTCGGCTGAACAAGCGCTTATTGGATGTTCTTCTTGCTGTGGCGTGATGGACTGGTGAACCGTAGCTATGTGTGGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAAATTTGTGTGTATGACTTCGGTTGTATGCATCTCAA
>AY332654.1 Phytophthora nemorosa
CCACACCTAAAAAACTTTCCACGTGAACTGTATCGAACAACTAGTTGGGGGTCTTGTTTGGCGTGCGGCTGCTTCGGTAGCTGCTGCTAGGCAAGCCCTATCACGGCGAGCGTTTGGACTTCGGTCTGAGCTAGTAGCTATTTTTTAAACCCATTCTTTAATACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGAGACGCCAGACGTGAAGTGTCTTGCGGCGAATTTTCGAATTTGCTGTGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCCGCGTGGCCAGTCGGCGACCGGTTTGTCTGCTGTAGCGTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTAGCTTCGGCTGAACAAGCGCTTATTGGATGTTCTTCTTGCTGTGGCGTGATGGACTGGTGAACCGTAGCTATGTGTGGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAAATTTGTGTGTATGACTTCGGTTGTATGCATCTCAA
>AY369368.1 Phytophthora ramorum
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAAAACCCTTAGTTGGGGGCTTCTGTTCGGCTGGCTTCGGCTGGCTGGGCGGCGGCTCTATCATGGCGAGCGCTTGAGCCTTCGGGTCTGAGCTAGTAGCCCACTTTTTAAACCCATTCCTAAATACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGCCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGACGTGCAGACGTGAAGTGTCTTGCGATTGGTCTTCGGGCCGGCTGCGAGTCCTTTGAAATGTACAGAACTGTACTTCTCTTTGCTCGAAAAGCATGACGTTGTTGGTTGTGGAGGCTGCCCGTGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTAGCTTCGGCTGAACAATGCGCTTATTGGATGCTTTTTCTGCTGTGGCGGTAATGACTGGTGAACCGTAGCTGTGCAGGGCTTGGCTTTTGAATCGACGGTGTTGTGCGAAGTAGAGTGGCGGTTCGGCGCAAGCTGGGCTGTCGAGGGTCGATCCATTTGGGAAACTTGTGTTGGCGGCTTCGGCTGCTGGCATCTCAA
>AY369370.1 Phytophthora hibernalis
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCTTTTAGTTGGGGGCTTCTGTTCGGCTGGCTTTTGCTGGCTGGGCGGCGGCTCTATCATGGCGAGCGCCTGGGCCTTCGGGTCTGAGCTAGTAGTCTTCTTTTAAACCCTTTCTTAAATACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGCCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGACGTGCAGACGTGAAGTGTCTTGCGATTGGTCTTCGGGCCGGCTGCGAGTCCTTTTAAATGTACAGAACGGTACTTCTCTTTGCTCGAAAAGCATAATGGAATTGGTTGTGGAAGCTTCCCGGTGGCAAGTCGGCGACTGGTTTGTCTGCTACGGCGTTTAATGGAGGAATGTTCGATTCGCGGTATGGTTAGCTTCGGCTGAACAATGCGCTTATTGGATGTTTTTCCTGCTGTGGTGGTAATGACTGGTGAACCGTAGCTATGCAGGGATTGGCCCTTGAACTGAGGATGTTGTGTGAAGTAGAGTGGCGGTTTGGCGCAAGCTGGGCTGTCGAGGGTCGATCCTATTTGGGAAATTTGTGTTGGCGGCTTCGGCTGTTGGCATCTCAA
>AY659736.1 Phytophthora parsiana
CCACACCTAAAAACTTTCCACGTGAACTGTCTGTGATGTTGGGGGCTGTCTGCCTGTTGGCGGCGGCTCCATCAAACGAGGCTCTGGGCTGCAAAGTCGAGGGTAGTAGTTACTTTTTGTAAACCCTTCCTATATTTTCTGATCATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTATGCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGCTTCCTTCCTTCCGTGTAGTCGGTGGCGGGGACGCGCAGACGTGAAGTGTCTTGCTGGCCGTTCGGCCGGCGAGTCCTTTGAAATGTAAGATACTGTTCTTCTCTTTGCTGGAAAAGCGTGCGTTGTGTGGTTGTGGAGGCTGCCGTTATGGCCAGTCGGCGACTGATTTCGTGCTGAGGCGTGTGGAAAGGAGCTCGATTCGCGGTATGGTTGGCTTCGGCTGAACTTCTGCTTATGGGGCTGTTTTTCTGCTGTTTCGGCGTTGGCGGGGTCGGTGAACCGTAGTCATATTGGCTTGGCTTTTGAACCGTGCGACTAGTGCGCGAAGTATGGTGGCTTCGGCCGAGAGGACGACCTATTTGGGACGATTGTGCGGCTTCGTGCTGCATCTCAA
>AY659737.1 Phytophthora parsiana
CCACACCTAAAAACTTTCCACGTGAACTGTCTGTGATGTTGGGGGCCTGTCTGCCTGTTGGCGGGCGGCTCCATCAAACGAGGCTCTGGGCTGAAAAGTCGAGGGTAGTAGTTACTTTTTGTAAACCCTTACTCAACCTTTCTGATCATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTACGCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGCGGGGACGCGCAGACGTGAAGTGTCTTGCTGGGCTTTGTGGCCGGCGAGTCCTTTGAAATGTAAGATACTGTTCTTCTCTTTGCTGGAAAAGCGTGCGCTGTGCGGTTGTGGAGGCTGCCGTTGTGGCCAGTCGGCGACTGATTTCGTGCTGAGGCGTGTGGAGAGGAGCTCGATTCGCGGTATGGTTGGCTTCGGCTGAACTTCTGCTTATGGGGCTGCTTGTCTGCTGCTTTGGCGTGGCGGGGTCGGTGAACCGTAGTCATTTCGGCTTGGCTTTTGAACTGCGCGGGCACTGCGCGAAGTATGGTGGCTTCGGCCGAGAGCACGACCTATTTGGGACGAGTGTGCGGTCTTCGTGCGCATATCAA
>AY659738.1 Phytophthora parsiana
CCACACCTAAAAACTTTCCACGTGAACTGTCTGTGATGTTGGGGGCGTGCCGTTCGCGGCGGCTCCATCAAACGAGGCTCTGGGCTGAAAAGTCGAGGGTAGTAGTTACTTTTTGTAAACCCTTTACCACATTTTCTGATCATACTGCGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTATGCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGCTTCCTTCCTTCCGTGTAGTCGGTGGCGGGAACGCTCAGACGTGAAGCGTCTTGCTGGCCTGTCTGGCCGGCGAGTCCTTTGAAATGTAAGATACTGTTCTTCTCTTTGCTGGAAAAGCGTGCGTTGTGCGGTTGTGGAGGCTGCCGTTATGGCCAGTCGGCGACTGATTTCGTGCTGAGGCGTGTGGAGAGGAGCTCGATTCGCGGTATGGTTGGCTTCGGCTGAACTTCTGCTTATGGGGCTGCTTGTCTGCCGTTTCGGCGTTGCGGGGTCGGTGAACCGTAGTCATGGTGGCTTGGCTTTTGAACCGTGCGACTGCTGCGCGAAGTATGGTGGCTTCGGCCGAGAGGACGACCTATTTGGGACGATTGTGCGGCTTCGTGCTGCATCTCAA
>AY659739.1 Phytophthora parsiana
CCGCACCTAAAAACTTTCCACGTGAACTGTCTGTGATGTTGGGGGCCTGTCTGCCTGTTGGCGGGCGGCTCCATCAAACGAGGCCTTGGGCTGAAAAGTCGAGGGTAGTAGTTACTTTTTGTAAACCCTTACCCAACCTTTCTGATCATACTGCGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTACGCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGCGGGGACGCGCAGACGTGAAGTGTCTTGCTGGGCTTTGTGGCCGGCGAGTCCTTTGAAATGTAAGATACTGTTCTTCTCTTTGCTGGAAAAGCGTGCGCTGTGCGGTTGTGGAGGCTGCCGTTGTGGCCAGTCGGCGACTGATTTCGTGCTGAGGCGTGTGGAGAGGAGCTCGATTCGCGGTATGGTTGGCTTCGGCTGAACTTCTGCTTATGGGGCTGCTTGTCTGCTGCTTTGGCGTGGCGGGGTCGGTGAACCGTAGTCATTTCGGCTTGGCTTTTGAACTGCGCGGGCGCTGCGCGAAGTATGGTGGCTTCGGCCGAGAGGACGACCTATTTGGGACGATTGTGCGGCTTCGTGCTGCATCTCAA
>AY659740.1 Phytophthora parsiana
CCACACCTAAAAACTTTCCACGTGAACTGTCTGTGAGGTTGGGGGCTGTCTGCCTGTTGGCGGCGGCTCCATCAAACGAGGCTCTGGGCTGCAAAGTCGAGGGTAGTAGTTACTTTTTGTAAACCCTTTACTATATTTTCTGATCATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTATGCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGCTTCCTTCCTTCCGTGTAGTCGGTGGCGGGAACGCTCAGACGTGAAGTGTCTTGCTGGCCGTTCCGGCCGGCGAGTCCTTTGAAATGTAAGATACTGTTCTTCTCTTTGCTGGAAAAGCGTGCGTTGTGCGGTTGTGGAGGCTGCCGTTATGGCCAGTCGGCGACTGATTTCGTGCTGAGGCGTGTGGAGAGGAGCTCGATTCGCGGTATGGTTGGCTTCGGCTGAACTTCTGCTTATGGGGCTGTTTTTCTGCTGTTTCGGCGTGGCGGGGTCGGTGAACCGTAGTCATAGTGGCTTGGCTTTTGAACCGTGCGACTAGTGCGCGAAGTATGGTGGCTTCGGCCGAGAGGACGACCTATTTGGGACGATTGTGCGGCTTCGTGCTGCATCTCAA
>AY659741.1 Phytophthora parsiana
CCACACCTAAAAACTTTCCACGTGAACTGTCTGTGATGTTGGGGGCGTGTCGTTCGCGGCGGCTCCATCAAACGAGGCTCTGGGCTGCAAAGTCAAGGGTAGTAGTTACTTTTTGTAAACCCTCTACCACTTTTTCTGATCATACTGCGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTATGCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGCTTCCTTCCTTCCGTGTAGTCGGTGGCGGGAACGCTCAGACGTGAAGCGTCTTGCTGGGCTGTGAGGCCGGCGAGTCCTTTGAAATGTAAGATACTGTTCTTCTCTTTGCTGGAAAAGCGTGCGTTGTGCGGTTGTGGAGGCTGCCGTTATGGCCAGTCGGCGACTGATTTCGTGCTGAGGCGTGTGGAGAGGAGCTCGATTCGCGGTATGGTTGGCTTCGGCTGAACTTCTGCTTATGGGGCTGCTTGTCTGCTGTTTCGGCGTTGCGGGGTCGGTGAACCGTAGTCATGGTGGCTTGGCTTTTGAACCGTGCGATTGCTGCGCGAAGTATGGTGGCTTCGGCCGAGAGGACGACCTATTTGGGACGATTGTGCGGCTTCGTGCTGCATCTCAA
>AY689131.1 Phytophthora alni subsp. alni
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCACTTAGTTGGGGGCYWGTCCYGGCGGCTGGCTGTCGATGTCAAAGTTGACGGCTGCTGCTGTGTGTCGGGCCCTATCATGGCGAGCGTTTGGGTCCCTCTCGGGGGAACTGAGCCAGTAGCCCTTWTCTTTTAAACCCATTCTTGAATACTGAAYATACTGYGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCTCTTCCTTCCGTGTAGTCGGTGGATGGRGACGCCAGACGTGAGGTGTCTTGCGGGWGGTCTTCGGGCTGCCTGCGAGTCCCTTGAAATGTACTGAACTGTACTTCTCTTTGCTYGAAAAGCGTGACGTTGTTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGACGTTCTTCCTGCTGTGGCGGTACGGATCGGTGAACCGTAGCTGTGCGRRGCTTGGCCTTTGAACCGGCGGTGTTGGTCGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTGGGAACTTGTGTCTCTGCGGCGCGCTTCGGTGTGCTGCGGGTGGCATCTCAA
>AY689134.1 Phytophthora alni subsp. uniformis
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCACTTAGTTGGGGGCTAGTCCCGGCGGCTGGCTGTCGATGTCAAAGTTGACGGCTGCTGCTGTGTGTCGGGCCCTATCATGGCGAGCGTTTGGGTCCCTCTCGGGGGAACTGAGCCAGTAGCCCTTTTCTTTTAAACCCATTCTTGAATACTGAATATACTGCGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCTCTTCCTTCCGTGTAGTCGGTGGATGGGACGCCAGACGTGAGGTGTCTTGCGGGTGGTCTTCGGGCTGCCTGCGAGTCCCTTGAAATGTACTGAACTGTACTTCTCTTTGCTTGAAAAGCGTGACGTTGTTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGACGTTCTTCCTGCTGTGGCGGTACGGATCGGTGAACCGTAGCTGTGCGGAGCTTGGCCTTTGAACCGGCGGTGTTGGTCGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTGGGAACTTGTGTCTCTGCGGCGCGCTTCGGTGTGCTGCGGGTGGCATCTCAA
>AY689136.1 Phytophthora alni subsp. multiformis
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCACTTAGTTGGGGGCTAGTCCCGGCGGCTGGCTGTCGATGTCAAAGTTGACGGCTGCTGCTGTGTGTCGGGCCCTATCATGGCGAGCGTTTGGGTCCCTCTCGGGGGAACTGAGCCAGTAGCCCTTATTTTTTAAACCCATTCTTGAATACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCTCTTCCTTCCGTGTAGTCGGTGGATGGGGACGCCAGACGTGAGGTGTCTTGCGGGTGGTCTTCGGGCTGCCTGCGAGTCCCTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGTTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGACGTTCTTCCTGCTGTGGCGGTACGGATCGGTGAACCGTAGCTGTGCGRRGCTTGGCCTTTGAACCGGCGGTGTTGGTCGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTGGGAACTTGTGTCTCTGCGGCGCGCTTCGGTGTGCTGCGGGTGGCATCTCAA
>AY707987.1 Phytophthora hedraiandra
CCACACCTAAAAACTTTCCACGTGAACCGTTTCAAACCAAATAGTTGGGGGTCTTGTCTGGTGGCGGCTGCTGACTTTATTGTTGGCGGCTGCTGCTGGACGAGCCCTATCATGGCGAGCGTTTGGGCTTCGGCCTGAGCTAGTAGCTTTTCTTTTAAACCCATTCCTTAATACTGATTATACTGTGGGGACGAAAGTCCTTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGAGATGCCAGATGTGAAGTGTCTTGCGGCTGGTTTTCGGACCGACTGCGAGTCCTTTTAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCTATTGTAGCAAGTTGGCGACCGGTTTGTCTGCTGCGGCGTTAATGGAAGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGATGTTTTTTCTGCTGTGGCGTGATGGACCGGTGAACCATAGCTCAGTGGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAAATGTGTGTGTACTTCGGTATGCATCTCAA
>AY770742.1 Phytophthora ipomoeae
CCACACCTAAAAACTTTCCACGTGAACCGTTTCAACCCAATAGTTGGGGGTCTTACTTGGCGGCGGCTGCTGGCTTTATTGCTGGCGGCTACTGCTGGGCGAGCCCTATCAAAAGGCGAGCGTTTGGACTTCGGTCTGAGCTAGTAGCTTTTTTATTTTAAACCCTTTACTTAATACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGAAGTATGCCTGTATCAGTGTCCGTACAATAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGAGATGCCAGATGTGAAGTGTCTTGCGGTTGGTTTTCGGACCGACTGCGAGTCCTTTTAAATGTACTAAACTGTACTTCTCTTTGCTCCAAAAGTGGTGGCATTGCTGGTTGTGGACGCTGCTATTGTAGCGAGTTGGCGACCGGTTTGTCTGCTGCGGCGTTAATGGAGAAATGCTCGATTCGTGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGGTGATTTTCCTGCTGTGGCGTGATGGACTGGTGAACCATGGCTCTTTAGCTTGGCATTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGCCGAGGGTCGATCCATTTGGGAAATGTTGTGTACTTCGGTATGCATCTCAA
>AY787025.1 Phytophthora gonapodyides
CCACACCTAAAAAAACTTTCCACGTGAACCGTATCAACCCCTATAATTTGGGGGCTTGCTCGGCGGCGTGCGTGCTGGCCTGTAATGGGTCGGCGTGCTGCTGCTGGGCGGGCTCTATCATGGGCGAGCGTTTGGGCTTCGGCTCGAGCTAGTAGCTATCAATTTTAAACCCTTTCTTAAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTGGCGGTCTTTCGAGTCTGCCGGTGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGCGCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACCGGTTTGTTAGCTGTGACGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGGGTGCTTTTCCTGTCATTGGCGGTACGAACTGGTGAACCGTAGCTGTGTGGTGCTTGGCTTTTGAACCGGCTTTGCTTTGCGAAGTAGTGTGACGGCTTYGGCTGTCGAGGGGTCGATCCATTTTGGGAAACTTTTGTGTGTGCGGCTTCGTGCTGCGCGCATCTCAA
>AY787028.1 Phytophthora cambivora
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCACTTAGTTGGGGGCTAGTCCCGGCGGCTGGCTGTCGATGTCAAAGTTGACGGCTGCTGCTGTGTGTCGGGCCCTATCATGGCGAGCGTTTGGGTCCCTCTCGGGGGAACTGAGCCAGTAGCCCTTATTTTTTAAACCCATTCTTGAATACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCTCTTCCTTCCGTGTAGTCGGTGGATGGGGACGCCAGACGTGAGGTGTCTTGCGGGTGGTCTTCGGGCTGGCCTGCGAGTCCCTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGTTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCCTCGGCTGAACAATGCGCTTATTGGACGTTCTTCCTGCTGTGGCGGTACGGATCGGTGAACCGTAGCTGTGCGAGGCTTGGCCTTTGAACCGGCGGTGTTGGTCGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTGGGAACTTGTGTCTCTGCGGCGCGCTTCGGTGTGCTGCGGGTGGCATCTCAA
>AY787029.1 Phytophthora cambivora
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCACTTAGTTGGGGGCTAGTCCCGGCGGCTGGCTGTCGATGTCAAAGTTGACGGCTGCTGCTGTGTGTCGGGCCCTATCATGGCGAGCGTTTGGGTCCCTCTCGGGGGAACTGAGCCAGTAGCCCTTATTTTTTAAACCCATTCTTGAATACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCTCTTCCTTCCGTGTAGTCGGTGGATGGGGACGCCAGACGTGAGGTGTCTTGCGGGTGGTCTTCGGGCTGCCTGCGAGTCCCTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGTTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGACGTTCTTCCTGCTGTGGCGGTACGGATCGGTGAACCGTAGCTGTGCGAGGCTTGGCCTTTGAACCGGCGGTGTTGGTCGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTGGGAACTTGTGTCTCTGCGGCGCGCTTCGGTGTGCTGCGGGTGGCATCTCAA
>AY787030.1 Phytophthora cambivora
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCACTTAGTTGGGGGCTAGTCCCGGCGGCTGGCTGTCGATGTCAAAGTTGACGGCTGCTGCTGTGTGTCGGGCCCTATCATGGCGAGCGTTTGGGTCCCTCTCGGGGGAACTGAGCCAGTAGCCCTTATTTTTTAAACCCATTCTTGAATACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCTCTTCCTTCCGTGTAGTCGGTGGATGGGGACGCCAGACGTGAGGTGTCTTGCGGGTGGCCTTCGGGCTGCCTGCGAGTCCCTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGTTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGACGTTCTTCCTGCTGTGGCGGTACGGATCGGTGAACCGTAGCTGTGCGAGGCTTGGCCTTTGAACCGGCGGTGTTGGTCGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTGGGAACTTGTGTCTCTGCGGCGCGCTTCGGTGTGCTGCGGGTGGCATCTCAA
>AY787031.1 Phytophthora cambivora
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCACTTAGTTGGGGGCTAGTCCCGGCGGCTGGCTGTCGATGTCAAAGTTGACGGCTGCTGCTGTGTGTCGGGCCCTATCATGGCGAGCGTTTGGGTCCCTCTCGGGGGAACTGAGCCAGTAGCCCTTATTTTTTAAACCCATTCTTGAATACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCTCTTCCTTCCGTGTAGTCGGTGGACGCGGACGCCAGACGTGAGGTGTCTTGCGGGTGGCCTTCGGGCTGCCTGCGAGTCCCTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGTTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGACGTTCTTCCTGCTGTGGCGGTACGGATCGGTGAACCGTAGCTGTGCGAGGCTTGGCCTTTGAACCGGCGGTGTTGGTCGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTGGGAACTTGTGTCTCTGCGGCGCGCTTCGGTGTGCTGCGGGTGGCATCTCAA
>AY787032.1 Phytophthora syringae
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAAAACCCTTTTATTGGGGGCTTCTGTCTGGTCTGGCTTCGGCTGGATTGGGTGGCGGCTCTATCATGGCGACCGCTCTGAGCTTCGGCCTGGAGCTAGTAGCCCACTTTTTAAACCCATTCTTAATTACTGAACAAACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGATGCACAGACGTGAAGTGTCTTGCGACTGGGCTTCGGCTCGGCTGCGAGTCCTTTTAAATGTACAGAACTGTACTTCTCTTTGCTCGAAAAGCGTTATATTACTGGTTGTGGAGGCTGCCTGTGCGGCAAGTCGGCGACCGGTTTGTTAACTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGATGGCTTCGGCTGAACTGACGCTTATTGAGTACTTTTCCTGCTGTGGTGGTACGAACTGGTGAACCGTAGCTGTGTTTGGCTTGGCTTTTGAACTGGCGATGTGGTGCGAAGTAGAGTGACGGTTGTTCCGGCGCAAGCTGGAGTGACTGTCGAGGGTCGATCCATTTGGGAAATTTTGTGTCTGTGCGACTTCGGTTGCGTGGGCATCTCAA
>AY787033.1 Phytophthora syringae
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAAAACCCTTTTATTGGGGGCTTCTGTCTGGTCTGGCTTCGGCTGGATTGGGTGGCGGCTCTATCATGGCGACCGCTCTGAGCTTCGGCCTGGAGCTAGTAGCCCACTTTTTAAACCCATTCTTAATTACTGAACAAACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGATGCACAGACGTGAAGTGTCTTGCGACTGGGCTTCGGCTCGGCTGCGAGTCCTTTTAAATGTACAGAACTGTACTTCTCTTTGCTCGAAAAGCGTTATATTACTGGTTGTGGAGGCTGCCTGTGCGGCAAGTCGGCGACCGGTTTGTTAACTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGATGGCTTCGGCTGAACTGACGCTTATTGAGTACTTTTCCTGCTGTGGCGGTACGAACTGGTGAACCGTAGCTGTGTTTGGCTTGGCTTTTGAACTGGCGATGTGGTGCGAAGTAGAGTGACGGTTGTTCCGGCGCAAGCTGGAGTGACTGTCGAGGGTCGATCCATTTGGGAATTTTGTGTCTGTGCGACTTCGGTTGCGTGGGCATCTCAA
>AY787034.1 Phytophthora syringae
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAAAACCCTTTTATTGGGGGCTTCTGTCTGGTCTGGCTTCGGCTGGATTGGGTGGCGGCTCTATCATGGCGACCGCTCTGAGCTTCGGCCTGGAGCTAGTAGCCCACTTTTTAAACCCATTCTTAATTACTGAACAAACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGATGCACAGACGTGAAGTGTCTTGCGACTGGGCTTCGGCTCGGCTGCGAGTCCTTTTAAATGTACAGAACTGTACTTCTCTTTGCTCGAAAAGCGTTATATTACTGGTTGTGGAGGCTGCCTGTGCGGCAAGTCAGCGACCGGTTTGTTAACTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGATGGCTTCGGCTGAACTGACGCTTATTGAGTACTTTTCCTGCTGTGGCGGTACGAACTGGTGAACCGTAGCTGTGTTTGGCTTGGCTTTTGAACTGGCGATGTGGTGCGAAGTAGAGTGACGGTTGTTCCGGCGCAAGCTGGAGTGACTGTCGAGGGTCGATCCATTTGGGAAATTTTGTGTCTGTGCGACTTCGGTTGCGTGGGCATCTCAA
>AY940661 Phytophthora kernoviae
CCACACCTAAAAAACTTTCCACGTGAACCGTTAAAAAACTAGTTGGGGGCTCCTCGTTGGCAGTTTCGACTGTTAGTGGGTTGAGTTCTATCAAACGATCGATTTGGGCTGCAAAGTCGGAATTGAGAGTTTCTTTTTTAAACCCATTACTTAATTACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTATGCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGCTTCCTTCCTTCCGTGTAGTCGGTGGATGGGAATGCCAGACGTGAAGTGTCTTGCTGTAGCGGTTTTCGGACTGTGGCGGCGAGTCCTTTGAAATGTAGATACTGTTCTTCTCTTTGCTCGAAAAGCTGTGCAGAATGTAGTTGTGGAGGCTGCCATCACGGCATGTCGGCGACTAATTTCGTGCTGAGGCGTTTTGGAGAGGGGTTCGATTCGCGGTAGTGTTGGTTTCGGCCGAACAATCTGCTTATTGTGTCTTTTTCCTTGCTTTGGCGTTTGCGAAGTTGGTGTACCGTAGTAGTGTGTAGCTTGGCTTTTGAACGAAACTTTGTGTGCGAAGTAGAGTGGCTAAGTGTTGTTTTCGGACAATGCGGTGCCGAGGGGACGAAACCTTTTGGGAACAATTTTTTGTGTGCACGCTTTCGGGCGTTGTGCATCTCAA
>DQ286726 Phytophthora gallica
CCACACCTAAAAAACTTTCCACGTGAACCGTATTACCACCCAGTTGGGGACTCTCTGTTGGTGGTTGGGGCTGGCTAGCGCTGGCTTTGACTGTCGATGGGTTGAGACCTATCAAACGACGCGTTTGGGCTGCAAGGCCTGAGTGTTGTCTGTTTCTCTTGTAAACCCATTTCTAAAAACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTATGCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGCTTCCTTCCTTCCGTGTAGTCGGTGGATGGGAATGCCAGACGTGAAGTGTCTTGCAGCTGTGTTGTCTTCGGACAATGTGTGCGTGAGTCCTTTGAAATGTAGATACTGTTCTTCTCTTTGCTTGAAAAGCTTTCGAGATGTGGTTGTGGAGGCTGCCATCATGGCAAGTCGGCGACTGATTTCGTGCTGAGGCGTTTTGGAGAGGGCTTGATTCGCGGTAGTGTTGGCTTCGGCTGAACAATCTGCTTATTGTGCTTTTTTCCTTGTCTTGGCGTATGCGGGGTCGGTGAACCGTAGTCGTGTGTGGCTTGGCTTTTGAACCGTGTTTTGTGTGCGAAGTAGAGTGGCTGCAGCAATGTGGCCGAGGGGACGATACCATTTGGGAACTTTTGTGTCAGTGTTTTCGGACGTTGGCATCTCAA
>DQ297392.1 Phytophthora fallax
CCACACCTTAAAAAATTCCACGTGAACTGTATTGTCAACCAAATTCGGGGATTCCTTGCTAGCGTGCCTTCGGGCGTGCCGGTAGGTTGAGACCCATCAAACGAAAACATCGGCTGAAAGGTCGGAGCCAGTAGTTACCTTTGTAAACCCTTTACTAAATACTGAAAAACTGTGGGGACGAAAGTCCTTGCTTTTACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTATGCCTGGAAGTATGCCTGTATCAGTGTCCGTATATCAAACTTGCCTTCCTTCCTTCCGTGTAGTCGGTGGATGGGAACGTGCAGATGTGAAGTGTCTTGGCGGCGTCGCTTCGGCGGTGCTGGCGAGTCCTTTTAAATACAAGATACTGTTCTTCTCTTTGCTCGAAAAGCGTGTGGGATGTGGTTGTGGAGGCTGCCTGCGTGGCCAGTCGGCGACTGATTTCGTGCTGAGACGGTGGGGAGGGGCTTGATTCGCGGTATGGTTGGCTTCGGCTGAACTTCTGCTTATTGGGCTTCTTTCATGTCTTGGCGTATGCGATTTCGGTGTACCGTAGCTGTGTGTGGCTTGGCTTTTGAACCGTGTTCTGTGTGCGAAGTAGAGTGGGCATGTGGTTTTCGGACTGCGTGGCCGAGGGGACGAGACCTTTTGGGAAATTGTGCTGCTACTTTATTGTGGCGGCATCTCGA
>DQ297398.1 Phytophthora fallax
CCACACCTAAAAAAATTCCACGTGAACTGTATTGTCAACCAAATTCGGGGATTCCTTGCTAGCGTGCCTTCGGGCGTGCCGGTAGGTTGAGACCCATCAAACGAAAACATCGGCTGAAAGGTCGGAGCCAGTAGTTACCTTTGTAAACCCTTTACTAAATACTGAAAAACTGTGGGGACGAAAGTCCTTGCTTTTACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTATGCCTGGAAGTATGCCTGTATCAGTGTCCGTATATCAAACTTGCCTTCCTTCCTTCCGTGTAGTCGGTGGATGGGAACGTGCAGATGTGAAGTGTCTTGGCGGCGTCGCTTCGGCGGTGCTGGCGAGTCCTTTTAAATACAAGATACTGTTCTTCTCTTTGCTCGAAAAGCGTGTGGGATGTGGTTGTGGAGGCTGCCTGCGTGGCCAGTCGGCGACTGATTTCGTGCTGAGACGGTGGGGAGGGGCTTGATTCGCGGTATGGTTGGCTTCGGCTGAACTTCTGCTTATTGGGCTTCTTTCATGTCTTGGCGTATGCGATTTCGGTGTACCGTAGCTGTGTGTGGCTTGGCTTTTGAACCGTGTTCTGTGTGCGAAGTAGAGTGGGCATGTGGTTTTCGGACTGCGTGGCCGAGGGGACGAGACCTTTTGGGAAATTGTGCTGCTACTTTATTGTGGCGGCATCTCGA
>DQ297399.1 Phytophthora captiosa
CCACACCTTAAAAAATTCCACGTGAACTGTATTGTCAACCAAATTCGGGGACTCCTTGCTAGTGTGCTTTCGGGCGTGCCGGTAGGTTGAGACCCATCAAACGAAAACATCGACTGAAAGGTCGGAGCCAGTAGTTACCTTTGTAAACCCTTTACTAAATACTGAAAAACTGTGGGGACGAAAGTCCTTGCTTTTACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTATGCCTGGAAGTATGCCTGTATCAGTGTCCGTATATCAAACTTGCCTTCCTTCCTTCCGTGTAGTCGGTGGATGGGAACGTGCAGATGTGAAGTGTCTTGGCGGCGTCGCTTCGGCGGTGCTGGCGAGTCCTTTTAAATACAAGATACTGTTCTTCTCTTTGCTCGAAAAGCGTGTGGGATGTGGTTGTGGAGGCTGCCTGCGTGGCCAGTCGGCGACTGATTTCGTGCTGAGACGGTGGGGAGGGGCTTGATTCGCGGTATGGTTGGCTTCGGCTGAACTTCTGCTTATTGGGCTTCTTTCATGTCTTGGCGTATGCGATTTTGGTGTACCGTAGCTGTGTGTGGCTTGGCTTTTGAACCGTGTTCTGTGTGCGAAGTAGAGTGGGCGTGTGGTTTTCGGACTGCGTGGCCGAGGGGACGAGACCTTTTGGGAAATTGTGCTGCTACTTTATTGTGGCGGCATCTCGA
>DQ302411 Phytophthora bisheria
CCACACCTAAAAAAAACTTTCCACGTGAACCGTATCAACCCTTTTAATTGGGGATTTGCTTGGTGTGCGGCTAGCTGGCCTTTATTGGCTGGCAGGCTGCGTGCTGGGCGAGTCCTATCATGGCAAACGTTTGGGCTTCGGTCTGAGCAAGTAGCTTTTTTTAAACCATTTCTTAATACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGATGTGAAGTGTCTTGCGGCTGGCTTTCGGGTCGGCTGCGCGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGTGAGGCGTTGCTGGTTGTGGAGGCTGCCCGCGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGTGTTTTACTGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGGCGCTTATTGGACGTTTTCCCTGCTGTGGCGTGATGAGCTGGTGAACCGTAGCTATGTAGGCTTGGCTTTTGAACCGGTGATACTATTGCGAAGTAGAGTGGCGGCTTCGGCTGCCGAGGGGTCGATCCATTTTGGGAAACTTTGTGTGTACTTCGGTGCGCATCTCAA
>DQ313223 Phytophthora quercetorum
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAAACTTTAGTTGGGGGTCTTTTTCGGCGACGGCTGCTGGCTTCATTGCTGGCGGCTGCTGTTGGATGAGCCCTATCATGGCGAGCGTTTGGGCTTCGGTCTGAACTAGTAGCTTTTTTAAACCCATTCTTTATTACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGATGTGCCAGATGTGAAGTGTCTTGCGGCTGGTCTTCGGGTCGGCTGTGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGCGTTGCTGATTGTGGAGGCTGCCTGCGTAGCCAGTCTGGCGACCAGTTTGTCTGCTGTGGCGTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGACGCTTATTGAATGTTTTTCCTGCTGTGGCGGTACGGGTTGGTGAACCGTAGCTATGTGGGCTTGGCTTTTGAATTGGCTTTGCTGTTGCGAAGTAGAGTGGCAGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAATTTGTGTGTACTTCGGTATGCATCTCAA
>DQ396409.1 Phytophthora polonica
CCACACCTAAAAACTGTCCACGTGAACTGTGCTGAAGATGATTGGGGCGTGCTGGGCGTGCGTGTGCGGCTTCGGCTGCGTGCGTGTGTTTGGCGTGCCCATCAAACGAGGAGGCGGGTCGCAAGGCCTGCTTTAGTAGTTACTATTTTAAACCCTTTTACTGTTAACTGATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAAGCCTGGGAGTATGCCTGTATCAGTGTCCGTACAACAACCTTGGCTTCCTTCCTTCCGTGTAGTCGGTGGCGGGAACGCTCAGACGTGAAGTGTCTTGCGTCAGTAATTTGTAATTGCTAAGGCGAGTCCTTTGAAAAGTAGGACGTGTTCTTCTCTTTGCTGGAAAAGCGTTGCGGTGCGTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACTGATTTCGTGCTGATGCGTGTGGGGAGTGGTTCGATTCGCGGTATGGTTGGCTTCGGCTGATTTTCTGCTTATTGGGTCGTTTTCTGCTGCGTTGGCGTTTGCGGGGTCGGTGAACCGTAGCTGCGTTGGGCTTGGCTTTTGAACTGTGTGCTGTGCTGCGAAGTATGGTGGCTGCGTGCGCCTTCGGGTGCGTGCGGCCGAGAGACGGTCACCATTTGGGAAGTTTGTGCTTTGGGCCTCGGCCTTTGGCATCTCAA
>DQ396417.1 Phytophthora polonica
CCACACCTAAAAACTGTCCACGTGAACTGTGCTGAAGATGATTGGGGCGTGCTGGGCGTGCGTGTGCGGCTTCGGCTGCGTGCGTGTGTTTGGCGTGCCCATCAAACGAGGAGGCGGGTCGCAAGGCCTGCTTTAGTAGTTACTATTTTAAACCCTTTTACTGTTAACTGATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAAGCCTGGGAGTATGCCTGTATCAGTGTCCGTACAACAACCTTGGCTTCCTTCCTTCCGTGTAGTCGGTGGCGGGAACGCTCAGACGTGAAGTGTCTTGCGTCAGTAATTTGTAATTGCTAAGGCGAGTCCTTTGAAAAGTAGGACGTGTTCTTCTCTTTGCTGGAAAAGCGTTGCGGTGCGTGGTTGTGGACGCTGCCTGTGTGGCCAGTCGGCGACTGATTTCGTGCTGATGCGTGTGGGGAGTGGTTCGATTCGCGGTATGGTTGGCTTCGGCTGATTTTCTGCTTATTGGGTCGTTTTCTGCTGCGTTGGCGTTTGCGGGGTCGGTGAACCGTATCTGCGTTGGGCTTGGCTTTTGAACTGTGTGCTGTGCTGCGAAGTATGGTGGCTGCGTGCGCCTTCGGGTGCGTGCGGCCGAGAGACGGTCACCATTTGGGCAAGTTTGTGTCTTTGGGCCTCGGCCTTTGGCATCTCAA
>DQ396422.1 Phytophthora polonica
CCACACCTAAAAACTGTCCACGTGAACTGTGCTGAAAATGATTGGGGCGTGCTGGGCGTGCGTGTGCGGCTTCGGGCTGCGTGCGTGTGTTTGGCGTGCCCATCAAACGAGGAGGCGGGTCGCAAGGCCTGCTTTAGTAGTTACTATTTTAAACCCTTTTACTGTTAAATGATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATAGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAAGCCTGGGAGTATGCCTGTATCAGTGTCCGTACAACAACCTTGGCTTCCTTCCTTCCGTGTAGTCGGTGGCGGGAACGCTCAGACATGAAGTGTCTTGCTGTCAGTAATTTGTAGATTGCTAAGGCGAGTCCTTTGAAAAGTAGGACGTGTTCTTCTCTTTGCTGGAAAAGCGTTACGGTGCGTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACTGATTTCGTGCTGATGCGTGTGGGGAGTGGTTCGATTCGCGGTATGGTTGGCTTCGGCTGATTTTCTGCTTATTGGGTCGTTTTCTGCTGCGTTGGCGTCTGCGGGGTCGGTGAACCGTAGCTGCGTTGGGCTTGGCTTTTGAACTGTGTGCTGTGCTGCGAAATATGGTGGCTGCGTGCGCCTTCGGGTGCGTGCGGCCGAAGAGACGGTCACCATTTGGGCAAGTTTGTGTCTTTGGGCCTCGGACCTTTGGCATCTCAA
>DQ988207.1 Phytophthora boehmeriae
TTGGGGACTCCTCGTCGGCGGCTTTCGGGCTGTTGGTGGGTTGAGGCCTATCAAAGCGAGTGATTTGGGCCGCAAGGTCTGAGTTGCGAGTTTTTCCTTTTAAACCCATTACTTAAATACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTATGCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGCTTCCTTCCTTCCGTGTAGTCGGTGGATGGGAATGCCAGACGTGAAGTGTCTTGCTGTAGCGGTTTTCGGACTGCGGCGCGAGTCCTTTGAAATGTAGATACTGTTCTTCTCTTTGCTCGAAAAGCTGTGCAGAATGTATGTTGTGGAGGCTGCCATCACGGCATGTCGGCGACTGATTTCGTGCTGAGGCGTTTTGGAGAGGGGTTCGATTCGCGGTAGTGTTGGTTTCGGCCGAACAATCTGCTTATTGGGTTCTTTTCCAGGCTTTGGCGTATGCGGAGTCGGTGTACCGTAGTTGTGCGTGGCTTGGCCTTTGAACGCTCGTTTTGCGTGGCGAAGTAGAGTGGCGAAGTGTTTGTTTTCGGACGAATGCGGCTGCCGAGGAGGGGATAAAATATTTTGGGAAAATTTAGTGTGCG
>DQ995184.1 Phytophthora austrocedrae
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAAAACCCTTTTATTGGGGGCTTCTGTCTGGTCTGGCTTCGGCTGGTCTGGGTGGCGGCTCTATCATGGTGACCGCTCTGGGCTTCGGCTTGGAGTTAGTAGCCCACTTTTTAAACCCATTCTTAATTACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGATGCGCAGACGTGAAGTGTCTTGCGACTGGGCTTCGGCTCGGCTGCGAGTCCTTTGAAATGTACAGAACTGTACTTCTCTTTGCTCGAAAAGCGTTATATTACTGGTTGTGGAGGCGTGCTTGTACTGGCAAGTCGGCGACCGGTTTGTTAGCTGCGGCGTTTAATGGGGGAGTGTTCGATTCGCGGTATGATCGGCTTCGGCTGGACAGACGCTTATTGGATGCTTTTCCTGCTGTGGCGGTACGAACTGGTGAACCGTAGCTGTATTTAAGCTTGGCATTTGAACCGACGATGTGGTGCGAAGTAGAGTGGCGGTTGTTCCGGCGCAAGCTGGAGTGGCTGTCGAGGGTCGATCCATTTGGGAAATTTGTGTCAGTGCGACTTCGGTTGCGCTGGCATCTCAA
>EF120469 Phytophthora foliorum
CCACACCTAAAAACTTTCCACGTGAACCGTATCAACACCCTTAGTTGGGGGCTGCTGTTCGGCTGGCTTCGGCTGGCTGGGCGGCGGCTCTATCATGGCGAGCGCCTGGGCCTTCGGGTCTGAGCTAGTAGCCCACTTTTAAACCCATTCCTAATTACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGACGCGCAGACGTGAAGTGTCTTGCGGCTGGTCTTCGGTCCGGCTGCGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTTGTTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGAGCGCTTATTGGATGCTTTTCCTGCTGTGGCGGTACGGACTGGTGAACCGTAGCTGTGCGAGGCTTGGCTTTTGAATCGGCGATGTGGTGCGAAGTAGAGTGGCTGTTCGGCGCAAGCTGGGCGGTCGAGGGTCGATCCATTTGGGAAATTTGTGTTGGCGGCTTCGGCTTGCTGGCATCTCAA
>EF490682.1 Phytophthora siskiyouensis
CCACACCTAAAAAAACTTTCCACGTGAACCGTATCAACCCTTTTAGTTGGGGGTCTTGTACCCTATCATGGCGAATGTTTGGACTTCGGTCTGGGCTAGTAGCTTTTGTTTTAAACCCATTTCACAATACTGATAATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGATGTGCCAGATGTGAAGTGTCTTGCGGGTTGTCCTTCGGGTCGGCTGCGAGTCCTTTTAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGTGTTGCTGGTTGTGGAGGCTGCCTGCGTGGCCAGTCGGCGACCGGTTTGTCTGCTACGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGGCGCTTATTGTATGCTTTTCCTGCTGTGGCGTGATGGGCTGGTGAACCGTAGCTGTGTGTGGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGACGGCTTCGGCTGTCGAGGGTCGATCCATTTTGGGAACTTTGTGTGCACTTCGGTGCGCATCTCAA
>EU583793.1 Phytophthora hydropathica
CCACACCTAAAAACTTTCCACGTGAACTGTCTGTGATGTTGGGGGCTGTCTGCCTGTTGGCGGCGGCTCCATCAAACGAGGCTCTGGGCTGCAAAGTCGAGGGTAGTAGTTACTTTTTGTAAACCCTTCCTATATTTTCTGATCATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTATGCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGCGGGGACGCGCAGACGTGAAGTGTCTTGCTGGCCGTTCGGCCGGCGAGTCCTTTGAAATGTAAGATACTGTTCTTCTCTTTGCTGGAAAAGCGTGCGTTGTGTGGTTGTGGAGGCTGCCGTTATGGCCAGTCGGCGACTGATTTCGTGCTGAGGCGTGTGGAGAGGAGCTCGATTCGCGGTATGGTTGGCTTCGGCTGAACTTCTGCTTATGGGGCTGTTTTTCTGCTGTTTCGGCGTGGCGGGGTCCGTGAACCGTAGTCATAGTGGCTTGGCTTTTGAACCGTGCGACTAGTGCGCGAAGTATGGTGGCTTCGGCCGAGAGGACGACCTATTTGGGACGATTGTGCGGCTTCGTGCTGCATCTCAA
>EU595771.1 Phytophthora irrigata
CCACACCTAAAAACTTTCCACGTGAACTGTCTGTGATGTTGGGGGCCGCTGGTCGGCTCCATCAAACGAGGCTCTGGGCTGCAAAGTCGAGGGTAGTAGTTACTTTTTTGTAAACCCTTTTTACTATTTTCTGATGATACTGTGGGGACGAAAGTCTCTGCTTTGAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTATGCCTGGGAGTATGCCTGTATCAGTGTCCGTACATGAACTTTGTCTTCCTTCCTTCCGTGTAGTCGGTGGCGGGGACGCGCAGACGTGAAGTGTCTTGCTGGTCATTTTGTAAGAGATGGCAGGCGAGTCCTTTGAAATGTAAGATACTGTTCTTCTCTTTGCTGGAAAAGCGTGCGTTGTGTGGTTGTGGAGGCTGCCGTTGTGGCCAGTCGGCGACTGATTTCGTGCTGAGGCGTGTGGAGAGGAGGTCGATTCGCGGTATGGTTGGCTTCGGCTGAACTTCTGCTTATGGGTCTGTTTTCTGCTGTTTTGGCGTGGCGGGGTCGGTGCACCGTAGTCATGGCGGCTTGGCTTTTGAACCGCGCAGGTGTTGCGCGAAGTATGGTGGCTTGTGCCGAGAGGACGACCTATTTGGGACAATTGTGCGGCTTTTGGCTGCATCTCAA
>L41373.1 Phytophthora macrochlamydospora
CCACACCTAAAACTTTTCCACGTGAACCGTATTTTTGACGTTGGGGACTCCTGGCTTCGGCCGGTTGAGACCTATCACGCGACCGCTCGGGCTGAAAAGTCGGGGCCAGTAGCCTTTTTTTAAACCCATTCCGAACTACTGAACCATACTGTGGGGACGAAAGTCCTTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGGAGGACCCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTATGCCTGGGAGTATGCCTGTATCAGTGTCCGTACATGAAACTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGACGCGCAGACGTGAAGTGTCTTGCGGCGGCGGCTTCGGCCGGCTGCTGCGAGTCCTTTGAAATGTAAGACCTGTTCTTCTCTTTGCCCGAAAAGCGTTGCGTGAGTGTGGTTGTGGAGGCTGCCGTTTGCGGCATGTCGGCGACTGATTTCGTGCTGAGGCGTGTGGGGAGAGGCTCGATTCGCGGTATGGTTGGCTTCGGCTGAACTTCTGCTTATTGGTCTTTTTTTCGCGGCTTTGGCGTTTTGCGGGGTCGGTGTACCGTAGCTGCGGGTGGCTTGGCCTTTGAACCGTGTTCGTGCGGAGCGAAGTAAGGTGGGCGTGACTGTAGTGTTATGGCCGAGGGGGACGCAACCATTTGGGAAAGTAATGCTTTTCTGCCGGCTTTGGGGCTGGCGGGGCTGCATCTCGA
>HQ261661 Phytophthora quininea
CCACACCTAAAACTTTCCACGTGAACCGTATTTTTGACGTTGGGGACTCCTGGCTTCGGCCGGTTGAGACCTATCACGCGACCGCTCGGGCTGAAAAGTCGGGGCCAGTAGCCTTTTTTAAACCCATTCCGAACTACTGAACCATACTGTGGGGACGAAAGTCCTTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTATGCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGACGCGCAGACGTGAAGTGTCTTGCGGCGGCGGCTTCGGCCGGCTGCTGCGAGTCCTTTGAAATGTAAGACCTGTTCTTCTCTTTGCCCGAAAAGCGTTGCGTGAGTGTGGTTGTGGAGGCTGCCGTTTGCGGCATGTCGGCGACTGATTTCGTGCTGAGGCGTGTGGGGAGAGGCTCGATTCGCGGTATGGTTGGCTTCGGCTGAACTTCTGCTTATTGGGCTTTTTTCCTGGCTTTGGCGTTTTGCGGGGTCGGTGTACCGTAGCTGCGGGTGGCTTGGCCTTTGAACCGTATTCGTGCGGAGCGAAGTAAGGTGGCCGTGACTGTAGTGTTATGGCCGAGGGGGACGCAACCATTTGGGAAACTACTGCTTTTCTGCCGGCTTTCGGGCTGGCGGGGCTGCATCTCAA
>EU301169.2 Phytophthora sp.personii
CCACACCTAAAACACTTTCCACGTGAACCGTATCAACCCCTTAAGATTGGGGGCTTGCTCGGCGGCGTGCGTGCTGGCCTGTAATGGGTCGGCGTGCTGCTGCTGGGCGGGCTCTATCATGGGCGAGCGTTTGGGCTTCGGCTCGAGCTAGTAGCTTTTTCTTTTAAACCCCTTCTCTAATGACTGAAATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTCGTCCTGGAAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGTTGGCGGTCTTTCGAGTCTGCCGGCGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGCTCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCCGTATGGCCAGTCGGCGACCGGTTTGTTCGCGGTAGTGTATACTGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGGTCGCTTTTCCTGCTATTGTCGGTACAAACTGGTGAACCGTAGCTGTGTGGAGCTTGGCTTTTGAATCGGCTTTGCTTGCGGAGTAGAGTGGCGGCTTCGGCTGCCGAGGGGTCGATCCATTTTGGGAACTTATGTGTGTGCGGCTTCGTGCTGCGCGCATCTCAA
>EU925375 Phytophthora sansomeana
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCTTTTTAAATTGGGGGCTTCCGTCTGGCCGGCCGGTTTTCGGCTGGCTGGGTGGCGGCTCTATCATGGCGACCGCTTGGGCCTCGGCCTGGGCTAGTAGCTTCTCTTTTAAACCCATTCCTAATGACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACACTAAACTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGACGCGCAGATGTGAAGTGTCTTGCGGCTGGTCTTCGGTCCGGCTGCGAGTCCTTTGAAATGTACTAAACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCATGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGACGCTTATTGGGTGCTTTTCCTGCTGTGGCGGGACGGACTGGTGAACCGTAGCTGTGCTAGGCTTGGCGTTTGAACCGGTGGTGTGGTGCGAAGTAGGGTGTCTGTCTTGGCGCAAGCTGGGGTGGGCGAGGGTCGATCCATTTGGGAAACGTTGTGTGCGCTTCGGCGCGCATCTCAA
>EF564803 Phytophthora taxon cuyabensis
CCACACCTAAAAACTTTCCACGTGAACTGTCTGTGATGTTGGGGGCCGCCGCTTGCGGTGGCTTCATCAAACGAGGCTCTGGGCTGCAAAGTCGAGGGTAGTAGTTACTTTTTGTAAACCCTTTCGTATTTTCTGATCATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTATGCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGCTTCCTTCCTTCCGTGTAGTCGGTGGCGGGAATGCGCAGACGTGAAGTGTCTTGCCTGGCGTGTGCTGGGCGAGTCCTTTGAAATGTAAGATACTGTTCTTCTCTTTGCTGGAAAAGCGTGCGTTGTGTGGTTGTGGAGGCTGCCGTTATGGCCAGTCGGCGACTGACTTCGTGCTGAGGCGTGTGGAGAGGAGCTCGATTCGCGGTATGGTTGGCTTCGGCTGAACTTCTGCTTATGGGGCTGCTTTGTGCTGCTTTGGCGTGTCGGGGTTGGTGAACCGTAGTCATGGTGGCTTGGCTTTTGAACCGCGCGGCTGTTGCGCGAAGTATGGTGGTTGTCCAAGACGACCGAGAGGACGACCTATTTGGGACAATTGTGTGGCTTCGTGCTGCATCTCAA
>EF590256 Phytophthora taxon lagoariana
CCACACCTAAAAACTTTCCACGTGAACTGTCTGTGATGTTGGGGGCGTGCCGTTCGCGGCGGCTCCATCAAACGAGGCTCTGGGCTGAAAAGTCGAGGGTAGTAGTTACTTTTTGTAAACCCTTTACCACATTTTCTGATCATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTATGCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGCTTCCTTCCTTCCGTGTAGTCGGTGGCGGGAACGCTCAGACGTGAAGCGTCTTGCTGGCCTTATGGCCGGCGAGTCCTTTGAAATGTAAGATACTGTTCTTCTCTTTGCTGGAAAAGCGTGCGTTGTGCGGTTGTGGAGGCTGCCGTTATGGCCAGTCGGCGACTGATTTCGTGCTGAGGCGTGTGGAGAGGAGCTCGATTCGCGGTATGGTTGGCTTCGGCTGAACTTCTGCTTATGGGGCTGCTTGTCTGCTGTTTCGGCGTAGCGGGGTCGGTGAACCGTAGTCATGATGGCTTGGCTTTTGAACCGTGCGACTGTTGCGCGAAGTATGGTGGCTTCGGCCGAGAGGACGACCTATTTGGGACGATTGTGCGGCTTCGTGCTGCATCTCAA
>AY369375.1 Phytophthora hibernalis
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCTTTTAGTTGGGGGCTTCTGTTCGGCTGGCTTTTGCTGGCTGGGCGGCGGCTCTATCATGGCGAGCGCCTGGGCCTTCGGGTCTGAGCTAGTAGTCTTCTTTTAAACCCTTTCTTAAATACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGCCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGACGTGCAGACGTGAAGTGTCTTGCGATTGGTCTTCGGGCCGGCTGCGAGTCCTTTTAAATGTACAGAACGGTACTTCTCTTTGCTCGAAAAGCATAATGGAATTGGTTGTGGAAGCTTCCCGGTGGCAAGTCGGCGACTGGTTTGTCTGCTACGGCGTTTAATGGAGGAATGTTCGATTCGCGGTATGGTTAGCTTCGGCTGAACAATGCGCTTATTGGATGTTTTTCCTGCTGTGGTGGTAATGACTGGTGAACCGTAGCTATGCAGGGATTGGCCTTTGAACTGAGGATGTTGTGTGAAGTAGAGTGGCGGTTTGGCGCAAGCTGGGCTGTCGAGGGTCGATCCTATTTGGGAAATTTGTGTTGGCGGCTTCGGCTGTTGGCATCTCAA
>AY251649.1 Phytophthora meadii
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCTTTTAGTTGGGGGTGTTGCTTGGCATTTTGCTGAGCCGCGCCCTATCATGGCGAATGTTTGGACTTCGGTCTGGGCTAGTAGCTTTTTTGTTTTAAACCATTTAACAATACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGATGTGCCAGATGTGAAGTGTCTTGCGGTTTGTGTGCCTTCGGGCCGAGGCTGCGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGTGTTGCTGGTTGTGGAGGCTGCCTGCGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGGCGCTTATTGTATGCTTTTCCTGCTGTGGCGTGATGGGCTGGTGAACCGTAGCTGTGTGTGGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGTGTCGATCCATTTTGGGAACTTTTGTGTGCGCTTTCGAGTGTGCATCTCAA
>DQ988177.1 Phytophthora frigida
GTGCGGCTAGCTGGCCTTATTGGCTGGCAGGCTGCGTGCTAGGCGAGTCCTATCATGGCAAACGTTTGGGTTCGGCTTGAGCTAGTAGCTTTTTTTTAAACCATTTCTTTAATACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGAAGTATGCCTGTATCAGTGTCCGTACATCAAAATTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGATGTGCCAGAATGTGAAGTGTCTTGCGGCTGGCCTTCGGGTTGACTGCGAGTCCTTTTAAATGTACTGAACTGTATTCCTCTTTGCTCGAAAAGTAAGGCGTTGCTTGTTGTGGAGGCTGCCCGTGTGGCCAGTCGGCGACTGGTTTGTCTGCTGCGGTGTTTTACTGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGACGCTTATTGGGTGTTTTTCCTGCTGTGGCGTGATGGACTGGTGAACCGTAGCTATGTAGGCTTGGCTTTTGAACCGGTGATACTATTGCGAAGTAGAGTGGCGGCTTCGGCTGCCGAGGGGTCGATCCATTTTGGGAACTTCATGTGTGTA
>DQ988196.1 Phytophthora alticola
GGGGGTCTTTTTCGGCGACGGCTGCTGGCTTCATTGCTGGCGGCTGCTGTTGAATGAGCCCTATCATGGCGAGCGTTTAGGCTTCGGTCTGAACTAGTAGCTTTTTTAAACCCATTCTTTATTACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAAGATGTGCCAGATGTGAAGTGTCTTGCGGCTGGTCTTCGGGTCGGCTGTGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGCGTTGCTGATTGTGGAGGCTGCCTGCGTAGCCAGTCTGGCGACCAGTTTGTCTGCTGTGGCGTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGACGCTTATTGAATGGTTTTTCCTGCTATGGCGGTACAGGTTGGTGAACCGTAGCTATGTGGGCTTGGCTTTTGAATTGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCAGCTGTCGAGGGTCGATCCATTTGGGAAGTGGTGGGTA
>EU725805.1 Phytophthora pinifolia
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCCTTAAATTTGGGGGCTTGCTCGGCGGCGTGCGTGCTGGCCTGTAATGGGTCGGCGTGCTGCTGCTGGGTAGGCTCTATCGCGAGCGTTTGGGCTTCGGCTCGAGCTAGTAGCTATTAATTTTAAACCCTTTCTTAAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGAAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTAGCGGTCTTTCGAGTCTGCCGGTGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGCTCGAAAAGCGTGGCGTTGCTAGTTGTGGAGGCTGCCCGTGTGGCCAGTCGGCGACCAGTTTGTTAGCTTTGGCGTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTAAACAATCTGCTTATTGGGTGCTTTTTCAGTCATTTGCGGTACGAACGGGTGAACCGTAGCTGTGTGGTGCTTGGCTTTTGAACCGGCTTTGCTTTGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAACTTTGTGTGTGCGGCTTCGTGCTGCGCGCATCTCA
>EF590257.1 Phytophthora taxon sulawesiensis
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCTTTTAGTTGGGGGCTTGCTTGGCGGCTTGGCGTGCTGGCCTGTCATGGGTCGGCGTGCTGCTGCTGAGCGGGCTCTATCATGGGCGAGCGTTTGGGCTTCGGCTCGAGCTAGTAGCTTTTCAATTTTAAACCCATTCGTAATGACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGATACGCCAGACGTGAAGTGTCTTGCAGCTGGCCTTTCGAGGCTGTCGGTGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGCTCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCCGTGTGGCCAGTCGGCGACCGGTTTGTCTGCTGTGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAAATCTGCTTATTGGGCGTTTTTCCTGCTGCGGCGGTATGAGCTGGTGAACCGTAGCTGTGCGAGGCTTGGCCTTTGAATCGGCTTTGCTGTTTGCGAAGTAGAGTGGCGGTCCTTTCGGGGAGCTGTCGAGGGGTCGATCCATTTTGGGAATTTTTGTGTGTGCGGCTTCGGCTGCGTGCATCTCAA
>EU035774.1 Phytophthora taxon novaeguinee
CCACACCTAAAAAACTTTCCACGTGAACCGTAACAAACCAATAGTTGGGGGCGAATTTGGCGGCGGCTGCTGGCTTTGGCTGGCTGGCTGCTGCTGGACGAGCTCTATCATGGCGAGCGTTTGGACTTCGGTCTGAACTAGTAGCCCTTCTTTTAAACCCATTCCTAATTACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACAACAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGAGATGCCAGATGTGAAGTGTCTTGCGGCTGGTTCTCGAACTGGCTGCGAGTCCTTTGAAATGTACTGAACTGTACTCCTCTTTGCTCGAAAAGCATGGCGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCATGTCGGCGACCGGTCTGTCTGCTGCGGCGTTAATGGAGGCGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGATGTGTTTCCTGCTGTGGCGGTAATGGGCTGGTGTACCGTAGCTATGTGGTGCTTGGGCTTTTGAATTGGCTTTGCTGTTGTGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAAATTTGTGTGTGCCTTTATTGGCGTGCATCTCAA
>EF437222.1 Phytophthora taxon kunnunara
CCACACCTAAAAACTTTCCACGTGAACTGTCTGTGATGTTGGGGGCGTGTCGTTCGCGGCGGCTCCATCAAACGAGGCTCTGGGCTGCAAAGTCGAGGGTAGTAGTTACTTTTTGTAAACCCTTTACCACTTTTTCTGATCATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTATGCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGCTTCCTTCCTTCCGTGTAGTCGGTGGCGGGAACGCTCAGACGTGAAGCGTCTTGCTGGCTGTGAGGCCGGCGAGTCCTTTGAAATGTAAGATACTGTTCTTCTCTTTGCTGGAAAAGCGTGCGTTGTGCGGTTGTGGAGGCTGCCGTTATGGCCAGTCGGCGACTGATTTCGTGCTGAGGCGTGTGGAGAGGAGCTCGATTCGCGGTATGGTTGGCTTCGGCTGAACTTCTGCTTATGGGGCTGCTTGTCTGCTGTTTCGGCGTTGCGGGGTCGGTGAACCGTAGTCATGGTGGCTTGGCTTTTGAACCGTGCGATTGCTGCGCGAAGTATGGTGGCTTCGGCCGAGAGGACGACCTATTTGGGACGATTGTGCGGCTTCGTGCTGCATCTCA
>EF680320.1 Phytophthora taxon napoensis
CCACACCTAAAAACTTTCCACGTGAACTGTGTTGTCGCTGTTGGGGGCTGGGCTGGCGCTTTTGGGCGTCGGCTGGGCTCCATCAAACGAGGCTCTGGGCTGCAAAGTCGAGGGTAGTAGTTACTTTTTGTAAACCCTTTCTTCTTTTTCTGATCATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTATGCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAATCTTGGCTTCCTTCCTTCCGTGTAGTCGGTGGCGGGAACGCGCAAACGTGAAGTGTCTTGCTGGCTGGTCTTCCTTTTGGGGGGGGCGTGGCCGGCGAGTCCTTTGAAATGTAAAACACTGTTCTTCTCTTTGCTGGAAAAGCGCGCGTTGTGTGGTTGTGGAGGCTGCCGTTGTGGCCAGTCGGCGACTGACTTCTTGCTGAGGCGTGTGGAGAGGAGCTCGATTCGCGGTATGGTTGGCTTCGGCTGAACTTCTGCTTATGGGGCTGTTTTCTGCTGTTTCGGCGTGGCGGGGTTGGTGAACCGTAGTCATGGTGGCTTGGCTTTTGAACCGCGCGGCTGGTGCGCGAAGTATGGTGGCTTCGGCCGAGAGGACGACCTATTTGGGACGATTGTGCGGCCTCGTGCTGCATCTCAA
>EU164426.1 Phytophthora taxon zentmyerii
CCACACCTAAAAACTTTCCCACGTGAACTGTCTGTGATGTTGGGGGCGTGCCGTTCGCGGCGGCTCCATCAAACGAGGCTCTGGGCTGAAAAGTCGAGGGTAGTAGTTACTTTTTGTAAACCCTTTACCACATTTTCTGATCATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTATGCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGCTTCCTTCCTTCCGTGTAGTCGGTGGCGGGAACGCTCAGACGTGAAGCGTCTTGCTGGCCNTCTGGCCGGCGAGTCCTTTGAAATGTAAGATACTGTTCTTCTCTTTGCTGGAAAAGCGTGCGTTGTGCGGTTGTGGAGGCTGCCGTTATGGCCAGTCGGCGACTGATTTCGTGCTGAGGCGTGTGGAGAGGAGCTCGATTCGCGGTATGGTTGGCTTCGGCTGAACTTCTGCTTATGGGGCTGCTTGTCTGCCGTTTCGGCGTTGCGGGGTCGGTGAACCGTAGTCATGGTGGCTTGGCTTTTGAACCGTGCGACTGCTGCGCGAAGTATGGTGGCTTCGGCCGAGAGGACGACCTATTTGGGACGATTGTGCGGCTTCGTGCTGCATCTCAA
>AY770741.1 Phytophthora andina
CCACACCTAAAAACTTTCCACGTGAACCGTTTCAACCCAATAGTTGGGGGTCTTACTTGGCGGCGGCTGCTGGCTTTATTGCTGGCGGCTACTGCTGGGCGAGCCCTATCAAAAGGCGAGCGTTTGGACTTCGGTCTGAGCTAGTAGCTTTTTTATTTTAAACCCTTTACTTAATACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGAAGTATGCCTGTATCAGTGTCCGTACAACAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGAGATGCCAGATGTGAAGTGTCTTGCGGTTGGTTTTCGGACCGACTGCGAGTCCTTTTAAATGTACTAAACTGTACTTCTCTTTGCTCCAAAAGTGGTGGCATTGCTGGTTGTGGACGCTGCTATTGTAGCGAGTTGGCGACCGGTTTGTCTGCTGCGGCGTTAATGGAGAAATGCTCGATTCGTGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGGTGATTTTCCTGCTGTGGCGTGATGGACTGGTGAACCATGGCTCTTTAGCTTGGCATTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGCCGAGGGTCGATCCATTTGGGAAATGTTGTGTACTTCGGTATGCATCTCAATTGGACCTGATATCAGGC
>EU748545.1 Phytophthora mengei
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCTTTTAGTTGGGGGTCTTGTACCCTATCATGGCGAATGTTTGGACTTCGGTCTGGGCTAGTAGCTTTTTGTTTTAAACCCATTTCACAATACTGATTATACTGCGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGATGTGCCAGATGTGAAGTGTCTTGCCGGTTTGTCCTTCGGGTCGGCTGCGAGTCCTTTTAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGTGTTGCTGGTTGTGGAGGCTGCCTGCGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGGCGCTTATTGTATGCTTTTCCTGCTGTGGCGTGATGGGCTGGTGAACCGTAGCTGTGTGTGGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTTGGGAACTTTGTGTGCACTTCGGTGCGCATCTCAA
>FJ469147.1 Phytophthora morindae
CCACACCTAAAAAACTTTCCACGTGAACCGTTAAAACTTTAGTTGGGGACTCCTCATTGGCAGTTTGTTCTGTTGGTGAGTTGAGGCCTATCAAACGATCAATTTGGGCTGCAAAGTCGGAGTTGAGAGTTTCTTTTTTAAACCCATGCTTTTATTACTGAAAATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTATGCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGCTTCCTTCCTTCCGTGTAGTCGGTGGATGGGAATGCCAGACGTGAAGTGTCTTGCTGTAGCGGTTTTCGGACTGCGGCGGCGAGTCCTTTGAAATGTAGATACTGTTCTTCTCTTTGCTCGAAAAGCTGTGCAGAATGTAGTTGTGGAGGCTGCCATCACGGCATGTCGGCGACTGATTTCGTGCTGAGGCGTTTTTGGAGAGGGGTTCGATTCGCGGTAGTGTTGGTTTCGGCCGAACAATCTGCTTATTGTGTCTTTTTCCTTGCTTTGGCGTTAGCGGAGTTAGTGTACCGTAGTAGTGTGTGGCTTGGCTTTTGAACGAAAGTTTGTGCGCGAAGTAGAGTGGCCACGTGTGGTTTTCGGACGATGCGGTGCCGAGGGGACGAAACGATTTGGGAACAATTTGTGTGCGCGCTTTCGGGCGTTGTACATCTCAA
>FJ665225.1 Phytophthora plurivora
CCACACCTAAAAAACTTTTCACGTGAACCGTATCAACCCTTTTAGTTGGGGGTCTTGCTTTTTTTGCGAGCCCTATCATGGCGAATGTTTGGACTTCGGTCTGGGCTAGTAGCTTTTTGTTTTAAACCCATTCTACAATACTGATTATACTGCGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGATGTGCCAGACGTGAAGTGTCTTGCGGGTGTCCTTCGGGTCGTCTGCGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGTGTTGCTGGTTGTGGAGGCTGCCTGCGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGGCGCTTATTGTATGCTTTTCCTGCTGTGGCGTGATGGGCTGGTGAACCGTAGCTGTGTGTGGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAACTTTGTGTGCACCTCGGTGCGCATCTCAA
>FJ665227.1 Phytophthora plurivora
CCACACCTAAAAAACTTTTCACGTGAACCGTATCAACCCTTTTAGTTGGGGGTCTTGCTTTTTTTGCGAGCCCTATCATGGCGAATGTTTGGACTTCGGTCTGGGCTAGTAGCTTTTTGTTTTAAACCCATTCTACAATACTGATTATACTGCGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCATACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGATGTGCCAGACGTGAAGTGTCTTGCGGGTGTCCTTCGGGTCGTCTGCGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGTGTTGCTGGTTGTGGAGGCTGCCTGCGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGGCGCTTATTGTATGCTTTTCCTGCTGTGGCGTGATGGGCTGGTGAACCGTAGCTGTGTGTGGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAACTTTGTGTGCACCTCGGTGCGCATCTCAA
>FJ237521.2 Phytophthora multivora
CCACACCTAAAAAAACTTTCCACGTGAACCGTATCAACCCTTTCAGTTGGGGGCCTTGCTTTTTTGCGAGCCCTATCATGGCGAATGTTTGGACTTCGGTCTGGGCTAGTAGCTTTTTGTTTTAAACCCATTCTACAATACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGATGTGCCAGATGTGAAGTGTCTTGCAGGTGTCCTTCGGGTCGTCTGCGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCTGCGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGGCGCTTATTGTATGCTTTTCCTGCTGTGGCGTGATGGACTGGTGAACCGTAGCTGTGTGTGGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTCCGGCTGTCGAGGGTCGATCCATTTGGGAACTTTGTGTGCACCTCGGTGCGCATCTCAA
>AB511826.1 Phytophthora chrysanthemi
CCACACCTAAAAACTTTCCACGTGAACTGTATTGTCTGTTGTTGGGGGCTTGGGCTGGCGTTTCGGCGCCGGCTCGGGCTCCATCAAACGAGGCTCTGGGCTGCAAAGTCTGGGGTAGTAGTTACATTTTTTGTAAACCCTTTTCTTAATTTTTCTGATCATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTATGCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGCTTCCTTCCTTCCGTGTAGTCGGTGGCGGGAACGCGCAGACGTGAAGTGTCTTGCTGGCCGTGTCTCCTCCTCTTTTGGGGTGGGGGGCATGGTCGGCGAGTCCTTTGAAATGTAAGATACTGTTCTTCTCTTTGCTGGAAAAGCGTGGGTGGTGTGGTTGTGGAGGCTGCCGTTGTGGCAAGTCGGCGACTGACTTCGTGCTGAGGCGTGTGGAGAGGAGCTCGATTCGCGGTATGGTTGGCTTCGGCTGAACTTCTGCTTATTGGGCTGTTTTCTGCTGTTTCGGCGTGGCGTTGTTGGTGAACCGTAGTCATGGTGGCTTGGCTTTTGAACCGCGCTGTGTTGCGCGAAGTATGGTGGCTTCGGCCGAGAGGACGACCTATTTGGGACCATTTGTGCGGCTTCGTGCTGCATCTCAA
>AB437136.1 Phytophthora chrysanthemi
CCACACCTAAAACTTTCCACGTGAACTGTGTTGTCTGTTGTTGGGGGCTTGGGCTGGCGTTTCGGCGCCGGCTCGGGCTCCATCAAACGAGGCTCTGGGCTGCAAAGTCTGGGGTAGTAGTTACATTTTTTGTAAACCCTTTTCTTAATTTTTCTGATCATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTATGCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGCTTCCTTCCTTCCGTGTAGTCGGTGGCGGGAACGCGCAGACGTGAAGTGTCTTGCTGGCCGTGTCTCCTCCTCTTTTGGGGTGGGGGGCATGGTCGGCGAGTCCTTTGAAATGTAAGATACTGTTCTTCTCTTTGCTGGAAAAGCGTGCGTGGTGTGGTTGTGGAGGCTGCCGTTGTGGCAAGTCGGCGACTGACTTCGTGCTGAGGCGTGTGGAGAGGAGCTCGATTCGCGGTATGGTTGGCTTCGGCTGAACTTCTGCTTATTGGGCTGTTTTCTGCTGTTTCGGCGTGGCGTTGTTGGTGAACCGTAGTCATGGTGGCTTGGCTTTTGAACCGCGCTGTGTTGCGCGAAGTATGGTGGCTTCGGCCGAGAGGACGACCTATTTGGGACCATTTGTGCGGCTTCGTGCTGCATCTCAA
>GQ847755.1 Phytophthora elongata
CCACACCTAAAAAAACTTTCCACGTGAACCGTATCAACCCTTTAAATTGGGGGTCTTGCTTGGTGTGCGGCTGGCTGGCCTTTATTGGCTGGCAGGCTGCGTGCTGGGCGAGCCCTATCATGGCAAACGTTTGGGCTTCGGTCTGAGCTAGTAGCTTTATTTTTAAACCCATTTCTTAATACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGAGACGCCAGATGTGAAGTGTCTTGCGGCTGGCTTTCGGGCCGACTGCGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGTGTGGCGTTGCTGGTTGTGGAGGCTGCCCGCGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTTACTGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGGCGCTTATTGGACGTTTTTCCTGCTGTGGCGTGATGGGCTGGTGAACCGTAGCTGTGTGTGGCTTGGCTTTTGAATCGGTTTTGCTATTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAACTTTGTGTGTACTTCCGGTGCGCATCTCAA
>GU191219.1 Phytophthora capensis
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCTTTTAGTTGGGGGTCTTGCTTTTTTGCGAGCCCTATCATGGCGAATGTTTGGACTTCGGTCTGGGCTAGTAGCTTTTTGTTTTAAACCCATTCAACAATACTGATTATACTGCGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGATGTGCCAGATGTGAAGTGTCTTGCGGGTGTCCTTCGGGTCGCCTGCGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGTGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACCAGTTTGTCTGCTGTGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGGCGCTTATTGTATGCTTTTCCTGCTGTGGCGTGATGGGCTGGTGAACCGTAGCTGTGTGTGGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAATTTTGTGTGCACCTCGGTGCGCATCTCAA
>HQ012948.1 Phytophthora litoralis
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCTTTTAATTGGGGGCTTGTTTGGCGGCGTGTGTGCTGGCCTGTAATGGGTCGGCGTGCTGCTGCTGGGCGAGCTCTATCATGGGCGAGCGTTTGGGCTTCGGCTCGAGCTAGTAGCTATCACTTTTAAACCCTTTCTTAAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTAGCGGTCTTTCGAGTCTGCCGGTGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGCCCCAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACCGGTTTGTTAGCTGTGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGGGTGCTTTTCCTGTCATTGGCGGTACGAACTGGTGAACCGTAGCTATGTGGTGCTTGGCTTTTGAACCGGCTTTGCTTTGCGAAGTAGGGTGACAGCTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAACTTTGTGTGTGCGGCTTCGTGCTGCGTGCATCTCAA
>HQ012947.1 Phytophthora litoralis
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCTTTTAATTGGGGGCTTGTTCGGCGGCGTGTGTGCTGGCCTGTAATGGGTCGGCGTGCTGCTGCTGGGCGAGCTCTATCATGGGCGAGCGTTTGGGCTTCGGCTCGAGCTAGTAGCTATCACTTTTAAACCCTTTCTTAAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTAGCGGTCTTTCGAGTCTGCCGGTGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGCCCCAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACCGGTTTGTTAGCTGTGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGGGTGCTTTTCCTGTCATTGGCGGTACGAACTGGTGAACCGTAGCTATGTGGTGCTTGGCTTTTGAACCGGCTTTGCTTTGCGAAGTAGGGTGACAGCTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAACTTTGTGTGTGCGGCTTCGTGCTGCGTGCATCTCAA
>HQ012946.1 Phytophthora litoralis
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCTTTTAATTGGGGGCTTGTTTGGCGGCGTGTGTGCTGGCCTGTAATGGGTCGGTGTGCTGCTGCTGGGCGAGCTCTATCATGGGCGAGCGTTTGGGCTTCGGCTCGAGCTAGTAGCTATCACTTTTAAACCCTTTCTTAAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTAGCGGTCTTTCGAGTCTGCCGGTGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGCCCCAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACCGGTTTGTTAGCTGTGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGGGTGCTTTTCCTGTCATTGGCGGTACGAACTGGTGAACCGTAGCTATGTGGTGCTTGGCTTTTGAACCGGCTTTGCTTTGCGAAGTAGGGTGACAGTTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAACTTTGTGTGTGCGGCTTCGTGCTGCGTGCATCTCAA
>EU301173.2 Phytophthora gregata
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCCTTAAATTTGGGGGCTTGCTCGGCGGCGTGCGTGCTGGCCTGTAACGGGTCGGCGTGCTGCTGCTGGGCGGGCTCTATCATGGGCGAGCGTTTGGGCTTCGGCTCGAGCTAGTAGCTTTAATTTTAAACCCATTCTTTAAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTGGCGGTCTTTCGAGTCTGCCGGTGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGCTCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCCGTGTGGCTTGTCGGCGACCGGTTTGTTAGCTGTGACGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGGGTGCTTTTTCCTGTCATTGGCGGTGCGAACTGGTGAACCGTAGTCATGTGGTGCTTGGCTTTTGAATCGGTTTTGCTGTTGCGAAGTAGTGTGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAACTTTGTGTGTGCGGCTTCGTGCTGCGCGCATCTCAA
>EU301171.2 Phytophthora gregata
CCACACCTAAAAAAACTTTCCACGTGAACCGTATCAACCCCTTAAATTTGGGGGCTTGCTCGGCGGCGTGCGTGCTGGCCTGTAATGGGTCGGCGTGCTGCTGCTGGGCGGGCTCTATCATGGGCGAGCGTTTGGGCTTCGGCTCGAGCTAGTAGCTTTAATTTTAAACCCATTCTTTAATTACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTGGCGGTCTTTCGAGTCTGCCGGTGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGCTCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCCGTGTGGCTTGTCGGCGACCGGTTTGTTAGCTGTGACGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGGGTGCTTTTTCCTGTCATTGGCGGTGCGAACTGGTGAACCGTAGTCATGTGGTGCTTGGCTTTTGAATCGGTTTTGCTGTTGCGAAGTAGTGTGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAACTTTGTGTGTGCGGCTTCGTGCTGCGCGCATCTCAA
>HQ012938.1 Phytophthora gregata
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCCTTAAATTTGGGGGCTTGCTCGGCGGCGTGCGTGCTGGCCTGTAATGGGTCGGCGTGCTGCTGCTGGGCGGGCTCTATCATGGGCGAGCGTTTGGGCTTCGGCTCGAGCTAGTAGCTTTAATTTTAAACCCATTCTTTAAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTGGCGGTCTTTCGAGTCTGCCGGTGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGCTCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCCGTGTGGCTTGTCGGCGACCGGTTTGTTAGCTGTGACGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGGGTGCTTTTTCCTGTCATTGGCGGTGCGAACTGGTGAACCGTAGTCATGTGGTGCTTGGCTTTTGAACCGGTTTTGCTGTTGCGAAGTAGTGTGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAACTTTGTGTGTGCGGCTTCGTGCTGCGCGCATCTCAA
>HQ012942.1 Phytophthora gregata
CCACACCTAAAAAAACTTTCCACGTGAACCGTATCAACCCCTTAAATTTGGGGGCTTGCTCGGCGGCGTGCGTGCTGGCCTGTAATGGGTCGGCGTGCTGCTGCTGGGCGGGCTCTATCATGGGCGAGCGTTTGGGCTTCGGCTCGAGCTAGTAGCTTTAATTTTAAACCCATTCTTTAAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTGGCGGTCTTTCGAGTCTGCCGGTGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGCTCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCCGTGTGGCTTGTCGGCGACCGGTTTGTTAGCTGTGACGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGGGTGCTTTTTCCTGTCATTGGCGGTGCGAACTGGTGAACCGTAGTCATGTGGTGCTTGGCTTTTGAATCGGTTTTGCTGTTGCGAAGTAGTGTGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAACTTTGTGTGTGCGGCTTCGTGCTGCGCGCATCTCAA
>HQ012936.1 Phytophthora gibbosa
CCACACCTAAAAAAACTTTCCACGTGAACCGTATCAACCCCTTAAATTTGGGGGCTTGCTCGGCGGCGTGCGTGCTGGCCTGTAATGGGTCGGCGTGCTGCTGCTGGGCGGGCTCTATCATGGGCGAGCGTTTGGGCTTCGGCTCGAGCTAGTAGCTTTAATTTTAAACCCATTCTTTAAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTGGCGGTCTTTCGAGTCTGCCGGTGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGCTCGAAAAGCGTGGCATTGCTGGTTGTGGAGGCTGCCCGTGTGGCTTGTCGGCGACCGGTTTGTTAGCTGTGACGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGGGTGCTTTTTCCTGTCATTGGCGATACGAACTGGTGAACCGTGGTCATGTGGTGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGTGTGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAACTTTGTGTGTGCGGCTTCGTGCTGCGTGCATCTCAA
>HQ012933.1 Phytophthora gibbosa
CCACACCTAAAAAAACTTTCCACGTGAACCGTATCAACCCCTTAAATTTGGGGGCTTGCTCGGCGGCGTGCGTGCTGGCCTGTAATGGGTCGGCGTGCTGCTGCTGGGCGGGCTCTATCATGGGCGAGCGTTTGGGCTTCGGCTCGAGCTAGTAGCTTTAATTTTAAACCCATTCTTTAAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTGGCGGTCTTTCGAGTCTGCCGGTGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGCTCGAAAAGCGTGGCATTGCTGGTTGTGGAGGCTGCCCGTGTGGCTTGTCGGCGACCGGTTTGTTAGCTGTGACGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGGGTGCTTTTTCCTGTCATTGGCGATACGAACTGGTGAACCGTGGTCATGTGGTGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGTGTGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAACTTTGTGTGTGCGGCTTCGTGCTGCGCGCATCTCAA
>EU301155.2 Phytophthora thermophila
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCTTTAAATTTGGGGGCTTGCTCGGCGGCGTGTGTGCTGGCCTGTAATGGGTCGGCGTGCTGCTGCTGGGCGGGCTCTATCATGGGCGAGCGTTTGGGCTTCGGCTCGAGCTAGTAGCTATCACTTTTAAACCCTTTCTTAAAAAACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTAGCGGTCTTTCGAGTCTGCCGGTGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGTCCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACCGGTTTGTTAGCTGTGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGGGTGCTTTTCCTGTCATTGGCGGTACGAACTGGTGAACCGTAGCTATGTGGTGCTTGGCTTTTGAACCGGCTTTGCTTTGCGAAGTAGGGTGGCAGCTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAACTTTGTGTGTGCGGCTTCGTGCTGCGTGCATCTCAA
>HQ012950.1 Phytophthora thermophila
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCTTTAAATTTGGGGGCTTGTTCGGCGGCGTGTGTGCTGGCCTGTAATGGGTCGGCGTGCTGCTGCTGGGCGGGTTCTATCATGGGCGAGCGTTTGGGCTTCGGCTCGAGCTAGTAGCTATCACTTTTAAACCCTTTCTTAAAAAACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTAGCGGTCTTTCGAGTCTGCCGGTGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGTCCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACCGGTTTGTTAGCTGTGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGGGTGCTTTTCCTGTCATTGGCGGTACGAACTGGTGAACCGTAGCTATGTGGTGCTTGGCTTTTGAACCGGCTTTGCTTTGCGAAGTAGGGTGGCAGCTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAACTTTGTGTGTGCGGCTTCGTGCTGCGTGCATCTCAA
>EU301158.2 Phytophthora thermophila
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCTTTAAATTTGGGGGCTTGCTCGGCGGCGTGTGTGCTGGCCTGTAATGGGTCGGCGTGCTGCTGCTGGGCGGGCTCTATCATGGGCGAGCGTTTGGGCTTCGGCTCGAGCTAGTAGCTATCACTTTTAAACCCTTTCTTAAAAAACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTAGCGGTCTTTCGAGTCTGCCGGTGAGTCCTTTGAAATGTACTGAACTGTACTCTCTTTTTGTCCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACCGGTTTGTTAGCTGTGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGGGTGCTTTTCCTGTCATTGGCGGTACGAACTGGTGAACCGTAGCTATGTGGTGCTTGGCTTTTGAACCGGCTTTGCTTTGCGAAGTAGGGTGGCAGCTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAACTTTGTGTGTGCGGCTTCGTGCTGCGTGCATCTCAA
>HM752784.1 Phytophthora himalsilva
CCACACCTAAAAAAACTTTCCACTAACCGTATCAACCCTTTTAGTTGGGGGTGTTGCTTGGCGTTTTGCTGAGCCACGCCCTATCATGGCGAATGTTTGGACTTCGGTCTGGGCTAGTAGCTTTTTTGTTTTAAACCCATTTAACAATACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGATGTGCCAGATGTGAAGTGTCTTGCGGTTTTTGTGCCTTCGGGCCGTGGCTGCGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGTGTTGCTGGTTGTGGAGGCTGCCTGCGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGGCGCTTATTGTATGCTTTTCCTGCTGTGGCGTGATGGGCTGGTGAACCGTAGCTGTGTGTGGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGTGTCGATCCATTTTGGGAACGTTTGTGTGCGCTTTCGAGTGTGCATCTCAA
>GQ324989.1 Phytophthora pini
CCACACCTAAAAAAACTTTTCACGTGAACCGTATCAACCCTTTTAGTTGGGGGTCTTGCTTTTTTGCGAGCCCTATCATGGCGAATGTTTGGACTTCGGTCTGGGCTAGTAGCTTTTTGTTTTAAACCCATTCTACAATACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGATGTGCCAGATGTGAAGTGTCTTGCGGGTGTCCTTCGGGTCGTCTGCGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGTGTTGCTGGTTGTGGAGGCTGCCTGCGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGGCGCTTATTGTATGCTTTTCCTGCTGTGGCGTGATGGGCTGGTGAACCGTAGCTGTGTGTGGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGACGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAACTTTGTGTGCACCTCGGTGCGCATCTCAA
>AF279127.2 Phytophthora glovera
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCTTTTAGTTGGGGGTCTTGTACCCTATCATGGCGAATGTTTGGACTTCGGTCCGGGCGAGTAGCTTTTTGTTTTAAACCCATTTCACAATTCTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGATGTGCCAGATGTGAAGTGTCTTGCGTGTTGTCCTTCGGGTCGACTGCGAGTCCTTTTAAATGTACTGAACTGTACTTCTCTTTGCTCAAAAAGCGTGGTGTTGCTGGTTGTGGAGGCTGCCTGCGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAAAGGAGGAGTGTCCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGGCGCTTATTGAATGCTTTTCCTGCTGTGGCGTGATGGGCTGGTGAACCGTAGCTGTGTTTGGCTTGGCTTTTGAATCCGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTTGGGAATTTTGTGTGCACTTCGGTGCGCATCTCAA
>EU725806.1 Phytophthora pinifolia
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCCTTAAATTTGGGGGCTTGCTCGGCGGCGTGCGTGCTGGCCTGTAATGGGTCGGCGTGCTGCTGCTGGGTAGGCTCTATCGCGAGCGTTTGGGCTTCGGCTCGAGCTAGTAGCTATTAATTTTAAACCCTTTCTTAAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGAAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTAGCGGTCTTTCGAGTCTGCCGGTGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGCTCGAAAAGCGTGGCGTTGCTAGTTGTGGAGGCTGCCCGTGTGGCCAGTCGGCGACCAGTTTGTTAGCTTTGGCGTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTAAACAATCTGCTTATTGGGTGCTTTTTCAGTCATTTGCGGTACGAACGGGTGAACCGTAGCTGTGTGGTGCTTGGCTTTTGAACCGGCTTTGCTTTGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAACTTTGTGTGTGCGGCTTCGTGCTGCGCGCATCTCA
>HQ013215.1 Phytophthora arenaria
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAAACTTTAGTTGGGGGTCTTTTTCGGCGACGGCTGCTGGCTTCATTGCTGGCGGCTGCTGTTGAATGAGCCCTATCATGGCGAGCGTTTAGGCTTCGGTCTGAACTAGTAGCTTTTTTAAACCCATTCTTTATTACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAAGATGTGCCAGATGTGAAGTGTCTTGCGGCTGGTCTTCGGGTCGGCTGTGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGCGTTGCTGATTGTGGAGGCTGCCTGCGTAGCCAGTCTGGCGACCAGTTTGTCTGCTGTGGCGTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTTGGCTGAACAGACGCTTATTGAATGTTTTTCCTGCTATGGCGGTACGGGTTGGTGAACCGTAGCTATGTGGGCTTGGCTTTTGAATTGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAATTTGTGTGTACTTCGGTATGCATCTCAA
>HQ013219.1 Phytophthora arenaria
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAAACTTTAGTTGGGGGTCTTTTTCGGCGACGGCTGCTGGCTTCATTGCTGGCGGCTGCTGTTGAATGAGCCCTATCATGGCGAGCGTTTAGGCTTCGGTCTGAACTAGTAGCTTTTTTAAACCCATTCTTTATTACTGATTATACTGTAGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAAGATGTGCCAGATGTGAAGTGTCTTGCGGCTGGTCTTCGGGTCGGCTGTGAGTCCTTTGAAATGTACTGAACTGTACTTTTCTTTGCTCGAAAAGCGTGGCGTTGCTGATTGTGGAGGCTGCCTGCGTAGCCAGTCTGGCGACCAGTTTGTCTGCTGTGGCGTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTTGGCTGAACAGACGCTTATTGAATGTTTTTCCTGCTATGGCGGTACGGGTTGGTGAACCGTAGCTATGTGGGCTTGGCTTTTGAATTGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAATTTGTGTGTACTTCGGTATGCATCTCAA
>HQ013225.1 Phytophthora constricta
CCACACCTTAAAAAACCATCCACGTGAACCGTATTGTCAAAACATTTGGGGACTCCTTGCTAGCGGGCTTTCGGGCTTGCCGGTCGGTTGAGACCTAACACGAGAGCCTTGGCTGAAAAGCCGGGGTGAGTAGTTTCTTTTTAAACCCTTTACTAAAAACTGAAAAACTGTGGGGACGAAAGTCCTTGCTTTTACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTATGCCTGGAAGTATGCCTGTATCAGTGTCCGTATATCAAACTTGCCTTCCTTCCTTCCGTGTAGTCGGTGGATGGGAACGTGCAGATGTGAAGTGTCTTGGCGGCGTCGCTTTGGCGGTGCTGGCGAGTCCTTTTAAATACAAGATACTGTTCTTCTCTTTGCTCGAAAAGCGTGTGGGATGTGGTTGTGGAGGCTGCCTGCGTGGCATGTCGGCGACTGATTTCGTGCTGAGGCGTTGGAGGTGGGCTTGATTCGCGGTATGGTTGGCTTCGGCTGAACTTCTGCTTATTGGGTTCTAATCTTGCTTTGGCGTATGCGGGGTCGGTGTACCGTAGTCGTGTGGGCTTGGCTTTTGAACCGTGTTTTGCGTGCGAAGTAGAGTGGCCGTGTGGTTTTCGGACTGCGTGGCCGAGGGAACGAGACCTTTTGGGAAATTGTGCTGCTACTTTTGTGGCGGCATCTCGA
>EU301143.1 Phytophthora constricta
CCACACCTTAAAAAACCATCCACGTGAACCGTATCGTCAAAACATTTGGGGACTCCTTGCTAGCGGGCTTTCGGGCTTGCCGGTCGGTTGAGACCTAACACGAGAGCCTTGGCTGAAAAGCCGGGGTGAGTAGTTTCTTTTTAAACCCTTTACTAAAAACTGAAAAACNGTGGGGACGAAAGTCCTTGCTTTTACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTATGCCTGGAAGTATNCCTGNATCAGTGTCCGTATATCAAACTTGCCTTCCTTCCTTCCGTGTAGTCGGTGGATGGGAACGTGCAGATGTGAAGTGTCTTGGCGGCGTCGCTTTGGCGGTGCTGGCGAGTCCTTTTAAATACAAGATACTGTTCTTCTCTTTGCTCGAAAAGCGTGTGGGATGTGGTTGTGGAGGCTGCCTGCGTGGCATGTCGGCGACTGATTTCGTGCTGAGGCGTTGGAGGTGGGCTTGATTCGCGGTATGGTTGGCTTCGGCTGAACTTCTGCTTATTGGGTTCTAATCTTGCTTTGGCGTATGCGGGGTCGGTGTACCGTAGTCGTGTGGGCTTGGCTTTTGAACCGTGTTTTGCGTGCGAAGTAGAGTGGCCGTGTGGTTTTCGGACTGCGTGGCCGAGGGAACGAGACCTTTTGGGAAATTGTGCTGCTACTTTTGTGGCGGCAT
>FJ217680.1 Phytophthora gemini
CCACACCTAAAAACTTTCCACGTGAACCGTATCAACCCCTTATAATTGGGGGCTTGCTTGGCGGCGTGCGTGCTGGCCTATTATGGGTCGGCGTGCTGCTGCTGGGCGAGCTCTATCATGGCGAGCGYTTGGGCTTCGGCTCGAGCTAGTAGCTTTTTTAAACCCATTCTTTAATTACTGAAATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTCGTCCTGGAAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTGGCGGTCTTTCGAGTCTGCTGGCGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGCTCGAAAAGTGTGGCGTTGCTGGTTGTGGAGGCTGCCCGTATGGCCAGTCGGCGACCGGTTTGTTAGCTGTAGTGTATACTGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGGGTGCTTTTCCTGCTATTGGCGGTACAAACTGGTGAACCGTAGCTGTGTGGTTTGGCTTTTGAATCGGCTTTGCTTGCGAAGTAGAGTGGCGGCTTCGGCTGCCGAGGGGTCGATCCATTTTGGGAACTTTGTGTGTGTGGCTTCGTGCTGCGCGCATCTCAA
>FJ217679.1 Phytophthora gemini
CCACACCTAAAAACTTTCCACGTGAACCGTATCAACCCCTTATAATTGGGGGCTTGCTTGGCGGCGTGCGTGCTGGCCTATTATGGGTCGGCGTGCTGCTGCTGGGCGAGCTCTATCATGGCGAGCGTTTGGGCTTCGGCTCGAGCTAGTAGCTTTTTTAAACCCATTCTTTAATTACTGAAATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTCGTCCTGGAAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTGGCGGTCTTTCGAGTCTGCTGGCGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGCTCGAAAAGTGTGGCGTTGCTGGTTGTGGAGGCTGCCCGTATGGCCAGTCGGCGACCGGTTTGTTAGCTGTAGTGTATACTGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGGGTGCTTTTCCTGCTATTGGCGGTACAAACTGGTGAACCGTAGCTGTGTGGTTTGGCTTTTGAATCGGCTTTGCTTGCGAAGTAGAGTGGCGGCTTCGGCTGCCGAGGGGTCGATCCATTTTGGGAACTTTGTGTGTGTGGCTTCGTGCTGCGCGCATCTCAA
>HQ917909 Phytophthora obscura
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAAAACCCTTTATTGGGGGCTTCTGTCTGGTCTGGCTTCGGCTGGGCTGGGTGGCGGCTCTATCATGGCGACCGCTCTGGGCTTCGGCCTGGAGTTAGTAGCTCACTTTTAAACCCATTCTTAATTACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGATGACAGACGTGAAGTGTCTTGCGACTGGGCTTCGGCTCGGCTGCGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTAATATTACTGGTTGTGGAGGCTGCTTGTGTGGCATGTCGGCGACCGGTTTGTTAGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGATCGGCTTCGGCTGGGCAGACGCTTATTGGGTGCTTTTCTTGCTGTGGCGGTACGAACTGGTGAACCGTAGCTATGTTTGGCTTGGCTTTTGAACCGGCGATGTGGTGCGAAGTAGAGTGGCGGTTGTTCCAGCGCAAGCTGGAGTGGCTGTCGAGGGTCGATCCATTTGGGAAATTTGTGTCTGTGCGACTTCGGTTGCGCTGGCATCTCAA
>HQ917911 Phytophthora obscura
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAAAACCCTTTATTGGGGGCTTCTGTCTGGTCTGGCTTCGGCTGGGCTGGGTGGCGGCTCTATCATGGCGACCGCTCTGGGCTTCGGCCTGGAGTTAGTAGCTCACTTTTAAACCCATTCTTAATTACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGATGACAGACGTGAAGTGTCTTGCGACTGGGCTTCGGCTCGGCTGCGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTAATATTACTGGTTGTGGAGGCTGCTTGTGTGGCATGTCGGCGACCGGTTTGTTAGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGATCGGCTTCGGCTGGGCAGACGCTTATTGGGTGCTTTTCTTGCTGTGGCGGTACGAACTGGTGAACCGTAGCTATGTTTGGCTTGGCTTTTGAACCGGCRATGTGGTGCGAAGTAGAGTGGCGGTTGTTCCAGCGCAAGCTGGAGTGGCTGTCGAGGGTCGATCCATTTGGGAAATTTGTGTCTGTGCGACTTCGGTTGCGCTGGCATCTCAA
>HQ917913 Phytophthora syringae
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAAAACCCTTTTATTGGGGGCTTCTGTCTGGTCTGGCTTCGGCTGGATTGGGTGGCGGCTCTATCATGGCGACCGCTCTGAGCTTCGGCCTGGAGCTAGTAGCCCACTTTTTAAACCCATTCTTAATTACTGAACAAACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGATGCACAGACGTGAAGTGTCTTGCGACTGGGCTTCGGCTCGGCTGCGAGTCCTTTTAAATGTACAGAACTGTACTTCTCTTTGCTCGAAAAGCGTTATATTACTGGTTGTGGAGGCTGCCTGTGCGGCAAGTCGGCGACCGGTTTGTTAACTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGATGGCTTCGGCTGAACTGACGCTTATTGAGTACTTTTCCTGCTGTGGCGGTACGAACTGGTGAACCGTAGCTGTGTTTGGCTTGGCTTTTGAACTGGCGATGTGGTGCGAAGTAGAGTGACGGTTGTTCCGGCGCAAGCTGGAGTGACTGTCGAGGGTCGATCCATTTGGGAAATTTTGTGTCTGTGCGACTTCGGTTGCGTGGGCATCTCAA
>HQ917914 Phytophthora syringae
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAAAACCCTTTTATTGGGGGCTTCTGTCTGGTCTGGCTTCGGCTGGATTGGGTGGCGGCTCTATCATGGCGACCGCTCTGAGCTTCGGCCTGGAGCTAGTAGCCCACTTTTTAAACCCATTCTTAATTACTGAACAAACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGATGCACAGACGTGAAGTGTCTTGCGACTGGGCTTCGGCTCGGCTGCGAGTCCTTTTAAATGTACAGAACTGTACTTCTCTTTGCTCGAAAAGCGTTATATTACTGGTTGTGGAGGCTGCCTGTGCGGCAAGTCGGCGACCGGTTTGTTAACTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGATGGCTTCGGCTGAACTGACGCTTATTGAGTACTTTTCCTGCTGTGGTGGTACGAACTGGTGAACCGTAGCTGTGTTTGGCTTGGCTTTTGAACTGGCGATGTGGTGCGAAGTAGAGTGACGGTTGTTCCGGCGCAAGCTGGAGTGACTGTCGAGGGTCGATCCATTTGGGAAATTTTGTGTCTGTGCGACTTCGGTTGCGTGGCATCTCAA
>FJ237515.2 Phytophthora multivora
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCTTTCAGTTGGGGGCCTTGCTTTTTTGCGAGCCCTATCATGGCGAATGTTTGGACTTCGGTCTGGGCTAGTAGCTTTTTGTTTTAAACCCATTCTACAATACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGATGTGCCAGATGTGAAGTGTCTTGCAGGTGTCCTTCGGGTCGTCTGCGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCTGCGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGGCGCTTATTGTATGCTTTTCCTGCTGTGGCGTGATGGACTGGTGAACCGTAGCTGTGTGTGGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTCCGGCTGTCGAGGGTCGATCCATTTGGGAACTTTGTGTGCACCTCGGTGCGCATCTCAA
>GQ303149.1 Phytophthora citricola III
CCACACCTAAAAAAACTTTTCACGTGAACCGTATCAACCCTTTTAGTTGGGGGTCTTGCTTTTTTGCGAGCCCTATCATGGCGAATGTTTGGACTTCGGTCTGGGCTAGTAGCTTTTTGTTTTAAACCCATTCTACAATACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGATGTGCCAGATGTGAAGTGTCTTGCGGGTGTCCTTCGGGTCGTCTGCGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGTGTTGCTGGTTGTGGAGGCTGCCTGCGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGGCGCTTATTGTATGCTTTTCCTGCTGTGGCGTGATGGGCTGGTGAACCGTAGCTGTGTGTGGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGACGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAACTTTGTGTGCACCTCGGTGCGCATCTCAA
>GU119914 Phytophthora niederhauserii
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACAAGTAGTTGGGGGCCTGCTCTGTGTGGCTGGCTGTCGATGTCAAAGTCGACGGCTGGCTGCTGTGTGGCGGGCTCTATCATGGCGATTGGTTTGGGTCCTCCTCGTGGGGAACTGGATCATGAGCCCACTTTTTAAACCCATTCTTAAATACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCTCTTCCTTCCGTGTAGTCGGTGGATGGAGACGCCAGACGTGAGGTGTCTTGCGGCGCGGCCTTCGGGCTGCCTGCGAGTCCCTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGTTGGTTGTGGAGGCTGCCTGTATGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGATGCTTTTCCTGCTGTGGCGGTATGGGCTGGTGAACCGTAGCTGTGCGAGGCTTGGCTTTTGAACCGGCGGTGTTGTTGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAACTCTGTGTTGTCTCTGCGGCTTGCTGCGGAGGTGGCATCTCAA
>FJ802007 Phytophthora cinncmomi var. parvispora
CCACACCTAAAAATCTTTCCACGTGAACCGTATCAACCCCCTTCGTTGGGGGCCTGCTCTGGGCGGCGGCTGCCGATGTCAAAGTCGGCGGCTGCTGCTGCGTGGCGGGCCCTATCACTGGCGAGCGTTTGGGTCCCTCTCGGGGGAACTGAGCTAGTAGCCCCTATTTTAAACCCATTCTGTAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCTCTTCCTTCCGTGTAGTCGGTGGATGGAGGCGCCAGACGTGAGGTGTCTTGCGGGCGGCCTTCGGGCTGTCTGTGAGTCCCTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGCTGGTTGTGGAGGCTGCCTGTATGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAAAGCGCTTATTGGATGTTCTTCCTGCTGTGGCGGTACGGATCGGTGAACCGTAGCTGTGCAAGGCTTGGCGTTTGAACCGGCGGTGTTGTGGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAACTCTGTGTCTTTTCTTGCGGCGCACGTTTGTGTGGTTGCGGGGGATGGGCATCTCAA
>GQ388261.1 Phytophthora cinnamomi var. parvispora
CCACACCTAAAAATCTTTCCACGTGAACCGTATCAACCCCCTTCGTTGGGGGCCTGCTCTGGGCGGCGGCTGTCGATGTCAAAGTCGGCGGCTGCTGCTGCGTGGCGGGCCCTATCACTGGCGAGCGTTTGGGTCCCTCTCGGGGGAACTGAGCTAGTAGCCCCTATTTTAAACCCATTCTGTAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCTCTTCCTTCCGTGTAGTCGGTGGATGGAGGCGCCAGACGTGAGGTGTCTTGCGGGCGGCCTTCGGGCTGTCTGTGAGTCCCTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGCTGGTTGTGGAGGCTGCCTGTATGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAAAGCGCTTATTGGATGTTCTTCCTGCTGTGGCGGTACGGATCGGTGAACCGTAGCTGTGCAAGGCTTGGCGTTTGAACCGGCGGTGTTGTGGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAACTCTGTGTCTTTTCTTGCGGCGCACGTTTGTGTGGTTGCGGGGGATGGCATCTCAATTGACCTGAATCAG
>JF701435.1 Phytophthora fluvialis
CCACACCTAAAAAAACTTTCCACGTGAACCGTATCAACCCTTTTTAATTGGGGGCTTGCTCGGCGGCGTGTGTGCTGGCCTGTAATGGGTCGGCGTGCTGCTGCTGGGCGGGCTCTATCATGGGCGAGCGTTTGGGCTTCGGCTCGAGCTAGTAGCTATCATTTTAAACTCATTCAATWATATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTYTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTAGCGGTCTTTCGAGTCTGCCGGTGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGTCCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACCGGTTTGTTTGCTGTGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGTGTGCTTTTCCTGTCATTGGCGATGCGAACTGGTGAACCGTAGCTATGTGGTGCTTGGCTTTTGAACGGGCTTTGCTTTGCGAAGTAGGGTGTCAGTTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAATTTWGTGTGTGCGGCTTCGTGCTGCGCGCATCTCAA
>JF701436.1 Phytophthora fluvialis
CCACACCTAAAAAAACTTTCCACGTGAACCGTATCAACCCTTTTTAATTGGGGGCTTGCTCGGCGGCGTGTGTGCTGGCCTGTAATGGGTCGGCGTGCTGCTGCTGGGCGGGCTCTATCATGGGCGAGCGTTTGGGCTTCGGCTCGAGCTAGTAGCTATCATTTTAAACTCATTCAATAATATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTAGCGGTCTTTCGAGTCTGCCGGTGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGTCCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACCGGTTTGTTTGCTGTGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGTGTGCTTTTCCTGTCATTGGCGATGCGAACTGGTGAACCGTAGCTATGTRGTGCTTGGCTTTTGAACGGGCTTTGCTTTGCGAAGTAGGGTGTCAGTTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAAYTTWGTGTGTGCGGCTTCGTGCTGCGCGCATCTCAA
>EU593261.2 Phytophthora fluvialis
CCACACCTAAAAAAACTTTCCACGTGAACCGTATCAACCCTTTTTAATTGGGGGCTTGCTCGGCGGCGTGTGTGCTGGCCTGAATGGGTCGGCGTGCTGCTGCTGGGCGGGCTCTATCATGGGCGAGCGTTTGGGCTTCGGCTCGAGCTAGTAGCTATCATTTTAAACTCATTCAATTATATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTAGCGGTCTTTCGAGTCTGCCGGTGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGTCCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACCGGTTTGTTTGCTGTGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGTGTGCTTTTCCTGTCATTGGCGATGCGAACTGGTGAACCGTAGCTATGTGGTGCTTGGCTTTTGAACGGGCTTTGCTTTGCGAAGTAGGGTGTCAGTTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAATTTTGTGTGTGCGGCTTCGTGCTGCGCGCATCTCAA
>HQ012953.1 Phytophthora taxon paludosa
CCACACCTAAAAAAACTTTCCACGTGAACCGTATCAACCCCTTTAATTGGGGGCTTGCTCGGCGGCGTGCGTGCTGGCCTGTAATGGGTCGGCGTGCTGCTGCTAGGCGGGCTCTATCATGGGCGAGCGTTTGGGCTTCGGCTCGAGCTAGTAGCTATTTTTTAAACCCATTCTTAATTACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTGGCGGTCTTTCGAGTCTGCCGGTGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGCTCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCTTGTCGGCGACCGGTTTGTTAGCTGTGACGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGGGCGCTTTTTCCTGTCATTGGCGATACGAACCGGTGAACCGTAGTCATGTGGTGCTTGGCTTTTGAACTGGCTTTGCTGTTGCGAAGTAGTGTGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAACTTTGTGTGTGCGGCTTCGTGCTGCGCGCATCTCAA
>JQ256377.1 Phytophthora bilorbang
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCCTTTAATTTGGGGGCTTGCTCGGCGGCGTGCGTGCTGGCCTGTAATGGGTCGGCGTGCTGCTGCTGGGCGGGCTCTATCATGGGCGAGCGCTTGGGCTTCGGCTCGAGCTAGTAGCTTTAATTTTAAACCCATTCTTAAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTGGCGGTCTTTCGAGTCTGCCGGTGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGCTCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCCGTGTGGCCAGTCGGCGACCGGTTTGTTAGCTGTGGCGTTTAATGGAGAAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGGGTGTTTTTCCTGTCATTGGCGGTACGAACTGGTGAACCGTAGCTGTGTGGTGCTTGGCTTTTGAACTGGCTTTGCTTTGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAACTTTGTGTGTGCGGCTTCGTGCTGCGCGCATCTCAA
>JQ814693.1 Phytophthora litchii
CCACACCTAAAAAACTTTCCACGTGAACCGTTTCAAAATTTAGTTGGGGGTCTTTTTCGGCGACGGCTGCTGGCTTCATTGCTGGTGGCTGTTGTTGGACGAGCCCTATCATGGCGAGCGTTTGGGCTTCGGTCTGAACTAGTAGCTTTTTTAAACCCATTCATTATTACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATGTGCCAGACGTGAAGTGTCTTGCAGCTGTCCTTCGGGTCGGCTGTGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGCGTTGCTGATTGTGGAGGCTGCTTGTGTAGCCAGTCTGGCGACCAGTTTGTCTGCTGTAGCGTTAATGGAGGAGTGTTTGATTCGCGGTATGGTTGGCTTCGGCTGAACAAACGCTTATTGAATGTTCTTTCTGCTATGGCGGTATGAGTTGGTGAACCGTAGCTATGTAAGCTTGGCTTTTGAATTGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAATTTGTGTGTGCTTCGGCATGCATCTCAA
>JQ626594.1 Phytophthora riparia
CAGGTCTGTGATGCCCTTAGATGTTCTGGGCCGCACGCGCGCTACACTGATGCGTTCAACGAGTATACAACCTTGATCGATAGGTCTGGGTAATCTTGTGAATGCGCATCGTGCTAGGGATAGACTGTTGCAATTTTCAGTCTTGAACGAGGAATTCCTAGTAAACGCAAGTCATCAGCTTGCATTGATTACGTCCCTGCCCTTTGTACACACCGCCCGTCGCACCTACCGATTGAATGACTCGGTGAAAAATTGGGACCGTGAGTCCGTTTGCTTCATTGCGAGTGGATTGATGGGAACTTTTTTAAACCTCGCCATTTAGAGGAAGGTGAAGTCGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTACCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCCTATAATTTGGGGGCTTGCTCGGCGGCGTGCGTGCTGGCCTGTAATGGGTCGGCGTGCTGCTGCTGGGCGGGCTCTATCATGGGCGAGCGCTTGGGCTTCGGCTCGAGCTAGTAGCTTTCCTTTTAAACCCATTCTTTAAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTRGCGGTCTTTCGAGTCTGCCGGTGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGCTCCAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCCGTGTGGCCAGTCGGCGACCGGTTTGTTAGCTGTGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGGGTGCTTTTCCTGCTGCAGCGGTATGAACTGGTGAACCGTAGCTGTGTGGTGCTTGGCTTTTGAACCGGCTTTGCTTTGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAAGTTTGTGTGTGCGGCTTCGTGCTGCGCGCATCTCAA
>JQ626602.1 Phytophthora riparia
CAGGTCTGTGATGCCCTTAGATGTTCTGGGCCGCACGCGCGCTACACTGATGCGTTCAACGAGTATACAACCTTGATCGATAGGTCTGGGTAATCTTGTGAATGCGCATCGTGCTAGGGATAGACTGTTGCAATTTTCAGTCTTGAACGAGGAATTCCTAGTAAACGCAAGTCATCAGCTTGCATTGATTACGTCCCTGCCCTTTGTACACACCGCCCGTCGCACCTACCGATTGAATGACTCGGTGAAAAATTGGGACCGTGAGTCCGTTTGCTTCATTGCGAGTGGATTGATGGGAACTTTTTTAAACCTCGCCATTTAGAGGAAGGTGAAGTCGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTACCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCCTATAATTTGGGGGCTTGCTCGGCGGCGTGCGTGCTGGCCTGTAAYGGGTCGGCGTGCTGCTGCTGGGCGGGCTCTATCATGGGCGAGCGCTTGGGCTTCGGCTCGAGCTAGTAGCTTTCCTTTTAAACCCATTCTTTAAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTGGCGGTCTTTCGAGTCTGCCGGTGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGCTCCAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCCGTGTGGCCAGTCGGCGACCGGTTTGTTAGCTGTGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGGGTGCTTTTCCTGCTGCAGCGGTATGAACTGGTGAACCGTAGCTGTGTGGTGCTTGGCTTTTGAACCGGCTTTGCTTTGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAAGTTTGTGTGTGCGGCTTCGTGCTGCGCGCATCTCAA
>JQ626600.1 Phytophthora riparia
CAGGTCTGTGATGCCCTTAGATGTTCTGGGCCGCACGCGCGCTACACTGATGCGTTCAACGAGTATACAACCTTGATCGATAGGTCTGGGTAATCTTGTGAATGCGCATCGTGCTAGGGATAGACTGTTGCAATTTTCAGTCTTGAACGAGGAATTCCTAGTAAACGCAAGTCATCAGCTTGCATTGATTACGTCCCTGCCCTTTGTACACACCGCCCGTCGCACCTACCGATTGAATGACTCGGTGAAAAATTGGGACCGTGAGTCCGTTTGCTTCATTGCGAGTGGATTGATGGGAACTTTTTTAAACCTCGCCATTTAGAGGAAGGTGAAGTCGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTACCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCCTATAATTTGGGGGCTTGCTCGGCGGCGTGCGTGCTGGCCTGTAATGGGTCGGCGTGCTGCTGCTGGGCGGGCTCTATCATGGGCGAGCGCTTGGGCTTCGGCTCGAGCTAGTAGCTTTCCTTTTAAACCCATTCTTTAAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTGGCGGTCTTTCGAGTCTGCCGGTGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGCTCCAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCCGTGTGGCCAGTCGGCGACCGGTTTGTTAGCTGTGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGGGTGCTTTTCCTGCTGCAGCGGTATGAACTGGTGAACCGTAGCTGTGTGGTGCTTGGCTTTTGAACCGGCTTTGCTTTGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAAGTTTGTGTGTGCGGCTTCGTGCTGCGCGCATCTCAA
>JQ626598.1 Phytophthora borealis
CAGGTCTGTGATGCCCTTAGATGTTCTGGGCCGCACGCGCGCTACACTGATGCGTTCAACGAGTATACAACCTTGATCGATAGGTCTGGGTAATCTTGTGAATGCGCATCGTGCTAGGGATAGACTGTTGCAATTTTCAGTCTTGAACGAGGAATTCCTAGTAAACGCAAGTCATCAGCTTGCATTGATTACGTCCCTGCCCTTTGTACACACCGCCCGTCGCACCTACCGATTGAATGACTCGGTGAAAAATTGGGACCGTGAGTCCGTTTGCTTCATTGCGAGTGGATTGATGGGAACTTTTTTAAACCTCGCCATTTAGAGGAAGGTGAAGTCGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTACCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCCTTAAATTTGGGGGCTTGCTCGGCGGCGTGCGTGCTGGCCTGTAATGGGTCGGCGTGYTGCTGCTGGGCGGGCTCTATCATGGGCGAGCGTTTGGGCTTCGGCTCGAGCTAGTAGCTATCAATTTTAAACCCCTTCTTAAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTGGCGGTCTTTCGAGTCTGCCGGTGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGCTCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCCGTGTGGCCAGTCGGCGACCGGTTTGTTACCTGTGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGGGTGCTTTTCCTGTCATTGGCGGTACGAACTGGTGAACCGTAGCTGTGTGGTGCTTGGCTTTTGAACCGGCTTTGCTTTGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAACTTTGTGTGTGCGGCTTCGTGCTGCGCGCATCTCAA
>GU191210.1 Phytophthora cinnamomi
AGGATCATTACCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCAATTAGTTGGGGGCCTGCTCTGGGCGGCGGCTGTCGATGTCAAAGTCGACGGCTGCTGCTGCGTGGCGGGCCCTATCACTGGCGAGCGTTTGGGTCCCTCTCGGGGGAACTGAGCTAGTAGCCTCTCTTTTAAACCCATTCTGTAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCTCTTCCTTCCGTGTAGTCGGTGGATGGAGGTGCCAGACGTGAGGTGTCTTGCGGGCGGTCTTCGGACTGGCTGTGAGTCCCTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGCTGGTTGTGGAGGCTGCCTGTATGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAAAGCGCTTATTGGATGTTCTTCCTGCTGTGGCGGTACGGATCGGTGAACCGTAGCTGTGCTAGGCTTGGCGTTTGAACCGGCGGTGTTGTTGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAACTCTGTGTCTCTGCGGCGCACTTGTGTGCTTGTGGTGGCATCTCAATTGGACCTGATATCAGG
>JX276017.1 Phytophthora polonica
CCACACCTAAAAACTGTCCACGTGAACTGTGCTGAAGATGATTGGGGCGTGCTGGGCGTGCGTGTGCGGCTTCGGCTGCGTGCGTGTGTTTGGCGTGCCCATCAAACGAGGAGGCGGGTCGCAAGGCCTGCTTTAGTAGTTACTATTTTAAACCCTTTTACTGTTAACTGATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAAGCCTGGGAGTATGCCTGTATCAGTGTCCGTACAACAACCTTGGCTTCCTTCCTTCCGTGTAGTCGGTGGCGGGAACGCTCAGACGTGAAGTGTCTTGCTGTCAGTAATTTGTAGATTGCTAAGGCGAGTCCTTTGAAAAGTAGGACGTGTTCTTCTCTTTGCTGGAAAAGCGTTGCGGTGCGTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACTGATTTCGTGCTGATGCGTGTGGGGAGTGGTTCGATTCGCGGTATGGTTGGCTTCGGCTGATTTTCTGCTTATTGGGTCGTTTTCTGCTGCGTTGGCGTTTGCGGGGTCGGTGAACCGTAGCTGCGTTGGGCTTGGCTTTTGAACTGTGTGCTGTGCTGCGAAGTATGGTGGCTGCGTGCGCCTTCGGGTGCGTGCGGCCGAGAGACGGTCACCATTTGGGAAGTTTGTGCTTTGGGCCTCGGCCTTTGGCATCTCAA
>HQ261646.1 Phytophthora polonica
CCACACCTAAAAACTGTCCACGTGAACTGTGCTGAAGATGATTGGGGCGTGCTGGGCGTGCGTGTGCCGCTTCGGCTGCGTGCGTGTGTTTGGCGTGCCCATCAAACGAGGAGGCGGGTCGCAAGGCCTGCTTTAGTAGTTACTATTTTAAACCCTTTTACTGTTAACTGATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAAGCCTGGGAGTATGCCTGTATCAGTGTCCGTACAACAACCTTGGCTTCCTTCCTTCCGTGTAGTCGGTGGCGGGAACGCTCAGACGTGAAGTGTCTTGCCGTCAGTAATTTGTAGATTGCTAAGGCGAGTCCTTTGAAAAGTAGGACGTGTTCTTCTCTTTGCTGGAAAAGCGTTGCGGTGCGTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACTGATTTCGTGCTGATGCGTGTGGGGAGTGGTTCGATTCGCGGTATGGTTGGCTTCGGCTGATTTTCTGCTTATTGGGTCGTTTTCTGCTGCGTTGGCGTTTGCGGGGTCGGTGAACCGTAGCTGCGTTGGGCTTGGCTTTTGAACTGTGTGCTGTGCTGCGAAGTATGGTGGCTGCGTGCGCCTTCGGGTGCGTGCGGCCGAGAGACGGTCACCATTTGAGAAGTTTGTGCTTTGGGCCTCGGCCTTTGGCATCTCAA
>HQ261647.1 Phytophthora polonica
CACACTTAAAAACTGTCCACGTGACTGTGCTGAAGATGATTGGGGCGTGCTGGGCGTGCGTGTGCGGCTTCGGCTGCGTGCGTGTGTTTGGCGTGCCCATCAAACGAGGAGGCGGGTCGCAAGGCCTGCTTTAGTAGTTACTATTTTAAACCCTTTTACTGTTAACTGATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAAGCCTGGGAGTATGCCTGTATCAGTGTCCGTACAACAACCTTGGCTTCCTTCCTTCCGTGTAGTCGGTGGCGGGAACGCTCAGACGTGAAGTGTCTTGCCGTCAGTAATTTGTAGATTGCTAAGGCGAGTCCTTTGAAAAGTAGGACGTGTTCTTCTCTTTGCTGGAAAAGCGTTGCGGTGCGTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACTGATTTCGTGCTGATGCGTGTGGGGAGTGGTTCGATTCGCGGTATGGTTGGCTTCGGCTGATTTTCTGCTTATTGGGTCGTTTTCTGCTGCGTTGGCGTTTGCGGGGTCGGTGAACCGTAGCTGCGTTGGGCTTGGCTTTTGAACTGTGTGCTGTGCTGCGAAGTATGGTGGCTGCGTGCGCCTTCGGGTGCGTGCGGCCGAGAGACGGTCACCATTTGGAAGTTTGTGACTTTGGGCCTCGGGCCTTTGGCATCTAA
>AF266793.2 Phytophthora lacustris
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCCTATAATTCGGGGGCTTGCTCGGCGGCGTGCGTGCTGGCCTGTAATGGGTCGGCGTGCTGCTGCTGGGCGGGCTCTATCATGGGCGAGCGCTTGGGCTTCGGCTCGAGCTAGTAGCTTTTTCTTTTAAACCCATTCTTTAAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTGGCGGTCTTTCGAGTCTGCCGGTGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGCTCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCCGTGTGGCCAGTCGGCGACCGGTTCGTTAGCTGTGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGGGTGCTTTTCCTGCTGCAGCGGTATGAACTGGTGAACCGTAGCTGTGTGGTGCTTGGCTTTTGAACCGGCTTTGCTTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAACTTTGTGTGTGCGGCTTCGTGCTGCGCGCATCTCAA
>HQ261507.1 Phytophthora botryosa
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCTTTTAGTTGGGGGTGTTGCTTGGCGTTTTGCTGAGCCGCGCCCTATCATGGCGAATGTTTGGGCTTCGGTCTGGGCTAGTAGCTTTTTGTTTTAAACCCATTTAACAATACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGATGTGCCAGATGTGAAGTGTCTTGCGGTTTTTGTGCCTTCGGGCCGTGGGCCGCGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGTGTTGCTGGTTGTGGAGGCTGCCTGCGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGGCGCTTATTGTATGCTTTTCCTGCTGTGGCGTGATGGGCTGGTGAACCGTAGCTGTGTGTGGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGTGTCGATCCATTTTGGGAACTTTTGTGTGCGCTTTCGAGTGTGCATCTCAA
>HQ261508.1 Phytophthora botryosa
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCTTTTAGTTGGGGGTGTTGCTTGGCGTTTTGCTGAGCCGCGCCCTATCATGGCGAATGTTTGGACTTCGGTCTGGGCTAGTAGCTTTTTTGTTTTAAACCCATTTAACAATACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGATGTGCCAGATGTGAAGTGTCTTGCGGTTTTTGTGCCTTCGGGCCGTGGCTGCGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGTGTTGCTGGTTGTGGAGGCTGCCTGCGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGGCGCTTATTGTATGCTTTTCCTGCTGTGGCGTGATGGGCTGGTGAACCGTAGCTGTGTGTGGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGTGTCGATCCATTTTGGGAAATTTTGTGTGCGCTTTCGAGTGTGCATCTCAA
>HQ261509.1 Phytophthora botryosa
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCTTTTAGTTGGGGGTGTTGCTTGGCGTTTTGCTGAGCCGCGCCCTATCATGGCGAATGTTTGGACTTCGGTCTGGGCTAGTAGCTTTTTTGTTTTAAACCCATTTAACAATACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGATGTGCCAGATGTGAAGTGTCTTGCGGTTTTTGTGCCTTCGGGCCGTGGCCGCGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGTGTTGCTGGTTGTGGAGGCTGCCTGCGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGGCGCTTATTGTATGCTTTTCCTGCTGTGGCGTGATGGGCTGGTGAACCGTAGCTGTGTGTGGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGTGTCGATCCATTTTGGGAAATTTTGTGTGCGCTTTCGAGTGTGCATCTCAA
>KC529657.1 Phytophthora pluvialis
CAGGTCTGTGATGCCCTTAGATGTTCTGGGCCGCACGCGCGCTACACTGATGCGTTCAACGAGTATACAACCTTGATCGATAGGTCTGGGTAATCTTGTGAATGCGCATCGTGCTAGGGATAGACTGTTGCAATTTTCAGTCTTGAACGAGGAATTCCTAGTAAACGCAAGTCATCAGCTTGCATTGATTACGTCCCTGCCCTTTGTACACACCGCCCGTCGCACCTACCGATTGAATGACTCGGTGAAAAATTGGGACCGTGAGTCCGTTTGCTTCATTGYGAGTGGATTGATGGGAACTTTTTTAAACCTCGCCATTTAGAGGAAGGTGAAGTCGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTACCACACCTAAAAAACTTTCCACGTGAACTGTATCGAACAACTAGTTGGGGGTCTTGTTTGGCGTGCGGCTGCTTCGGTAGCTGCTGCTAGGCGAGCCCTATCACGGCGAGCGTTTGGACTTCGGTCTGAACTAGTAGCTATTTTTTAAACCCATTCTTTAATACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGAGACGCCAGACGTGAAGTGTCTTGCGGCGAATTTTCGAATTTGCTGTGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCCGCGTGGCCAGTCGGCGACCGGTTTGTCTGCTGTGGCATTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAAGCGCTTATTGGATGTTCTTCTTGCTGTGGCGTGACGGACTGGTGAACCGTAGCTATGTGTGGCTTGGCTTTTGAATCGGCTTTGCTGTTGCAAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAAATTTGTGTGTATGACTTCGGTTGTATGCATCTCAATT
>HQ643205.1 Phytophthora citrophthora
CCACACCTAAAAAAACTTTCCACGTGAACCGTATCAACCCTTTTAGTTGGGGGTGTTGCTTGGCGTTTTTGCTGAGCCACGCCCTATCATGGCGAATGTTTGGACTTCGGTCTGGGCTAGTAGCTTTTTTGTTTTAAACCCATTTAACAATACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGATGTGCCAGATGTGAAGTGTCTTGCGGTTTTTGTGCCTTCGGGCCATGGCTGCGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGTGTTGCTGGTTGTGGAGGCTGCCTGCGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCAGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGGCGCTTATTGTATGCTTTTCCTGCTGTGGCGTGATGGGCTGGTGAACCGTAGCTGTGTGTGGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGTGTCGATCCATTTTGGGAACGTTTGTGTGCGCTTTCGAGTGTGCATCTCAA
>HQ643207.1 Phytophthora citrophthora
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCTTTTAGTTGGGGGTGTTGCTTGGCGTTTTGCTGAGCCACGCCCTATCATGGCGAATGTTTGGACTTCGGTCTGGGCTAGTAGCTTTTTTGTTTTAAACCCATTTAACAATACTGATTTTACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGATGTGCCAGATGTGAAGTGTCTTGCGGTTTTTGTGCCTTCGGGCCGTGGCTGCGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGTGTTGCTGGTTGTGGAGGCTGCCTGCGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGGCGCTTATTGTATGCTTTTCCTGCTGTGGCGTGATGGGCTGGTGAACCGTAGCTGTGTGTGGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGTGTCGATCCATTTTGGGAACGTTTGTGTGCGCTTTCGAGTGTGCAT
>HQ643208.1 Phytophthora citrophthora
CCACACCTAAAAAAACTTTCCACGTGAACCGTATCAACCCTTTTAGTTGGGGGTGTTGCTTGGCGTTTTTGCTGAGCCACGCCCTATCATGGCGAATGTTTGGACTTCGGTCTGGGCTAGTAGCTTTTTTGTTTTAAACCCATTTAACAATACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGATGTGCCAGATGTGAAGTGTCTTGCGGTTTTTGTGCCTTCGGGCCATGGCTGCGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGTGTTGCTGGTTGTGGAGGCTGCCTGCGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCRGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGGCGCTTATTGTATGCTTTTCCTGCTGTGGCGTGATGGGCTGGTGAACCGTAGCTGTGTGTGGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGTGTCGATCCATTTTGGGAACGTTTGTGTGCGCTTTCGAGTGTGCAT
>AY659426.1 Phytophthora cryptogea
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCTTTTTAAATTGGGGGCTTCCGTCTGGCCGGCCGGTTCTCGGCTGGATGGGTGGCGGCTCTATCATGGCGACCGCTTGGGCCTCGGCCTGGGCTAGTAGCGTATTTTTTAAACCCATTCCTAATTACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACACTAAACTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGACGCGCAGATGTGAAGTGTCTTGCGGCTGGTCTTCGGTCCGGCTGCGAGTCCTTTGAAATGTACTACACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCATGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGATCGGCTTCGGCTGAACAGACGCTTATTGGGTGCTTTTCCTGCTGTGGCTGGATGGACTGGTGAACCGTAGCTGTGCTAGGCTTGGCTTTTGAACCGGCGGTGTGGTGCGAAGTAGGGTGTCTGTTCCGGCGCAAGCTGGGGTGGACGAGGGTCGATCCATTTGGGAAACGTTGTGTGCGCTTCGGCGCGCATCTCA
>AY659450.1 Phytophthora cryptogea
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCTTTTTAAATTGGGGGCTTCCGTCTGGCCGGCCGGYTCTCGGCTGGCTGGGTGGCGGCTCTATCATGGCGACCGCTTGGGCCTCGGCCTGGGCTAGTAGCRTATTTTTTAAACCCATTCCTAATTACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACACTAAACTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGACGCGCAGATGTGAAGTGTCTTGCGGCTGGTCTTCGGTCCGGCTGCGAGTCCTTTGAAATGTACTACACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCATGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGATCGGCTTCGGCTGAACAGACGCTTATTGGGTGCTTTTCCTGCTGTGGCTGGATGGACTGGTGAACCGTAGCTGTGCTAGGCTTGGCTTTTGAACCGGCGGTGTGGTGCGAAGTAGGGTGTCTGTTCCGGCGCAAGCTGGGGTGGACGAGGGTCGATCCATTTGGGAAACGTTGTGTGCGCTTCGGCGCGCATCTCA
>AY659428.1 Phytophthora cryptogea
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCTTTTTAATCCGGGGGCTTCCGTCTGGCCGGCCGGTTTTCGGCTGGCTGGGTGGCGGCTCTATCATGGCGACCGCTTGGGCCTCGGCCTGGGCTAGTAGCGTATTTTTAAACCCATTCCTAATTACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACACTAAACTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGACGCGCAGATGTGAAGTGTCTTGCGGCTGGTCTTCGGTCCGGCTGCGAGTCCTTTGAAATGTACTACACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCATGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGACGCTTATTGGGTGCTTTTCCTGCTGTGGCTGGATGGACTGGTGAACCGTAGCTGTGCTAGGCTTGGCGTTTGAACCGGCGGTGTGGTGCGAAGTAGGGTGTCTGTTCTGGCGTAAGCTGGGGTGGACGAGGGTCGATCCATTTGGGAAACGTTGTGTGCGCTTCGGCGCGCATCTCA
>AY659421.1 Phytophthora cryptogea
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCTTTTTAAATTGGGGGCTCCCGTCTGGCCGGCCGGTTTTCGGCTGGCTGGGTGGCGGCTCTATCATGGCGACCGCTTGGGCCTCGGCCTGGGCTAGTAGCGTATTTTTAAACCCATTCCTAATTACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACACTAAACTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGACGCGCAGATGTGAAGTGTCTTGCGGCTGGTCTTCGGTCCGGCTGCGAGTCCTTTGAAATGTACTACACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCATGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGACGCTTATTGGGTGCTTTTCCTGCTGTGGCTGGATGGACTGGTGAACCGTAGCTGTGCTAGGCTTGGCGTTTGAACCGGCGGTGTGGTGCGAAGTAGGGTGTCTGTTCYGGCGYAAGCTGGGGTGGACGAGGGTCGATCCATTTGGGAAACGTTGTGTGCGCTTCGGCGCGCATCTCA
>AY659431.1 Phytophthora cryptogea
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCTTTTTAAATTGGGGGCTCCCGTCTGGCCGGCCGGTTYTCGGCTGGCTGGGTGGCGGCTCTATCATGGCGACCGCTTGGGCCTCGGCCTGGGCTAGTAGCGTATTTTTTAAACCCATTCCTAATTACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACACTAAACTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGACGCGCAGATGTGAAGTGTCTTGCGGCTGGTCTTCGGTCCGGCTGCGAGTCCTTTGAAATGTACTACACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCATGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGATYGGCTTCGGCTGAACAGACGCTTATTGGGTGCTTTTCCTGCTGTGGCTGGATGGACTGGTGAACCGTAGCTGTGCTAGGCTTGGCGTTTGAACCGGCGGTGTGGTGCGAAGTAGGGTGTCTGTTCCGGCGCAAGCTGGGGTGGACGAGGGTCGATCCATTTGGGAAACGTTGTGTGCGCTTCGGCGCGCATCTCA
>AY659439.1 Phytophthora cryptogea
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCTTTTTAAATTGGGGGCTCCCGTCTGGCCGGCCGGTTTTCGGCTGGCTGGGTGGCGGCTCTATCATGGCGACCGCTTGGGCCTCGGCCTGGGCTAGTAGCGTATTTTTAAACCCATTCCTAATTACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACACTAAACTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGACGCGCAGATGTGAAGTGTCTTGCGGCTGGTCTTCGGTCCGGCTGCGAGTCCTTTGAAATGTACTACACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCATGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGACGCTTATTGGGTGCTTTTCCTGCTGTGGCTGGATGGACTGGTGAACCGTAGCTGTGCTAGGCTTGGCGTTTGAACCGGCGGTGTGGTGCGAAGTAGGGTGTCTGTTCCGGCGCAAGCTGGGGTGGACGAGGGTCGATCCATTTGGGAAACGTTGTGTGCGCTTCGGCGCGCATCTCA
>AY659422.1 Phytophthora cryptogea
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCTTTTTAAATTGGGGGCTCCCGTCTGGCCGGCCGGTTTTCGGCTGGCTGGGTGGCGGCTCTATCATGGCGACCGCTTGGGCCTCGGCCTGGGCTAGTAGCGTATTTTTTAAACCCATTCCTAATTACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACACTAAACTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGACGCGCAGATGTGAAGTGTCTTGCGGCTGGTCTTCGGTCCGGCTGCGAGTCCTTTGAAATGTACTACACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCATGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGATTGGCTTCGGCTGAACAGACGCTTATTGGGTGCTTTTCCTGCTGTGGCTGGATGGACTGGTGAACCGTAGCTGTGCTAGGCTTGGCGTTTGAACCGGCGGTGTGGTGCGAAGTAGGGTGTCTGTTCCGGCGCAAGCTGGGGTGGACGAGGGTCGATCCATTTGGGAAACGTTGTGTGCGCTTCGGCGCGCATCTCA
>AY659434.1 Phytophthora cryptogea
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCTTTTTAAATTGGGGGCTCCCGTCTGGCCGGCCGGTTCTCGGCTGGCTGGGTGGCGGCTCTATCATGGCGACCGCTTGGGCCTCGGCCTGGGCTAGTAGCGTATTTTTTAAACCCATTCCTAATTACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACACTAAACTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGACGCGCAGATGTGAAGTGTCTTGCGGCTGGTCTTCGGTCCGGCTGCGAGTCCTTTGAAATGTACTACACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCATGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGATTGGCTTCGGCTGAACAGACGCTTATTGGGTGCTTTTCCTGCTGTGGCTGGATGGACTGGTGAACCGTAGCTGTGCTAGGCTTGGCGTTTGAACCGGCGGTGTGGTGCGAAGTAGGGTGTCTGTTCCGGCGCAAGCTGGGGTGGACGAGGGTCGATCCATTTGGGAAACGTTGTGTGCGCTTCGGCGCGCATCTCA
>AY659446.1 Phytophthora drechsleri
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCTTTTTAAATTGGGGGCTTCCGTCTGGCCGGCCGGTTCTCGGCTGGCTGGATGGCGGCTCTATCATGGCGACCGCCTGGGCCTCGGCCTGGGCTAGTAGCGTATTTTTTAAACCATTCCTAATTACTGAAAAAACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACACTAAACTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGACGCGCAGATGTGAAGTGTCTTGCGGCTGGTCTTCGGTCCGGCTGCGAGTCCTTTTAAATGTACTACACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCATGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAAAGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGACGCTTATTGGGTGCTTTTTCTGCTGCGGCGGGATGGACTGGTGAACCGTAGCTGTGCTAGGCTTGGCGTTTGAACCGGCGGTATTGTGCGAAGTAGGGTGTCTGTCTCGGCGCAAGCTGGGGTGGGCGAGGGTCGATCCATTTGGGAAACGTTGTGTGCGCTTCGGCGTGCATCTCA
>AY659435.1 Phytophthora drechsleri
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCTTTTTAAATTGGGGGCTTCCGTCTGGCCGGCCGGTTCTCGGCTGGCTGGGTGACGGCTCTATCATGGCGACCGCCTGGGCCTCGGCCTGGGTTAGTAGCGTATTTTTTAAACCATTCCTAATTACTGAAAAAACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACACTAAACTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGACGCGCAGATGTGAAGTGTCTTGCGGCTGGTCTTCGGTCCGGCTGCGAGTCCTTTTAAATGTACTACACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCATGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGACGCTTATTGGGTGCTTTTTCTGCTGCGGCGGGATGGACTGGTGAACCGTAGCTGTGCTAGGCTTGGCGTTTGAACCGGCGGTATTGTGCGAAGTAGGGTGTCTGTCTCGGCGYAAGCTGGGGTGGGCGAGGGTCGATCCATTTGGGAAACGTTGTGTGCGCTTCGGCGTGCATCTCA
>AY659457.1 Phytophthora drechsleri
CCACACCTAAAAAACTTCCACGTGAACCGTACTCAACCTTTTCAAATAGGGGGCTTCCGTCTGGCCGGCCGGTTCTCGGCTGGCTGGGTGGCGGCTCTATCATGGCGACCGCCTGGGCCTCGGCCTGGGCTAGTAGCGTATTTTTTAAACCATTCCTAATTACTGAAAAAACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACACTAAACTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGACGCGCAGATGTGAAGTGTCTTGCGGCTGGTCTTCGGTCCGGCTGCGAGTCCTTTTAAATGTACTACACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCATGTCGCCGACCGGTTTGTCTGCTGCGGCGTTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGACGCTTATTGGGTGCTTTTTCTGCTGCGGCGGGATGGACTGGTGAACCGTAGCTGTGCTAGGCTTGGCGTTTGAACCGGCGGTATTGTGCGAAGTAGGGTGTCTGTCTCGGCGCAAGCTGGGGTGGGCGAGGGTCGATCCATTTGGGAAACGTTGTGTGCGCTTCGGCGTGCATCTCA
>AY659458.1 Phytophthora drechsleri
CCACACCTAAAAAACTTCCACGTGAACCGTACTCAACCTTTTTAAATAGGGGGCTTCCGTCTGGCCGGCCGGTTCTCGGCTGGCTGGGTGGCGGCTCTATCATGGCGACCGCCTGGGCCTCGGCCTGGGCTAGTAGCGTATTTTTTAAACCATTCCTAATTACTGAAAAAACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACACTAAACTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGACGCGCAGATGTGAAGTGTCTTGCGGCTGGTCTTCGGTCCGGCTGCGAGTCCTTTTAAATGTACTACACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCATGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGACGCTTATTGGGTGCTTTTTCTGCTGCGGCGGGATGGACTGGTGAACCGTAGCTGTGCTAGGCTTGGCGTTTGAACCGGCGGTATTGTGCGAAGTAGGGTGTCTGTCTCGGCGCAAGCTGGGGTGGGCGAGGGTCGATCCATTTGGGAAACGTTGTGTGCGCTTCGGCGTGCATCTCA
>AY659464.1 Phytophthora drechsleri
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCTTTTTAAATTGGGGGCTTCCATCTGGCCGGCCGGTTCTCGGCTGGCTGGGTGGCGGCTCTATCATGGCGACCGCCTGGGCCTCGGCCTGGGTTAGTAGCGTATTTTTTAAACCATTCCTAATTACTGAAAAAACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACACTAAACTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGACGCGCAGATGTGAAGTGTCTTGCGGCTGGTCTTCGGTCCGGCTGCGAGTCCTTTTAAATGTACTACACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCATGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGACGCTTATTGGGTGCTTTTTCTGCTGCGGCGGGATGGACTGGTGAACCGTAGCTGTGCTAGGCTTGGCGTTTGAACCGGCGGTATTGTGCGAAGTAGGGTGTCTGTCTCGGCGCAAGCTGGGGTGGGCGAGGGTCGATCCATTTGGGAAACGTTGTGTGCGCTTCGGCGTGCATCTCA
>AY659449.1 Phytophthora erythroseptica
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCTTTTTAAATTGGGGGCTTCCGTCTGGCCGGCCGGTTTTCGGCTGGCTGGGTGGCGGCTCTATCATGGCGACCGCTTGGGCCTCGGCCTGGGCTAGTAGCGTATTTTTAAACCATTCCTAATTACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACACTAAACTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGACGCGCAGATGTGAAGTGTCTTGCGGCTGGTCTTCGGTCCGGCTGCGAGTCCTTTGAAATGTACTACACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCATGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGACGCTTATTGGGTGCTTTTCCTGCTGTGGCTGGATGGACTGGTGAACCGTAGCTGTGCTAGGCTTGGCGTTTGAACCGGCGGTGTGGTGCGAAGTAGGGTGTCTGTTCCGGCGTAAGCTGGGGTGGACGAGGGTCGATCCATTTGGGAAACGTTGTGTGCGCTTCGGCGCGCATCTCA
>AY659445.1 Phytophthora erythroseptica
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCTTTTTAAATTGGGGGCTTCCGTCTGGCCGGCCGGTTCTCGGCTGGCTGGGTGGCGGCTCTATCATGGCGACCGCTTGGGCCTCGGCCTGGGCTAGTAGCGTATTTTTTAAACCCATTCCTAATTACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACACTAAACTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGACGCGCAGATGTGAAGTGTCTTGCGGCTGGTCTTCGGTCCGGCTGCGAGTCCTTTGAAATGTACTACACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCATGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGATCGGCTTCGGCTGAACAGACGCTTATTGGGTGCTTTTCCTGCTGTGGCTGGATGGACTGGTGAACCGTAGCTGTGCTAGGCTTGGCTTTTGAACCGGTGGTGTGGTGCGAAGTAGGGTGTCTGTTCCGGCGCAAGCTGGGGTGGACGAGGGTCGATCCATTTGGGAAACGTTGTGTGCGCTTCGGCGCGCATCTCA
>EU240038.1 Phytophthora taxon oaksoil
AGGTGAACCTGCGGAAGGATCATTACCACACCTAAAAAACTTTCCACGTGAACCGTAACAACCCCTTTAATTTGGGGGCTTGCTCGGCGGCGTGCGTGCTGGCCTGTAATGGGTCGGCGTGCTGCTGCTGGGCGGGCTCTATCATGGGCGAGCGCTTGGGCTTCGGCTCGAGCTAGTAGCTTTAATTTTAAACCCATTCTTAAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTGGCGGTCTTTCGAGTCTGCCGGTGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGCTCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCCGTGTGGCCAGTCGGCGACCGGTTTGTTAGCTGTGGCGTTTAATGGAGAAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGGGTGTTTTTCCTGTCATTGGCGGTACGAACTGGTGAACCGTAGCTGTGTGGTGCTTGGCTTTTGAACTGGCTTTGCTTTGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAACTTTGTGTGTGCGGCTTCGTGCTGCGCGCATCTCA
>KC855330.1 Phytophthora pachypleura
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCTTTTAGTTGGGGGTCTTGCTTTTTTGCGAGCCCTATCATGGCGAATGTTTGGACTTCGGTCTGGGCTAGTAGCTTTTTGTTTTAAACCCATTCTACAATACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGATGTGCCAGATGTGAAGTGTCTTGCGGGTGTCCTTCGGGTCGTCTGCGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGTGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGTGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGGCGCTTATTGTATGCTTTTCCTGCTGTGGCGTGATGGGCTGGTGAACCGTAGCTGTGTGTGGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAACTTTGTGTGCACCTCGGTGCGCATCTCAA
>KF112853.1 Phytophthora mississippiae
CCACACCTAAAAAAACTTTCCACGTGAACCGTATCAACCCCTTAAATTTGGGGGCTTGCTCGGCGGCGTGCGTGCTGGCCTGTAATGGGTCGGCGTGCTGCTGCTGGGCGGGCTCTATCATGGGCGAGCGTTTGGGCTTCGGCTCGAGCTAGTAGCTATCAATTTTAAACCCTTTCTTAAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTGGCGGTCTTTCGAGTCTGCCGGTGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGCTCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCCGTGTGGCCAGTCGGCGACCGGTTTGTTAGCTGTGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGTGTGCTTTTCCTGTCATTGGCGGTACGAACTGGTGAACCGTAGCTGTGTGGTGCTTGGCTTTTGAACCGGCTTTGCTTTGCGAAGTAGTGTGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAACTTTGTGTGTGCGGCTTCGTGCTGCGCGCATCTCAA
>KF112851.1 Phytophthora mississippiae
CCACACCTAAAAAAACTTTCCACGTGAACCGTATCAACCCCTTAAATTTGGGGGCTTGCTCGGCGGCGTGCGTGCTGGCCTGTAATGGGTCGGCGTGCTGCTGCTAGGCGGGCTCTATCATGGGCGAGCGTTTGGGCTTCGGCTCGAGCTAGTAGCTATCAATTTTAAACCCCTTCTTAAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTGGCGGTCTTTCGAGCCTGCCGGTGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGCTCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCCGTGTGGCCAGTCGGCGACCGGTTTGTTAGCTGTGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGTGTGCTTTTCCTGTCATTGGCGGTACGAACTGGTGAACCGTAGCTGTGTGGTGCTTGGCTTTTGAACCGGCTTTGCTTTGCGAAGTAGTGTGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAACTTTGTGTGTGCGGCTTCGTGCTGCGCGCATCTCAA
>HQ261638.1 Phytophthora parsiana
CCACACCTAAAAACTTTCCACGTGAACTGTCTGTGATGTTGGGGGCCTGTCTGCCTGTTGGCGGGCGGCTCCAACAAACGAGGCTCTGGGCTGAAAAGTCGAGGGTAGTAGTTACTTTTTGTAAACCCTTACTCATCCTTTCTGATCATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTACGCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGCGGGGACGCGCAGACGTGAAGTGTCTTGCTGGGCTTTGTGGCCGGCGAGTCCTTTGAAATGTAAGATACTGTTCTTCTCTTTGCTGGAAAAGCGTGCGCTGTGCGGTTGTGGAGGCTGCCGTTGTGGCCAGTCGGCGACTGATTTCGTGCTGAGGCGTGTGGAGAGGAGCTCGATTCGCGGTATGGTTGGCTTCGGCTGAACTTCTGCTTATGGGGCTGCTTGTCTGCTGCTTTGGCGTGGCGGGGTCGGTGAACCGTAGTCATTTCGGCTTGGCTTTTGAACTGCGCGGGCGCTGCGCGAAGTATGGTGGCTTCGGCCGAGAGAACGACCTATTTGGGACGATTGTGCGGCTTCGTGCTGCATCTCAA
>HQ261639.1 Phytophthora parsiana
CCACACCTAAAAACTTTCCACGTGAACTGTCTGTGATGTTGGGGGCCTGTCTGCCTGTTGGCGGGCGGCTCCATCAAACGAGGCCCTGGGCTGAAAAGTCGAGGGTAGTAGTTACTTTTTGTAAACCCTTACACAACCTTTCTGATCATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTACGCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGCGGGGACGCGCAGACGTGAAGTGTCTTGCTGGGCTTTGTGGCCGGCGAGTCCTTTGAAATGTAAGATACTGTTCTTCTCTTTGCTGGAAAAGCGTGCGCTGTGCGGTTGTGGAGGCTGCCGTTGTGGCCAGTCGGCGACTGATTTCGTGCTGAGGCGTGTGGAGAGGAGCTCGATTCGCGGTATGGTTGGCTTCGGCTGAACTTCTGCTTATGGGGCTGCTTGTCTGCTGCTTTGGCGTGGCGGGGTCGGTGAACCGTAGTCATTTCGGCTTGGCTTTTGAACTGCGCGGGCGCTGCGCGAAGTATGGTGGCTTCGGCCGAGAGGACGACCTATTTGGGACGATTGTGCGGCTTCGTGCTGCATCTCAA
>HQ261637.1 Phytophthora parsiana
CCACACCTAAAAACTTTCCACGTGAACTGTCTGTGATGTTGGGGGCGTGCCGTTCGCGGCGGCTCCATCAAACGAGGCTCTGGGCTGAAAAGTCGAGGGTAGTAGTTACTTTTTGTAAACCCTTTACCACATTTTCTGATCATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTATGCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGCTTCCTTCCTTCCGTGTAGTCGGTGGCGGGAACGCTCAGACGTGAAGCGTCTTGCTGGCCTTCTGGCCGGCGAGTCCTTTGAAATGTAAGATACTGTTCTTCTCTTTGCTGGAAAAGCGTGCGTTGTGCGGTTGTGGAGGCTGCCGTTATGGCCAGTCGGCGACTGATTTCGTGCTGAGGCGTGTGGAGAGGAGCTCGATTCGCGGTATGGTTGGCTTCGGCTGAACTTCTGCTTATGGGGCTGCTTGTCTGCCGTTTCGGCGTTGCGGGGTCGGTGAACCGTAGTCATGGTGGCTTGGCTTTTGAACCGTGCGATTGCTGCGCGAAGTATGGTGGCTTCGGCCGAGAGGACGACCTATTTGGGACGATTGTGCGGCTTCGTGCTGCATCTCAA
>FJ666127.1 Phytophthora aquimorbida
CCACACCTAAAAACTTTCCACGTGAACTGTCTGTGATGTTGGGGGGCTGCTGCTGCTGCTTCGGTGGCGGCGTGCTCCCATCAAACGAGGCCCTGGGCTGCAAAGTCGGGGGTAGTAGTTACTTTTTGTAAACCCTTTTCCTGTATTTTCTGAATATACTGGGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTATGCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAATCTTGGCTTCCTTCCTTCCGTGTAGTCGGTGGCGGGAACGCGCAGACGTGAAGTGTCTTGCCTGTGGCTCCAGCTGTTGTTGGGGTGGTGTGGGCGAGTCCTTTGAAATGTAAGATACTGTTCTTCTCTTTGCTGGAAAAGCGTGCGCTGTGTGGTTGTGGAGGCTGCCGTGGTGGCCAGTCGGCGACTGACTTCGTGCTGATGCGTGTGGAGAGGCTCTGGATTCGCGGTATGGTTGGCTTCGGCTGAACTTCTGCTTATTTGGGTGTCTTTTCGCTGCGTTGGCGTGTCGGGGTTGGTGAACCGTAGTCATTTCGGCTTGGCTTTTGAACCGCGTGGCTGTAGCGCGAAGTATGGCGGCTGCCTTTGTGGCGGCCGAGAGGACGACCTATTTGGGACGATTGTGCGGCCTCGTGCTGCATCTCAA
>FJ666126.1 Phytophthora taxon aquatilis
CCACACCTAAAAAAACTTTCCACGTGAACCGTATCAACCCTTTTAGTTGGGGGTCTTGCTTGGCGTGCGGCTGCTGGCCTTTATTGGTTAGCTGGCTGCGTGCTGGGCGAGCCCTATCATGGCGATCGTTTGGACTTCGGTCTGAGCTAGTAGCTTTTGTTTTAAACCCATTCTTTAATACTGATTATACTGTGGAGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGAGACGTCAGATGTGAAGTGTCTTGCGGCTGGCCTTCGGGCCGTCTGCGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGCTCGAAAAGCGTGGTGTTGCTGGTTGTGGAGACCGCCTGTGTAGCCAGTCGGCGACCGGTTTGTCTGTTGCAGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGACGCTTATTGGACGTTTTTCCTGCTGTGGCGTGATGGGTTGGTGAACCGTAGCTGTGTTTGGCTTGGTCTTTGAATCGGCTTTGCTGTTGCGAAGTAGAGCGGCGGCTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAGTTTGTGTGTACTTCGGTGCGCATCTCAA
>KC478759.1 Phytophthora taxon castitis
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCTTGATTATTGGGGGCTGCTGACTGGACTGGCTTCGGCTGGACTGGGCGGCGGCTCTAGCGCGACCGTTCTGGGCTTCGGCTTGGGGCTAGTAGCTTTTATTTTAAACCCATTCTTAATTTACTGAAATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTCGTCCTGGAAGTATGCCTGTATCAGTGTCCGTACAATAAACTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGATGCGCAGACGTGAAGTGTCTTGCGACTGGGCTTCGGTCTGGATGCGAGTCCTTTTAAATGTACTAAAACTGTATTCTCTTTGCTCGAAAAGCGTGACGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGTGTTACTGGAGAAGTGTTTTGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGAATGTTTTTCTTACTGCGGCGGATGGACTGGTGAACCGTAGCTGTGCGAGGCTTGGCTTTTGAACAGGCGATATTGTGCGAAGTAGAGTGACGGCTGTTCCGGCGCAAGCTGGGGTGGCTGTCGAGGGTCGATCCATTTGGGAAATGTTGTGCCGTTGCGACCTCGGTTGCTTTGGCATCTCAA
>KC478761.1 Phytophthora dauci
CCACACCTAAAAACTTTCCACGTGAACCGTATCAACCCTTGATTATTGGGGGCTGCTGACTGGACTGGCTTCGGCTGGACTGGGCGGCGGCTCTAGCGCGACCGTTCTGGGCTTCGGCTTGGGGCTAGTAGCTTTCATTTTAAACCCATTCTTAATTTACTGAAATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTCGTCCTGGAAGTATGCCTGTATCAGTGTCCGTACAATAAACTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGATGCGCAGATGTGAAGTGTCTTGCGACTGGGCTTCGGTCTGGATGCGAGTCCTTTTAAATGTACTAAAACTGTATTCTCTTTGCTCGAAAAGCGTGACGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGTGTTACTGGAGAAGTGTTTTGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGAAATGTTTTTCTTACTGCGGCGGATGGACTGGTGAACCGTAGCTGTGCAAGGCTTGGCTTTTGAACCGGCGATATTGTGCGAAGTAGAGTGACGGCTGTTCCGGCGCAAGCTGGGGTGGCTGTCGAGGGTCGATCCATTTGGGAAATGTTGTGCCGTTGTGACCTCGGTTGCTTTGGCATCTCAA
>KC478768.1 Phytophthora lactucae
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCTTGATTATTGGGGGCTGCTGACTGGACTGGCTTCGGCTGGACTGGGCGGCGGCTCTAGCGCGAGCGTTCTGGGCTTCGGCTTGGGGCTAGTAGCTTTTCTTTTAAACCCATTCTTAATTTACTGAAATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTCGTCCTGGAAGTATGCCTGTATCAGTGTCCGTACAATAAACTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGATGCGCAGATGTGAAGTGTCTTGCGACTGGGCTTCGGTCTGGATGCGAGTCCTTTTAAATGTACTAAAACTGTATTCTCTTTGCTCGAAAAGCGTGACGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGTGTTACTGGAGAAGTGTTTTGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGAATGTTTTTCTTACTGCGGCGGATGGACTGGTGAACCGTAGCTGTGCGAGGCTTGGCGTTTGAACCGGCGATATTGTGCGAAGTAGAGTGACGGCTGTTCCGGCGCAAGCTGGGGTGGCTGTCGAGGGTCGATCCATTTGGGAAATGTTGTGCCGTTGCGACCTCGGTTGCTTTGGCATCTCAA
>KC478773.1 Phytophthora cichorii
CCACACCTAAAAAACTTCCCACGTGAACCGTATCAACCCTTTAATATTGGGGGCTGCTGACTGGACTGGCTTCGGCTGGACTGGGTGGCAGCTCTAGCGCGAGCGTTCTGGGCTTCGGCTTGGGGCTAGTAGCTTTTCTTTTAAACCCATTCTTAATTACTGTAATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTCGTCCTGGAAGTATGCCTGTATCAGTGTCCGTACAATAAACTTGCCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGACGTGCAGACGTGAAGTGTCTTGCGACTGGGCTTCGGTCTAGATGCGAGTCCTTTTAAATGTACTAAAACTGTATTCTCTTTGCTCGAAAAGCGTGACGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGTGTTAATGGAGAAGTGTTTTGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGAAATGTTTTTCTTACTGCGGCGGATAGACTGGTGAACCGTAGCTGTGTGGGCTTGGCTTTTGAACCGGCGATATTGTGCGAAGTAGAGTGACGGCTGTTCCGGCGCAAGCTGGAGTGGCTGTCGAGGGTCGATCCATTTGGGAAATGTTGTGCCGTTGCGACCTCGGTTGCTTTGGCATCTCAA
>JX951285.1 Phytophthora acerina
CCACACCTAAAAAACTTTTCACGTGAACCGTATCAACCCTTTTAGTTGGGGGTCTTGCTTTTTTGCGAGCCCTATCATGGCGAATGTTTGGACTTCGGTCTGGGCTAGTAGCTTTTTGTTTTAAACCCATTCTACAATACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGATGTGCCAGATGTGAAGTGTCTTGCGGGTGTCCTTCGGGTCGTCTGCGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGTGTTGCTGGTTGTGGAGGCTGCCTGCGTGGCCAGTCGGCGACCGGTTTGTCTGCTGTGGCGTTTAATGGAGGAGTGTTCRATTCGCGGTATGGTTGGCTTCGGCTGAACAGGCGCTTATTGTATGCTTTTCCTGCTGTGGCGTGATGGGCTGGTGAACCGTAGCTGTGTGTGGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAACTTTGTGTGCACCTCGGTGCGCATCTCAAT
>KF192695.1 Phytophthora stricta
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCACCTAGTTGGGGTTCTCGGTAGCCGTGTGGCATGTGGACTCGTTCTACGTGCTGCTGGCGGCTGAGACCTATCATGGCGAGCGTTTGGGCTTTCGGGCCTGGATTAGTAGCCCTCTTTTAAACTCATTCTGTAAAACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGATGGAGACGCCAGATGTGAAGTGTCTTGCGGCTGTCCGTTTTTCGGTCGACTGCGAGTCCTTTTAAATGTACTGAACTGTATTCTCTTTGCTCGAAAAGCGTGACGTTGCTGGTTGTGGAGGCTGCCCGTGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGTGTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAAACGCTTATTGGGTGCTTTTGCTGCTGTGGCGTGATGGACTGGTGAACCGTAGCTGTGTGAGCTTGGCTTTTGAACTAGTGGTGTAATGCGAAGTAGAGTGGCGTCGTGCGTCTTTATGGCGTGTGTTCGTCGAGGGTCGATCCATTTTGGGAAATGTTGTGTGTGCTTCGGCGCGCATCTCAA
>KF192701.1 Phytophthora macilentosa
CCACACCTAAAAACTTTCCACGTGAACTGTGTTGTCGATGTTGGGGGCTGGGCTGGCGTTTGTTCGTCGGTTGGGCTCCATCAAACGAGGCTCTGGGCTGAAAAGTCGAGGGTAGTAGTTACTTTTTGTAAACCCTTTTTAAGTTTTTCTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTATGCCTGGGAGTATGCCTGTATCAGTGTCCGTACATGAATCTTGTCTTCCTTCCTTCCGTGTAGTCGGTGGCGGGAATGCGCAGACGTGAAGTGTCTTGCTGGTTGGTTCCTTTCGGGGAGGTGTCCGGCGAGTCCTTTGAAATGTAAGACACTGTTCTTCTCTTTGCTGGAAAAGCGTGCGTTGTGTGGTTGTGGAGGCTGCCGTTGTGGCCAGTCGGCGACTGACTTCGTGCTGAGGCGTGTGGAGAGGAGCTCGATTCGCGGTATGGTTGGCTTCGGCTGAACTTCTGCTTATGGAGCTGTTTTCTGGTGTTTCGGCGTGGCGGGGTTAGTGAACCGTAGTCATGGTGGCTTGGCTTTTGAACTGCGCGGCTGTTGCGCGAAGTATGGTGGCTTCGGCCGAGAGGACGACCTATTTGGGACGATTGTGTGGCTTGTTTGCTGCATCTCAA
>JQ029956.1 Phytophthora amnicola
CCACACCTAAAAAAACTTTCCACGTGAACCGTATCAACCCCTTTAATTTGGGGGCTTGTTCGGCGGCGTGCGTGCTGGCCTGTAATGGGTCGGCGTGCTGCTGCTGGGCGGGCTCTATCATGGGCGAGCGTTTGGGCTTCGGCTCGAGCTAGTAGCTATCACTTTTAAACTCCTTTCTTAAAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTAGCGGTCTTTCGAGTCTGCGTGTGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGTCCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCCGTGTGGCCAGTCGGCGACCGGTTTGTTAGCTGTGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGGGTGCTTTTCCTGTCATTGGCGGTACGAACTGGTGAACCGTAGCTATGTGGTGCTTGGCTTTTGAACCGGCTTTGCTTTGCGAAGTAGAGTGACAGTTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAATTTTGTGTGTGCGGCTTCGTGCTGCGTGCATCTCAA
>JQ029958.1 Phytophthora amnicola
CCACACCTAAAAAAACTTTCCACGTGAACCGTATCAACCCCTTTAATTTGGGGGCTTGTTCGGCGGCGTGCGTGCTGGCCTGTAATGGGTCGGCGTGCTGCTGCTGGGCGGGCTCTATCATGGGCGAGCGTTTGGGCTTCGGCTCGAGCTAGTAGCTATCACTTTTAAACTCCTTTCTTAAAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGAGGATATGCCAGACGTGAAGTGTCTTGCTAGCGGTCTTTCGAGTCTGCGTGTGAGTCCTTTGAAATGTACTGAACTGTACTCTCTCTTTGTCCGAAAAGCGTGGCGTTGCTGGTTGTGGAGGCTGCCCGTGTGGCCAGTCGGCGACCGGTTTGTTAGCTGTGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATCTGCTTATTGGGTGCTTTTCCTGTCATTGGCGGTACGAACTGGTGAACCGTAGCTATGTGGTGCTTGGCTTTTGAACCGGCTTTGCTTTGCGAAGTAGGGTGACAGTTTCGGCTGTCGAGGGGTCGATCCATTTTGGGAAACTTTGTGTGTGCGGCTTCGTGCTGCGTGCATCTCAA

>HQ894399.1 Phytophthora pisi
CCACACCTAAAAAACTTTTCCACGTGAACCGTATCAACAAGTAGTTGGGGGTCTGCTCTGTGTGGCTGTCTGTCGATGTCAAAGTCGGCGGCTGGCTGCTGTGTGGCGGGCTCTATCATGGCGATTGGTTTGGGTCCTCCTCGTGGGGATCTGGATCATGAGCCCACTTTTTAAATCCATTCTTAAATACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCTCTTCCTTCCGTGTAGTCGGTGGATGGAGACGCCAGACGTGAGGTGTCTTGCGGCGTGGCCTTCGGGCTGCCTGCGAGTCCCTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGACGCTGTTGGTTGTGGAGGCTGCCTGTATGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGATGCTTTTCCTGCTGTGGCGGTATGGGCTGGTGAACCGTAGCTGTGCGAGGCTTGGCTTTTGAACCGGCGGTGTTGTTGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAACTCTGTGTTGTCTCTGCGGCTCGCTGCGGAGGTGGCATCTCAA
>HQ894391.1 Phytophthora pisi
CCTAAAAAACTTTCCACGTGAACCGTATCAACAAGTAGTTGGGGGTCTGCTCTGTGTGGCTGTCTGTCGATGTCAAAGTCGGCGGCTGGCTGCTGTGTGGCGGGCTCTATCATGGCGATTGGTTTGGGTCCTCCTCGTGGGGATCTGGATCATGAGCCCACTTTTTAAATCCATTCTTAAATACTGAATATACTGTGGGGACGAAAGTCTCTACTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGRAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCTCTTCCTTCCGTGTAGTCGGTGGATGGAGACGCCAGACGTGAGGTGTCTTGCGGCGTGGCCTTCGGGCTGCCTGCGAGTCCCTTGAAATGTACTGAACTGCACTTCTCTTTGCTCGAAAAGCGTGACGCTGTTGGTTGTGGAGGCTGCCTGTATGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGATGCTTTTCCTGCTGTGGCGGTATGGGCTGGTGAACCGTAGCTGTGCGAGGCTTGGCTTTTGAACCGGCGGTGTTGTTGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAACTCTGTGTTGTCTCTGCGGCTCGCTTCGGAGGTGGCATCTCAA
>HQ894394.1 Phytophthora pisi
CCTAAAAAACTTTCCACGTGAACCGTATCAACAAGTAGTTGGGGGTCTGCTCTGTGTGGCTGTCTGTCGATGTCAAAGTCGGCGGCTGGCTGCTGTGTGGCGGGCTCTATCATGGCGATTGGTTTGGGTCCTCCTCGTGGGGATCTGGATCATGAGCCCACTTTTTAAATCCATTCTTAAATACTGAATATACTGTGGGGACGAAAGTCTCTACTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCTCTTCCTTCCGTGTAGTCGGTGGATGGAGACGCCAGACGTGAGGTGTCTTGCGGCGTGGCCTTCGGGCTGCCTGCGAGTCCCTTGAAATGTACTGAACTGCACTTCTCTTTGCTCGAAAAGCGTGACGCTGTTGGTTGTGGAGGCTGCCTGTATGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGATGCTTTTCCTGCTGTGGCGGTATGGGCTGGTGAACCGTAGCTGTGCGAGGCTTGGCTTTTGAACCGGCGGTGTTGTTGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAACTCTGTGTTGTCTCTGCGGCTCGCTTCGGAGGTGGCATCTCAA
>HQ894395.1 Phytophthora pisi
GGGGGTCTGCTCTGTGTGGCTGTCTGTCGATGTCAAAGTCGGCGGCTGGCTGCTGTGTGGCGGGCTCTATCATGGCGATTGGTTTGGGTCCTCCTCGTGGGGATCTGGATCATGAGCCCACTTTTTAAATCCATTCTTAAATACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCTCTTCCTTCCGTGTAGTCGGTGGATGGAGACGCCAGACGTGAGGTGTCTTGCGGCGTGGCCTTCGGGCTGCCTGCGAGTCCCTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGACGCTGTTGGTTGTGGAGGCTGCCTGTATGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGATGCTTTTCCTGCTGTGGCGGTATGGGCTGGTGAACCGTAGCTGTGCGAGGCTTGGCTTTTGAACCGGCGGTGTTGTTGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAACTCTGTGTTGTCTCTGCGGCTCGCTTGCGGAGGTGGCATCTCAA

>HQ261713.1 Phytophthora sp.thermophilum
CCACACCTAAAAACTTTCCACGTGAACTGTCTGTGATGTTGGGGGCCTGTCTGCCTGTTGGCGGGCGGCTCCATCAAACGAGGCCCTGGGCTGAAAAGTCGAGGGTAGTAGTTACTTTTTGTAAACCCTTACGCAACCTTTCTGATCATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATYGAAATTTTGAACGCATATTGCACTTCCGGGTTACGCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGCGGGGACGCGCAGACGTGAAGTGTCTTGCTGGGCTTTGTGGCCGGCGAGTCCTTTGAAATGTAAGATACTGTTCTTCTCTTTGCTGGAAAAGCGTGCGCTGTGCGGTTGTGGAGGCTGCCGTTGTGGCCAGTCGGCGACTGATTTCGTGCTGAGGCGTGTGGAGAGGAGCTCGATTCGCGGTATGGTTGGCTTCGGCTGAACTTCTGCTTATGGGGCTGCTTGTCTGCTGCTTTGGCGTGGCGGGGTCGGTGAACCGTAGTCATTTCGGCTTGGCTTTTGAACTGCGCGGGCGCTGCGCGAAGTATGGTGGCTTCGGCCGAGAGGACGAACCTATTTGGGACGATTGTGCGGCTTCGTGCTGCATCTCAA
>HQ261712.1 Phytophthora sp.thermophilum
CCACACCTAAAAACTTTCCACGTGAACTGTCTGTGATGTTGGGGGCCTGTCTGCCTGTTGGCGGGCGGCTCCATCAAACGAGGCCCTGGGCTGAAAAGTCGAGGGTAGTAGTTACTTTTTGTAAACCCTTACGCATCTTTTCTGATCATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTACGCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGCGGGGACGCGCAGACGTGAAGTGTCTTGCTGGGCTTTGTGGCCGGCGAGTCCTTTGAAATGTAAGATACTGTTCTTCTCTTTGCTGGAAAAGCGTGCGCTGTGCGGTTGTGGAGGCTGCCGTTGTGGCCAGTCGGCGACTGATTTCGTGCTGAGGCGTGTGGAGAGGAGCTCGATTCGCGGTATGGTTGGCTTCGGCTGAACTTCTGCTTATGGGGCTGCTTGTCTGCTGCTTTGGCGTGGCGGGGTCGGTGAACCGTAGTCATTTCGGCTTGGCTTTTGAACTGCGCGGGCGCTGCGCGAAGTATGGTGGCTTCGGCCGAGAGGACGACCTATTTGGGACGATTGTGCGGCTTCGTGCTGCATCTCAA

>HQ261620.1 Phytophthora mexicana
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCTTTTAGTTGGGGGTCTTGTACCCTATCATGGCGAATGTTTGGACTTCGGTCTGGGCGAGTAGCTTTTTGTTTTAAACCCATTTCACAATTCTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGATGTGCCAGATGTGAAGTGTCTTGCGTGTTGTCCTTCGGGTCGGCTGCGAGTCCTTTTAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGTGTTGCTGGTTGTGGAGGCTGCCTGCGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAAAGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGGCGCTTATTGAATGCTTTTCCTGCTGTGGCGTGATGGGCTGGTGAACCGTAGCTGTGTTTGGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTTGGGAACTTTGTGTGCACTTCGGTGCGCATCTCAA

>HQ261710.1 Phytophthora sp.ohioensis
CCACACCTAAAAAAACTTTCCACGTGAACCGTTTCAACCAAATATTTTGGGGGTCTTGTCTGGCGTATGGCTGCTGCTGTAAAAGGCGGCGGCTGTTGCTGGGTGAGCCCTATCATGGCGAACGTTTGGGCTTCGGTCTGAACAAGTAGCTCTTTTTTAAACCATTACTTATTACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGAGACGCCAGACGTGAAGTGTCTTGCGACTGGTTTTCGGACCGGATGCGAGTCCTTTGAAATGTACTTTACTGTACTTCTCTTTGCTCGAAAAGCGTGGCGTTGCTAATTGTGGAGGCTGCCAGTGTGGCCAGTCGGCGACCGGTTTTGTCTGCTGTGGCGTTTAATGGAGGAGTGTTTGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGACGTTTTTCCTGCTGTGGCGTGATGGACTGGTGAACCGTAGCTGTGCATGGCTTGGCCTTTGAATTGGCTTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAAATTTGTGTTGGTAGCTTCGGTTGCTGGCATCTCAA

>KC295542.1 Phytophthora virginiana
CCACACCTAAAAACTTTCCACGTGAACTGTCTGTGATGTTGGGGGCGTGCCGTTCGCGGCGGCTCCATCAAACGAGGCTCTGGGCTGAAAAGTCGAGGGTAGTAGTTACTTTTTGTAAACCCTTTACCACATTTTCTGATCATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTATGCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGCTTCCTTCCTTCCGTGTAGTCGGTGGCGGGAACGCTCAGACGTGAAGCGTCTTGCTGGCTGTAAGGCCGGCGAGTCCTTTGAAATGTAAGATACTGTTCTTCTCTTTGCTGGAAAAGCGTGCGTTGTGCGGTTGTGGAGGCTGCCGTTATGGCCAGTCGGCGACTGATTTCGTGCTGAGGCGTGTGGAGAGGAGCTCGATTCGCGGTATGGTTGGCTTCGGCTGAACTTCTGCTTATGGGGCTGCTTGTCTGCTGTTTCGGCGTAGCGGGGTCGGTGAACCGTAGTCATGGTGGCTTGGCTTTTGAACCGTGCGACTGTTGCGCGAAGTATGGTGGCTTCGGCCGAGAGGACGACCTATTTGGGACGATTGTGCGGCTTCGTGCTGCATCTCAA

>KC249960.1 Phytophthora hydrogena
CCACACCTAAAAACTTTCCACGTGAACTGTCTGTGATGTTGGGGGCTGCCCTTGGTGGTGGCTCCATCAAACGAGGCCCTGGGCTGCAAAGTCGAGGGTAGTAGTTACTTGTTGTAAACCCTTTTGTAATTTTTCTGATCATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTATGCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAACCTTGGCTTCCTTCCTTCCGTGTAGTCGGTGGCGGGAACGCGCAGACGTGAAGTGTCTTGCTGGGCGTGAGTCCGGCGAGTCCTTTGAAATGTAAGATACTGTTCTTCTCTTTGCTGGAAAAGCGTGCGTTGTGTGGTTGTGGAGGCTGCCGTTGTGGCCAGTCGGCGACTGATTTCGTGCTGAGGCGTGTGGGGAGGAGCTCGATTCGCGGTATGGTTGGCTTCGGCTGAACTTCTGCTTATGGGGCTGTTTTTCTGCTGTTTCGGCGTTGCGGGGTCGGTGAACCGTAGTCATGGTGGCTTGGCTTTTGAACCGCGCGGCTGTTGCGCGAAGTATGGTGGCTTCGGCCGAGAGGACGACCTATTTGGGACGATTGTGCGGCTTCGTGCTGCATCTCAA

>HQ643189.1 Phytophthora cinnamomi
CCACACTAAAAAACTTTCCACGTGAACCGTATCAACCCAATTAGTTGGGGGCCTGCTCTGGGCGGCGGCTGTCGATGTCAAAGTCGACGGCTGCTGCTGCGTGGCGGGCCCTATCACTGGCGAGCGTTTGGGTCCCTCTCGGGGGAACTGAGCTAGTAGCCTCTCTTTTAAACCCATTCTGTAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCTCTTCCTTCCGTGTAGTCGGTGGATGGAGGTGCCAGACGTGAGGTGTCTTGCGGGCGGTCTTCGGACTGGCTGTGAGTCCCTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGCTGGTTGTGGAGGCTGCCTGTATGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAAAGCGCTTATTGGATGTTCTTCCTGCTGTGGCGGTACGGATCGGTGAACCGTAGCTGTGCTAGGCTTGGCGTTTGAACCGGCGGTGTTGTTGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGTCTATCCATTTGGGAACTCTGTGTCTCTGCGGCGCACTTGTGTGCTTGTGGTGGCATCTCAA
>HQ261526.1 Phytophthora cinnamomi
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCAATTAGTTGGGGGCCTGCTCTGGGCGGCGGCTGTCGATGTCAAAGTCGACGGCTGCTGCTGCGTGGCGGGCCCTATCACTGGCGAGCGTTTGGGTCCCTCTCGGGGGAACTGAGCTAGTAGCCTCTCTTTTAAACCCATTCTGTAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCTCTTCCTTCCGTGTAGTCGGTGGATGGAGGTGCCAGACGTGAGGTGTCTTGCGGGCGGTCTTCGGACTGGCTGTGAGTCCCTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGCTGGTTGTGGAGGCTGCCTGTATGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAAAGCGCTTATTGGATGTTCTTCCTGCTGTGGCGGTACGGATCGGTGAACCGTAGCTGTGCTAGGCTTGGCGTTTGAACCGGCGGTGTTGTTGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAACTCTGTGTCTCTGCGGCGCACTTGTGTGCTTGTGGTGGCATCTCAA
>HQ643193.1 Phytophthora cinnamomi
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCAATTAGTTGGGGGCCTGCTCTGGGCGGCGGCTGTCGATGTCAAAGTCGACGGCTGCTGCTGCGTGGCGGGCCCTATCACTGGCGAGCGTTTGGGTCCCTCTCGGGGGAACTGAGCTAGTAGCCTCTCTTTTAAACCCATTCTGTAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTARTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCTCTTCCTTCCGTGTAGTCGGTGGATGGAGGTGCCAGACGTGAGGTGTCTTGCGGGCGGTCTTCGGACTGGCTGTGAGTCCCTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGCTGGTTGTGGAGGCTGCCTGTATGGCCAGTCGGCGATCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAAAGCGCTTATTGGATGTTCTTCCTGCTGTGGCGGTACGGATCGGTGAACCGTAGCTGTGCTAGGCTTGGCGTTTGAACCGGCGGTGTTGTTGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAACTCTGTGTCTCTGCGGCGCACTTGTGTGCTTGTGGTGGCAT

>AB305065.1 Phytophthora fragaefolia
TTTCCGTAGGTGAACCTGCGGAAGGATCATTACCACACCTAAAAAACTGTCCACGTGAACCGTATCAACCCAAGTAGTTGGGGGCGTTACTGTTCTGCGCAGGCTGTCGATGTGAAAGTCGGCGGCTGGTGCGGAGCGGTGATGCCCTATCATGGCGAAGGTGTTGTGTTCCTCTCGGGGGACGTGATGTCGGCTGCCCTCTGTTTTAAACCCATTCCTAATAACTGATTAATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCTCTTCCTTCCGTGTAGTCGGTGGATGGAGACGCCAGACGTGAGGTGTCTTACGTGTGGTCTTTGTGGCTGCGTGTGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGCGACGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACCGGTCTGTCTGCTGTGGCATTTAAAGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTAAACAAAACGCTTATTGGATGCTTTTTCTGCTATGGCGGTACGGATCTGGTGAACCGTAGCTATTCTAGGCTTGGCGTTTGAACCGGTTTTGTTGTTTTGCGAAGTAGAGCGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAACTTGTGTCTGGTTGGCGCCTTCGGGTGCTACCGTGGCATCTCAATTGGACCTGATATCAGGCAAGATTACCCGCTGAACTTAA

>AB688355.1 Phytophthora nagaii
CCACACCTAAAAAACTGTCCACGTGAACCGTATCAACCCAAGTAGTTGGGGGCGTTACTGTTCTGCGCAGGCTGTCGATGTGAAAGTCGGCGGCTGGTGCGGGGCGGTGATGCCCTATCATGGCGAAGGTGTTGTGTCCCTCTCGGGGGACGTGATGTCGGCTGCCCTCTGTTTTAAACCCATTCCTAATAACTGATTAATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCTCTTCCTTCCGTGTAGTCGGTGGATGGAGACGCCAGACGTGAGGTGTCTTACGTGTGGTCTTTGTGGCTGCGTGTGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGCGACGTTACTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACCGGTCTGTCTGCTGTGGCATTTAAAGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTAAACAAAACGCTTATTGGATGCTTTTTCTGCTATGGCGGTACGGATCTGGTGAACCGTAGCTATTCTAGGCTTGGCGTTTGAACCGGTTTTGTTGTTTTGCGAAGTAGAGCGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAACTTGTGTCTGGTTGGCGCCTTCGGGTGCTACCGTGGCATCTCAA

>JX978155.1 Phytophthora occultans
CCACACCTAAAAAAACTTTCCACGTGAACCGTATCAACCCTTTTAGTTGGGGGTGTTGCTTGGCGTTTTGCTGAGTCACGCCCTATCATGGCGAATGTTTGGACTTCGGTCTGGGCTAGTAGCTTTTTTGTTTTAAACCCATTTAACAATACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGATGTGCCAGATGTGAAGTGTCTTGCGGTTTTTGTGCCTTCGGGCCGTGGCTGCGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGTGTTGCTGGTTGTGGAGGCTGCCTGCGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGGCGCTTATTGTATGCTTTTCCTGCTGTGGCGTGATGGGCTGGTGAACCGTAGCTGTGTGTGGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGTGTCGATCCATTTTGGGAACGTTTGTGTGCGCTTTCGAGTGTGCATCTCAA
>JX978167.1 Phytophthora terminalis
CCACACCTAAAAAAACTTTCCACGTGAACCGTATCAACCCTTTTAGTTGGGGGTGTTGCTTGGCGTTTTGCTGAGCCACGCCCTATCATGGCGAATGTTTGGACTTCGGTCTGGGCTAGTAGCTTTTTTGTTTTAAACCCATTTAACAATACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGATGTGCCAGATGTGAAGTGTCTTGCGGTTTTTGTGCCTTCGGGCCGTGGCTACGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGTGTTGCTGGTTGTGGAGGCTGCCTGCGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGAAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGGCGCTTATTGTATGCTTTTCCTGCTGTGGCGTGATGGGCTGGTGAACCGTAGCTGTGTGTGGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGTGTCGATCCATTTTGGGAACGTTTGTGTGCGCTTTCGAGTGTGCATCTCAA
>HQ261491.1 Phytophthora aff. infestans
CCACACCTAAAAACTTTCCACGTGAACCGTTTCAACCCAATAGTTGGGGGTCTTACTTGGCGGCGGCTGCTGGCTTTATTGCTGGCGGCTACTGCTGGGCGAGCCCTATCAAAAGGCGAGCGTTTGGACTTCGGTCTGAGCTAGTAGCTTTTTTATTTTAAACCCTTTACTTAATACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGAAGTATGCCTGTATCAGTGTCCGTACAACAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGAGATGCCAGATGTGAAGTGTCTTGCGGTTGGTTTTCGGACCGACTGCGAGTCCTTTTAAATGTACTAAACTGTACTTCTCTTTGCTCCAAAAGTGGTGGCATTGCTGGTTGTGGACGCTGCTATTGTAGCGAGTTGGCGACCGGTTTGTCTGCTGCGGCGTTAATGGAGAAATGCTCGATTCGTGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGGTGATTTTCCTGCTGTGGCGTGATGGACTGGTGAACCATGGCTCTTTAGCTTGGCATTTGAATCGGC-TTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGCCGAGGGTCGATCCATTTGGGAAATGTTGTGTACTTCGGTATGCATCTCAA
>HQ261588.1 Phytophthora infestans
CCACACCTAAAAACTTTCCACGTGAACCGTTTCAACCCAATAGTTGGGGGTCTTACTTGGCGGCGGCTGCTGGCTTTATTGCTGGCGGCTACTGCTGGGCGAGCCCTATCAAAAGGCGAGCGTTTGGACTTCGGTCTGAGCTAGTAGCTTTTTTATTTTAAACCCTTTACTTAATACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGAAGTATGCCTGTATCAGTGTCCGTACAACAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGAGATGCCAGATGTGAAGTGTCTTGCGGTTGGTTTTCGGACCGACTGCGAGTCCTTTTAAATGTACTAAACTGTACTTCTCTTTGCTCCAAAAGTGGTGGCATTGCTGGTTGTGGACGCTGCTATTGTAGCGAGTTGGCGACCGGTTTGTCTGCTGCGGCGTTAATGGAGAAATGCTCGATTCGTGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGGTGATTTTCCTGCTGTGGCGTGATGGACTGGTGAACCATGGCTCTTTAGCTTGGCATTTGAATCGGC-TTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGCCGAGGGTCGATCCATTTGGGAAATGTTGTGTACTTCGGTATGCATCTCAA
>GU191228.1 Phytophthora sp. emzansi/CCB-2010
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCTTTTAGTTGGGGGTCTTGCTTTTTTGCGAGCCCTATCATGGCGAATGTTTGGACTTCGGTCTGGGCTAGTAGCTTTTTGTTTTAAACCCATTCAACAATACTGATTATACTGCGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGATGTGCCAGATGTGAAGTGTCTTGCGGGTGTCCTTCGGGCCGCCTGCGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGTGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGATCAGTTTGTCTGCTGTGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGGCGCTTATTGTATGCTTTTCCTGCTGTGGCGTGATGGGCTGGTGAACCGTAGCTATGTGTGGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAATTTTGTGTGCACCTCGGTGCGCATCTCAA
>FJ560913.1 Phytophthora citricola sensu stricto
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCTTTTAGTTGGGGGTCTTGCTTTTTTTGCGAGCCCTATCATGGCGAATGTTTGGACTTCGGTCTGGGCTAGTAGCTTTTTGTTTTAAACCCATTCTACAATACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGATGTGCCAGATGTGAAGTGTCTTGCGGGTGTCCTTCGGGTCGTCTGCGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGTGTTGCTGGTTGTGGAGGCTGCCTGCGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGGCGCTTATTGTATGCTTTTCCTGCTGTGGCGTGATGGGCTGGTGAACCGTAGCTGTGTGTGGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAACTTTGTGTGCACCTCGGTGCGCATCTCAA
>AF266788.1 Phytophthora citricola E
CCACACCTAAAAAACTTTTCACGTGAACCGTATCAACCCTTTTAGTTGGGGGTCTTGCTTTTTTGCGAGCCCTATCATGGCGAATGTTTGGACTTCGGTCTGGGCTAGTAGCTTTTTGTTTTAAACCCATTCTACAATACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGATGTGCCAGATGTGAAGTGTCTTGCGGGTGTCCTTCGGGTCGTCTGCGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGTGTTGCTGGTTGTGGAGGCTGCCTGCGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCAGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGGCGCTTATTGTATGCTTTTCCTGCTGTGGCGTGATGGGCTGGTGAACCGTAGCTATGTGTGGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAACTTTGTGTGCACCTCGGTGCGCATCTCAA
>AJ007370.1 Phytophthora plurivora
TCCGTAGGTGAACCTGCGGAAGGATCATTACCACACCTAAAAAACTTTTCACGTGAACCGTATCAACCCTTTTAGTTGGGGGTCTTGCTTTTTTTGCGAGCCCTATCATGGCGAATGTTTGGACTTCGGTCTGGGCTAGTAGCTTTTTGTTTTAAACCCATTCTACAATACTGATTATACTGCGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATCGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGATGTGCCAGACGTGAAGTGTCTTGCGGGTGTCCTTCGGGTCGTCTGCGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGGTGTTGCTGGTTGTGGAGGCTGCCTGCGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGGCGCTTATTGTATGCTTTTCCTGCTGTGGCGTGATGGGCTGGTGAACCGTAGCTGTGTGTGGCTTGGCTTTTGAATCGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAACTTTGTGTGCACCTCGGTGCGCATCTCAATTGGACCTGATATCAGGCAAGATTACC
>AB688422.1 Phytophthora asiatica (cinnamomi var. robiniae)
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACAAGTAGTTGGGGGCCTGCTCTGAGTGGCTGTCTGTCGATGTCAAAGTCGGCGGCTGGCTGCTGTGTGGCGGGCTCTATCATGGCGATTGGTTTGGGTCCTCCTCGTGGGGAACTGGATCATGAGCCCACTTTTTAAACCCATTCTTAAATACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCTCTTCCTTCCGTGTAGTCGGTGGATGGAGACGCCAGACGTGAGGTGTCTTGCGGCGTGGCCTTCGGGCTGCCTGCGAGTCCCTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGTTGGTTGTGGAGGCTGCCTGTATGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGATGCTTTTCCTGCTGTGGCGGTATGGGCTGGTGAACCGTAGCTGTGCGAGGCTTGGCTTTTGAACCGGCGGTGTTGTTGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAACTCTGTGTTGTCTCTGCGGCTCGCTGCGGAGGTGGCATCTCAA
>AB538396.1 Phytophthora asiatica (cinnamomi var. robiniae)
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACAAGTAGTTGGGGGCCTGCTCTGTGTGGCTGTCTGTCGATGTCAAAGTCGGCGGCTGGCTGCTGTGTGGCGGGCTCTATCATGGCGATTGGTTTGGGTCCTCCTCGTGGGGAACTGGATCATGAGCCCACTTTTTAAACCCATTCTTAAATACTGAATATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCTCTTCCTTCCGTGTAGTCGGTGGATGGAGACGCCAGACGTGAGGTGTCTTGCGGCGTGGCCTTCGGGCTGCCTGCGAGTCCCTTGAAATGTACTGAACTGTACTTCTCTTTGCTTGAAAAGCGTGACGTTGTTGGTTGTGGAGGCTGCCTGTATGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGATGCTTTTCCTGCTGTGGCGGTATGGGCTGGTGAACCGTAGCTGTGCGAGGCTTGGCTTTTGAACCGGCGGTGTTGTTGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAACTCTGTGTTGTCTCTGCGGCTCGCTGCGGAGGTGGCATCTCAA


>KC478663.1 Phytophthora cinnamomi
CATTACCACACTAAAAAACTTTCCACGTGAACCGTATCAACCCAATTAGTTGGGGGCCTGCTCTGGGCGGCGGCTGTCGATGTCAAAGTCGACGGCTGCTGCTGCGTGGCGGGCCCTATCACTGGCGAGCGTTTGGGTCCCTCTCGGGGGAACTGAGCTAGTAGCCTCTCTTTTAAACCCATTCTGTAATACTGAACATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGCTCTCTTCCTTCCGTGTAGTCGGTGGATGGAGGTGCCAGACGTGAGGTGTCTTGCGGGCGGTCTTCGGACTGGCTGTGAGTCCCTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCGTGACGTTGCTGGTTGTGGAGGCTGCCTGTATGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGCGTTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAAAGCGCTTATTGGATGTTCTTCCTGCTGTGGCGGTACGGATCGGTGAACCGTAGCTGTGCTAGGCTTGGCGTTTGAACCGGCGGTGTTGTTGCGAAGTAGGGTGGCGGCTTCGGCTGTCGAGGGTCTATCCATTTGGGAACTCTGTGTCTCTGCGGCGCACTTGTGTGCTTGTGGTGGCATCTCAA

>KP295324.1 Phytophthora castaneae
ATAGTTGGGGGCGAGTTTGGCGGCGGCTGCTGGCTTTGGCTGGCTGGCTGCTGCTGGACGAGCTCTATCATGGCGAGCGTTTGGACTTCGGTCTGAACTAGTAGCCCTTCTTTTAAACCCATTCCTAATTACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACAACAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGAGATGCCAGATGTGAAGTGTCTTGCGGCTGGTTCTCGAACTGGCTGCGAGTCCTTTGAAATGTACTGAACTGTACTCCTCTTTGCTCGAAAAGCATGGCGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCATGTCGGCGACCAGTCTGTCTGCTGCGGCGTTAATGGAGGCGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGATGTGTTTCCTGCTGTGGCGGTAATGGCTGGTGTACCGTAGCTATGTGGTGCTTGGCTTTTGAATCGGCTTTGCTGTTGTGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAAATTTGTGTGTGCCTTTATTGGCGTGCATCTCAATTGGACC
>KP295323.1 Phytophthora castaneae
ATAGTTGGGGGCGAGTTTGGCGGCGGCTGCTGGCTTTGGCTGGCTGGCTGCTGCTGGACGAGCTCTATCATGGCGAGCGTTTGGACTTCGGTCTGAACTAGTAGCCCTTCTTTTAAACCCATTCCTAATTACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACAACAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGAGATGCCAGATGTGAAGTGTCTTGCGGCTGGTTCTCGAACTGGCTGCGAGTCCTTTGAAATGTACTGAACTGTACTCCTCTTTGCTCGAAAAGCATGGCGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCATGTCGGCGACCGGTCTGTCTGCTGCGGCGTTAATGGAGGCGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGATGTGTTTCCTGCTGTGGCGGTAATGGCTGGTGTACCGTAGCTATGTGGTGCTTGGCTTTTGAATCGGCTTTGCTGTTGTGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAAATTTGTGTGTGCCTTTATTGGCGTGCATCTCAATTGGACC
>KP295314.1 Phytophthora agathidicida
CATTACCACACCTAAAAAACTTTCCACGTGAACCGTAACAAACCAATAGTTGGGGGCGAGTTTGGCGGCGGCTGCTGGCTTTGGCTGGCTGGCTGCTGCTGGACGAGCTCTATCATGGCGAGCGTTTGGACTTCGGTCTGAACTAGTAGCCCTTCTTTTAAACCCATTCCTAATTACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACAACAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGAGATGCCAGATGTGAAGTGTCTTGCGGCTGGTTCTCGAACTGGCTGCGAGTCCTTTGAAATGTACTGAACTGTACTCCTCTTTGCTCGAAAAGCATGGCGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCATGTCGGCGACCGGTCTGTCTGCTGCGGCGTTAATGGAGGCGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGATGTGTTTCCTGCTGTGGCGGTAATGGCTGGTGTACCGTAGCTATGTGGTGCTTGGCTTTTGAATCGGCTTTGCTGTTGTGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAAATTTGTGTGTGCCTTTATTGGCGT
>KP295306.1 Phytophthora cocois
CATTACCACACCTAAAAAACTTTCCACGTGAACCGTAACAAACCAATAGTTGGGGGCGAGTTTGGCGGCGGCTGCTGGCTTTGGCTGGCTGGCTGCTGCTGGACGAGCTCTATCATGGCGAGCGTTTGGACTTCGGTCTGAACTAGTAGCCCTTCTTTTAAACCCATTTCTAATTACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACAACAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGAGATGCCAGATGTGAAGTGTCTTGCGGCTGGTTCTCGAACTGGCTGCGAGTCCTTTGAAATGTACTGAACTGTACTCCTCTTTGCTCAAAAAGCATGGCGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCATGTCGGCGACCGGTCTGTCTGCTGCGGCGTTAATGGAGGCGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGATGTGTTTCCTGCTGTGGCGGTAATGGCTGGTGTACCGTAGCTATGTGGCGCTTGGCTTTTGAATCGGCTTTGCTGTTGTGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAAATTTGTGTGTGCCTTTATTGGCGTGCATCTCAATTGGACC
>AB894385.1 Phytophthora pseudolactucae
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCTTGATTATTGGGGGCTGCTGACTGGACTGGCTTCGGCTGGACTGGGCGGCGGCTCTAGCGCGAGCGTTCTGGGCTTCGGCTTGGGGCTAGTAGCTTTTCTTTTAAACCCATTCTTAATTTACTGAAATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTCGTCCTGGAAGTATGCCTGTATCAGTGTCCGTACAATAAACTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGATGCGCAGATGTGAAGTGTCTTGCGACTGGGCTTCGGTCTGGATGCGAGTCCTTTTAAATGTACTAAAACTGTATTCTCTTTGCTCGAAAAGCGTGACGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGTGTTACTGGAGAAGTGTTTTGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGAATGTTTTTCTTACTGCGGCGGATGGACTGGTGAACCGTAGCTGTGCGAGGCTTGGCGTTTGAACCGGCGATATTGTGCGAAGTAGAGTGACGGCTGTTCCGGCGCAAGCTGGGGTGGCTGTCGAGGGTCGATCCATTTGGGAAATGTTGTGCCGTTGCGACCTCGGTTGCTTTGGCATCTCAA
>AB894384.1 Phytophthora pseudolactucae
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAACCCTTGATTATTGGGGGCTGCTGACTGGACTGGCTTCGGCTGGACTGGGCGGCGGCTCCAGCGCGAGCGTTCTGGGCTTCGGCTTGGGGCTAGTAGCTTTTCTTTTAAACCCATTCTTAATTTACTGAAATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTCGTCCTGGAAGTATGCCTGTATCAGTGTCCGTACAATAAACTTGGCTCCCTTCCTTCCGTGTAGTCGGTGGATGGGGATGCGCAGATGTGAAGTGTCTTGCGACTGGGCTTCGGTCTGGATGCGAGTCCTTTTAAATGTACTAAAACTGTATTCTCTTTGCTCGAAAAGCGTGACGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCCAGTCGGCGACCGGTTTGTCTGCTGCGGTGTTACTGGAGAAGTGTTTTGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGAATGTTTTTCTTACTGCGGCGGATGGACTGGTGAACCGTAGCTGTGCGAGGCTTGGCGTTTGAACCGGCGATATTGTGCGAAGTAGAGTGACGGCTGTTCCGGCGCAAGCTGGGGTGGCTGTCGAGGGTCGATCCATTTGGGAAATGTTGTGCCGTTGCGACCTCGGTTGCTTTGGCATCTCAA
>KP295301.1 Phytophthora heveae
CATTACCACACCTAAAAAACTTTCCACGTGAACCGTAACAAACCAATAGTTGGGGGCGAGTTTGGCGGCTGCTGCTGCTGGACGAGCTCTATCATGGCGAGCGTTTGGACTTCGGTCTGAACTAGTAGCCCTTCTTTTAAACCCATTCCTAATTACTGATTATACTGTGGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACAACAAACTTGGCTTTCTTCCTTCCGTGTAGTCGGTGGAGGAGATGCCAGATGTGAAGTGTCTTGCGGCTGGTTCTCGAACTGGCTGCGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCGAAAAGCATGGCGTTGCTGGTTGTGGAGGCTGCCTGTGTGGCATGTCGGCGACCGGTCTGTCTGCTGCAGCGTTAATGGAGGCGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAATGCGCTTATTGGATGTGTTTCCTGTTGTGGCGGTAATGGCTGGTGTACCGTAGCTATGTGGTGCTTGGCTTTTGAATCGGCTTTGCTGTTGTGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAAATTTGTGTGTGCCTTTATTGGCGTGCATCTCAATTGGACC
>AF266780 Phytophthora palmivora
CCACACCTAAAAAACTTTCCACGTGAACCGTATCAAAACTTATTTGGGGGTCTCTTTCGGCGGCGGCTGCTGGCTTCATTGCTGGCGGCTGCTGTTGGGAGAGCTCTATCATGGCGAGCGTTTGGGCTTCGGTCTGAACTAGTAGCTTTTTTAAACCCATTCTTTATAACTGATTATACTGTAGGGACGAAAGTCTCTGCTTTTAACTAGATAGCAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGAATTGCAGGATTCAGTGAGTCATCGAAATTTTGAACGCATATTGCACTTCCGGGTTAGTCCTGGGAGTATGCCTGTATCAGTGTCCGTACATCAAACTTGGTTTTCTTCCTTCCGTGTAGTCGGTGGTGGATGTGCCAGATGTGAAGTGTCTTGCGGCTGGTCTTCGGATCGGCTGTGAGTCCTTTGAAATGTACTGAACTGTACTTCTCTTTGCTCCAAAAGCGTGGCGTTGCTGATTGTGGAGGCTGCTTGCGTAGCCAGTCTGGCGACCAGTTTGTCTGCTGTGGCGTTAATGGAGGAGTGTTCGATTCGCGGTATGGTTGGCTTCGGCTGAACAGACGCTTATTGAATATTTCTTCAGCTGTGGTGGTATGATTGGTGAACCGTAGCTATGTGAGCTTGGCTTTTGAATTGGCTTTGCTGTTGCGAAGTAGAGTGGCGGCTTCGGCTGTCGAGGGTCGATCCATTTGGGAACTTGTGTATGCTTCGGCATGCATCTCAA



"""

# Función para convertir FASTA a JSON
def fasta_to_json(fasta_input):
    sequences = {}
    current_sequence_name = None
    current_sequence = ""

    for line in fasta_input.splitlines():
        if line.startswith(">"):
            if current_sequence_name:
                sequences[current_sequence_name] = current_sequence
            current_sequence_name = line[1:]
            current_sequence = ""
        else:
            current_sequence += line

    if current_sequence_name:
        sequences[current_sequence_name] = current_sequence

    return json.dumps(sequences, indent=4)

# Convertir FASTA a JSON
json_output = fasta_to_json(fasta_input)



#print(json_output)



In [ ]:
print(json_output)

{'sequences': ['ACACACCTAAAAACACCCTTCCACGTGAACCGTTTTGTTTTGCTTTCGAGTGCTTTGTTGCGCTCGGAGCATGTTTGGGCTTCGCTGCTGGCGCTTGATTGTGCTGGCGGCTCGAGGCCATCAAGCGGCGTTTTGAGTGTGCTTGCGCAATTGAAACGTCCAAACCTTTTAAACCCATTTGATTGCAAACTGAAGTATACTGTGGGGACGAAAGTCCTCGCTTTGAAACTAGATAACAACTTTCAGCAGTGGATGTCTAGGCTCGCACATCGATGAAGAACGCTGCGAACTGCGATACGTAATGCGGATTGCAGGATTCAGTGAGTCATCGAACTTTTGAACGCATATTGCACTTTCGGGTTACGCCTGGAAGTATGTCTGTATCAGTGTCCGTACACTAAACTTGCGTCTCTTCCGTCGTGTAGTCGTCGGTTGTTTAGATTGCAGATGTGAGGTTGTCTCGCAACGTGCCCCCCCTTGTTTTTTTGGGGGGTGGATAGGTTGCGAGTCCCTTTAAAGTCGGACGGTGTTTTTTTCCGTTTTGTGCTCGATGGGGTGCGGCTGCGGCCGTGTCTGCTGGCGGGTCCGGTGACCTTTGGCGATGGCATGAGAGTGGATTGCTCGATTTGCGGTATGTTAGGCTTCGGCTTTGACAATGCAGCTTATTGGGTGTGTTCGCTTGGCTGTTGCTGTATGGGGTGAGCTGGATGGTCGGTGGGTGCGTTGTTGCGTGTCGTTTTCATTGTGTGCGTTTCGGTTGTCGTCGCCATTTGGGAATTTAATGTTTTGGGGGACCCCTCAATACATCTCAT'], 'pathogens': ['Pythium vexans', 'Phytophthora infestans', 'Alternaria solani']}


# Prepara los datos para el entrenamiento.

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# Carga el JSON en un objeto Python
data = json.loads(json_output)

# Convierte el diccionario en un array de secuencias de ADN
sequences = list(data.values())


# Limpieza de datos
# Elimina las secuencias que contengan caracteres no válidos
sequences = [seq for seq in sequences if seq and all(c.isalnum() for c in seq)]

# Elimina las secuencias que sean demasiado cortas o demasiado largas
sequences = [seq for seq in sequences if len(seq) >= 100 and len(seq) <= 500]

# Normalización de datos
# Escalar las secuencias de ADN a un rango común
if len(sequences) > 0:
    sequences = sequences / sequences.max()
else:
    # El array de secuencias de ADN está vacío, por lo que no es posible escalarlas
    pass

# Codificación de categorías
# Convertir los nombres de los patógenos en valores numéricos
pathogen_encoder = pd.get_dummies(pathogen_names)
pathogen_labels = pathogen_encoder.values

# División de datos
# Verifica si el array de secuencias de ADN está vacío
if len(sequences) > 0:
    # Divide los datos en dos conjuntos: un conjunto de entrenamiento y un conjunto de prueba
    X_train, X_test, y_train, y_test = train_test_split(sequences, pathogen_labels, test_size=0.25, random_state=42)
else:
    # El array de secuencias de ADN está vacío, por lo que no es posible dividir los datos
    X_train = np.array([])
    X_test = np.array([])
    y_train = np.array([])
    y_test = np.array([])

# Los datos ya están preparados para ser utilizados para entrenar un modelo de aprendizaje automático

print(sequences)


NameError: ignored

# Crea el modelo de aprendizaje automático.

In [ ]:
import keras

# Crea una red neuronal artificial con dos capas
model = keras.Sequential([
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(len(pathogen_names), activation="softmax")
])

# Compila el modelo
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Verifica si los arrays de entrenamiento están vacíos
if len(X_train) > 0:
    # Entrena el modelo
    model.fit(X_train, y_train, epochs=10)

    # Imprime la precisión del modelo
    score = model.evaluate(X_test, y_test, verbose=0)
    print("Precisión:", score[1])

    # Imprime la matriz de confusión del modelo
    confusion_matrix = model.evaluate(X_test, y_test, verbose=0)[1]
    print(confusion_matrix)
else:
    # Los arrays de entrenamiento están vacíos, por lo que no es posible entrenar el modelo
    pass



# Entrena el modelo.

In [ ]:
import keras

# Crea una red neuronal artificial con dos capas
model = keras.Sequential([
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(len(pathogen_names), activation="softmax")
])

# Compila el modelo
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Verifica si los arrays de entrenamiento están vacíos
if len(X_train) > 0:
    # Entrena el modelo
    model.fit(X_train, y_train, epochs=10)
else:
    # Los arrays de entrenamiento están vacíos, por lo que no es posible entrenar el modelo
    print("vacio")
    pass


vacio


# Evalúa el modelo.

In [ ]:
import keras

# Crea una red neuronal artificial con dos capas
model = keras.Sequential([
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(len(pathogen_names), activation="softmax")
])

# Compila el modelo
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Verifica si los arrays de entrenamiento están vacíos
if len(X_train) > 0:
    # Entrena el modelo
    model.fit(X_train, y_train, epochs=10)

    # Evalúa el modelo en el conjunto de datos de prueba
    score = model.evaluate(X_test, y_test, verbose=0)
    print("Precisión:", score[1])

else:
    # Los arrays de entrenamiento están vacíos, por lo que no es posible evaluar el modelo
    print("vacio")
    pass


vacio
